# EDA — Proyecto Asistente Coreográfico
Celdas extraídas automáticamente del notebook original.


## 🔧 0. Configuración para Colab (rutas, utilidades)
Ejecuta estas celdas **tal cual** en Google Colab. Ajusta las rutas solo si tus archivos están en otra carpeta.


In [ ]:

# Si estás en Colab, descomenta estas dos líneas para montar Drive
# from google.colab import drive
# drive.mount('/content/drive')

import os, glob, json, math
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

pd.set_option("display.max_columns", 120)
pd.set_option("display.width", 160)

def safe_read_csv(path, **kwargs):
    try:
        return pd.read_csv(path, **kwargs)
    except Exception as e:
        print(f"⚠️ No se pudo leer {path}: {e}")
        return None



## 📂 1. Rutas habituales del proyecto
Se buscarán **features** en `asistente_coreografico` dentro de tu Google Drive. Cambia `CANDIDATES` si es necesario.


In [ ]:

CANDIDATES = [
    "/content/drive/MyDrive/asistente_coreografico/artifacts/features_coreograficos.csv",
    "/content/drive/MyDrive/asistente_coreografico/data/features_coreograficos.csv",
    "/content/drive/MyDrive/asistente_coreografico/data/features/*.csv",
    "/content/drive/MyDrive/asistente_coreografico/artifacts/*.parquet",
    "/content/drive/MyDrive/asistente_coreografico/artifacts/features/*.parquet",
]

csv_paths = []
for pat in CANDIDATES:
    csv_paths.extend(glob.glob(pat))

print("Posibles archivos encontrados:", len(csv_paths))
for p in csv_paths[:10]:
    print(" -", p)

FEATURES_PATH = csv_paths[0] if csv_paths else None
FEATURES_PATH



## 📥 2. Carga de datos de features
Detecta de forma automática si el archivo es `.csv` o `.parquet`.


In [ ]:

if FEATURES_PATH is None:
    raise FileNotFoundError("No se encontraron archivos de features. Ajusta CANDIDATES en la celda anterior.")

if FEATURES_PATH.endswith(".csv"):
    df = safe_read_csv(FEATURES_PATH)
else:
    try:
        df = pd.read_parquet(FEATURES_PATH)
    except Exception as e:
        raise RuntimeError(f"No se pudo leer {FEATURES_PATH}: {e}")

print("Ruta:", FEATURES_PATH)
print("Shape:", df.shape)
display(df.head(3))



## 🧭 3. Vista general rápida
Tipos, `describe` y valores perdidos/duplicados.


In [ ]:

display(df.dtypes.to_frame("dtype").T)
display(df.describe(include=[np.number]))
display(df.describe(include=[object]))

nans = df.isna().sum().sort_values(ascending=False)
display(nans.to_frame("n_nans").T)
print("Duplicados exactos:", df.duplicated().sum())



## 📊 4. Balance de clases (si existe etiqueta)
Busca columnas típicas de etiqueta: `label`, `etiqueta`, `clase`, `y`, `target`.


In [ ]:

label_candidates = [c for c in df.columns if c.lower() in ("label","etiqueta","clase","y","target")]
if label_candidates:
    ycol = label_candidates[0]
    print("Usando etiqueta:", ycol)
    display(df[ycol].value_counts(dropna=False).to_frame("conteo"))
else:
    print("No se detectó columna de etiqueta común. Ajusta manualmente si procede.")



## 🔗 5. Correlación y primeras relaciones


In [ ]:

numeric_cols = [c for c in df.columns if pd.api.types.is_numeric_dtype(df[c])]
if len(numeric_cols) >= 2:
    corr = df[numeric_cols].corr(numeric_only=True)
    display(corr.iloc[:12,:12])
else:
    print("No hay suficientes columnas numéricas para correlación.")


1. Problema / Motivación

En la creación coreográfica, los coreógrafos trabajan con múltiples variables al mismo tiempo:
- calidad de movimiento,
- ocupación del espacio,
- dinámica rítmica,
- coordinación grupal,
- estética escénica.
Actualmente, no existe una herramienta inteligente que les dé retroalimentación objetiva en tiempo real o a partir de un vídeo grabado.

El problema:
- El proceso creativo depende únicamente de la percepción subjetiva del coreógrafo y carece de métricas automáticas o sugerencias asistidas por IA.

Objetivo:
- crear un asistente coreográfico basado en visión por computador y machine learning que analice vídeos (subidos o en directo por cámara) y proporcione sugerencias de mejora en términos de estética, técnica y composición espacial.

2. Obtención de Datos
- Fuentes de datos abiertas y específicas:
- Datasets de movimiento humano:
- AIST++ (Google Research): dataset de danza contemporánea y urbana con música sincronizada.
- Pose Estimation Pretrained Datasets:
- COCO, MPII (pose humana general).
- Videos propios / capturados:
- Posibilidad de grabar muestras de ensayos o fragmentos de repertorio clásico y contemporáneo.
- Herramientas para anotación:
- OpenPose, MediaPipe para generar puntos clave (skeletons).
- Enriquecer con metadatos: tiempo, nivel de energía, calidad espacial.

3. Procesamiento y Limpieza de Datos

- Conversión de vídeos a frames.
- Extracción de poses humanas (skeletons 2D o 3D con OpenPose/MediaPipe).
- Normalización de las secuencias (longitud, fps).
- Anotación de variables relevantes:
  - amplitud del movimiento,
  - simetría,
  - alineación corporal,
  - sincronización grupal,
  - desplazamiento espacial.

4. Análisis Exploratorio (EDA)
- Visualización de trayectorias de movimiento (plots de keypoints).
- Distribución de amplitud y velocidad en diferentes estilos.
- Comparación entre intérpretes y patrones.
- Extracción de métricas:
  - rango de movimiento,
  - fluidez (velocidad y aceleración),
  - nivel espacial (alto/medio/bajo),
  - balance entre simetría/asimetría.

5. Feature Engineering
- Diseñar features coreográficas a partir de las poses:
- Técnicos: alineación postural, equilibrio, extensiones.
- Espaciales: uso del espacio, direcciones, niveles.
- Rítmicos: tempo, sincronización con música (opcional).
- Estéticos: fluidez, originalidad de trayectorias.
- Los features serán el input del modelo de ML.

6. Modelado (Machine Learning / Deep Learning)
- Modelos candidatos:
  - Clasificación / recomendación:
    Red neuronal recurrente (LSTM/GRU) para secuencias de poses → sugiere patrones alternativos.
    SVM o Random Forest para evaluar calidad técnica.
  - Análisis en tiempo real:
  - Pose estimacion + métricas evaluadas frame a frame.
- Sistema de sugerencias:
  - Basado en similitud con patrones de repertorio (ej. Giselle, Lago de los Cisnes) o movimientos de datasets.
  - Basado en reglas estéticas (ejemplo: sugerir más uso de diagonales si todo está frontal).

7. Evaluación de Resultados
- Métricas de clasificación: accuracy, F1-score (para calidad técnica).
- Métricas cuantitativas: rango de movimiento, simetría, velocidad promedio.
- Validación cualitativa: feedback de coreógrafos/profesores.



**Introducción**

La creación coreográfica es un proceso complejo en el que confluyen múltiples dimensiones: técnica, expresiva, estética, espacial y musical. Tradicionalmente, los coreógrafos se apoyan en la observación subjetiva y en la interacción directa con los intérpretes para tomar decisiones durante la composición. Sin embargo, en la actualidad, el desarrollo de tecnologías de visión por computador y aprendizaje automático abre la posibilidad de incorporar herramientas inteligentes que analicen objetivamente el movimiento y ofrezcan sugerencias en tiempo real o a partir de grabaciones.
El presente proyecto propone el desarrollo de un asistente coreográfico inteligente que, a partir de la captura en vídeo (subido o en streaming), procese las secuencias de movimiento de los bailarines y proporcione retroalimentación automatizada para mejorar la calidad técnica, la expresividad y la organización espacial de la coreografía. Este enfoque busca situarse en la intersección entre arte y tecnología, ofreciendo al coreógrafo un apoyo complementario que no sustituye la creatividad humana, sino que la amplifica mediante análisis basados en datos.

Objetivos

Objetivo General

Diseñar e implementar un sistema de asistencia coreográfica basado en visión por computador y machine learning, capaz de analizar vídeos de danza y generar sugerencias constructivas que apoyen el proceso creativo de coreógrafos y bailarines.

Objetivos Específicos

Revisión crítica del problema: identificar los principales retos en la creación coreográfica que pueden beneficiarse de un análisis automatizado (técnicos, espaciales, rítmicos, estéticos).

Obtención y procesamiento de datos: recopilar datasets de danza y procesar secuencias de movimiento mediante técnicas de pose estimation (MediaPipe, OpenPose).

Definición de features coreográficas: diseñar métricas objetivas a partir de los keypoints del cuerpo humano (amplitud, simetría, ocupación espacial, fluidez).

Entrenamiento de modelos de machine learning: explorar algoritmos de clasificación y análisis secuencial (SVM, Random Forest, LSTM/GRU) para evaluar la calidad del movimiento y generar sugerencias.

Validación del sistema: comparar métricas técnicas con evaluaciones de expertos en danza, asegurando la pertinencia de las recomendaciones.
Desarrollo de una interfaz de uso: implementar un prototipo funcional en un notebook (y eventualmente en una app sencilla) donde se puedan subir vídeos o conectar una cámara en directo.

 Revisión del Problema

El proceso de creación coreográfica enfrenta varios desafíos:
Subjetividad en la evaluación: la calidad de un movimiento o de una secuencia depende en gran medida de la mirada del coreógrafo, lo que puede limitar la retroalimentación objetiva.

Dificultad de análisis en tiempo real: en ensayos, es complicado analizar simultáneamente la técnica individual, la coordinación grupal y el uso del espacio.

Falta de herramientas digitales específicas para danza: existen aplicaciones de análisis de movimiento en deportes o rehabilitación, pero pocas enfocadas en las necesidades de la composición coreográfica.

Limitada retroalimentación cuantitativa: la mayoría de las observaciones son cualitativas; faltan métricas visualizables que permitan al coreógrafo detectar patrones o desequilibrios.

El asistente coreográfico propuesto busca dar respuesta a estas limitaciones mediante:
- la captura y análisis automático de movimiento;
- la transformación de datos en sugerencias útiles;
- la complementariedad entre la intuición artística y el análisis objetivo.

Este proyecto se alinea con la tendencia emergente de integrar la inteligencia artificial en las artes escénicas, fomentando nuevas formas de creación y colaboración entre el arte y la tecnología.



In [ ]:
# Sistema (ffmpeg para vídeo; parallel útil en batch)
!apt-get -yq update
!apt-get -yq install ffmpeg parallel

# Paquetes Python esenciales para esta notebook
%pip -q install --upgrade pandas scipy scikit-learn tqdm matplotlib \
                         opencv-python-headless ultralytics


# **Búsqueda y Recopilación de Datos**

In [ ]:
# === Imports globales ===
import os, sys, glob, json, math, shutil, zipfile, itertools, pathlib
from typing import Optional, Tuple, List, Dict
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.spatial import ConvexHull

# === Constantes ===
# COCO-17: índices estándar usados por YOLOv8/MMPose (pose de 17 puntos)
COCO_EDGES = [
    (5,7),(7,9),   # brazo izquierdo: hombro->codo->muñeca
    (6,8),(8,10),  # brazo derecho
    (11,13),(13,15), # pierna izq: cadera->rodilla->tobillo
    (12,14),(14,16), # pierna dcha
    (5,6), (11,12),  # hombros, caderas
    (5,11), (6,12)   # hombro- cadera (lado)
]

# Rutas del proyecto
BASE = ROOT
RAW = f"{BASE}/data/raw/aistpp"
PROCESSED = f"{BASE}/data/processed/aistpp"
ANN = f"{BASE}/data/annotations/aistpp"
LOGS = f"{BASE}/logs"

print("Constantes inicializadas. COCO_EDGES:", len(COCO_EDGES), "aristas")


In [ ]:
# === Utilidades para keypoints 2D/3D ===

def find_first_glob(root: str, pattern: str) -> Optional[str]:
    rootp = pathlib.Path(root)
    m = sorted(rootp.rglob(pattern))
    return str(m[0]) if m else None

def load_any(path: str):
    suf = pathlib.Path(path).suffix.lower()
    if suf in (".pkl", ".pickle"):
        import pickle
        with open(path, "rb") as f:
            return pickle.load(f)
    if suf == ".npz":
        data = np.load(path, allow_pickle=True)
        return {k: data[k] for k in data.files}
    if suf == ".npy":
        return np.load(path, allow_pickle=True)
    raise ValueError(f"Formato no soportado: {suf} ({path})")

def to_TJD(obj) -> Optional[np.ndarray]:
    """
    Convierte estructuras comunes a un ndarray (T, J, D) float32, D in {2,3}.
    Devuelve None si no puede inferirse.
    """
    if obj is None:
        return None
    # Casos tipo dict con clave 'keypoints' o similar
    if isinstance(obj, dict):
        for k in ("keypoints", "poses", "kpts", "arr", "data"):
            if k in obj:
                arr = np.asarray(obj[k])
                break
        else:
            # si no hay claves típicas, prueba cualquier valor ndarray
            for v in obj.values():
                if isinstance(v, np.ndarray) and v.ndim >= 2:
                    arr = np.asarray(v); break
            else:
                return None
    else:
        arr = np.asarray(obj)

    arr = np.asarray(arr)
    # Si viene (N, J*D) → reshape
    if arr.ndim == 2 and arr.shape[1] in (17*2, 17*3):
        D = 2 if arr.shape[1] == 34 else 3
        J = 17
        T = arr.shape[0]
        arr = arr.reshape(T, J, D)
    # Si viene (T, J, D)
    if arr.ndim == 3 and arr.shape[-1] in (2,3):
        return arr.astype(np.float32)
    return None

def interpolate_nan(K: np.ndarray) -> np.ndarray:
    """Interplola NaNs por columna en cada joint-dim independiente."""
    K = K.copy()
    T, J, D = K.shape
    for j in range(J):
        for d in range(D):
            v = K[:, j, d]
            nans = ~np.isfinite(v)
            if nans.any():
                t = np.arange(T)
                if (~nans).sum() >= 2:
                    K[:, j, d] = np.interp(t, t[~nans], v[~nans])
                else:
                    # si casi todo es NaN, rellena con 0
                    K[:, j, d] = 0.0
    return K

def normalize_2d(K: np.ndarray) -> np.ndarray:
    """
    Normaliza 2D por traslación (centro de masa aproximado) y escala (distancia hombros).
    Devuelve (T,J,2).
    """
    K = K.copy().astype(np.float32)
    # centro de masa aprox = media de caderas (11,12) si existen; si no, media global
    if K.shape[1] >= 13:
        com = np.nanmean(K[:, [11,12], :], axis=1)
    else:
        com = np.nanmean(K, axis=1)
    K = K - com[:, None, :]

    # escala: distancia entre hombros (5 y 6) si existen; si no, mediana de distancias
    if K.shape[1] >= 7:
        shoulder_dist = np.linalg.norm(K[:,5,:] - K[:,6,:], axis=1)
        s = np.nanmedian(shoulder_dist)
    else:
        # mediana de distancias a COM
        s = np.nanmedian(np.linalg.norm(K, axis=2))
    if not np.isfinite(s) or s <= 1e-6:
        s = 1.0
    K /= s
    return K

def plot_skeleton_2d(frame: np.ndarray, title=""):
    xs, ys = frame[:,0], frame[:,1]
    plt.figure(figsize=(4,4))
    plt.scatter(xs, ys, s=12)
    for a,b in COCO_EDGES:
        if a < len(frame) and b < len(frame):
            plt.plot([xs[a], xs[b]], [ys[a], ys[b]])
    plt.gca().invert_yaxis()
    plt.axis("equal");
    plt.title(title)
    plt.show()
import os, time, pathlib
import numpy as np
import matplotlib.pyplot as plt

# Si no existiera COCO_EDGES en tu entorno, lo definimos para la visualización
try:
    COCO_EDGES
except NameError:
    COCO_EDGES = [
        (5,7),(7,9),(6,8),(8,10),(11,13),(13,15),
        (12,14),(14,16),(5,6),(11,12),(5,11),(6,12)
    ]

# ---------- Helpers de demo (usan TUS funciones) ----------
def pick_best_frame(K2: np.ndarray) -> int:
    """Elige el frame con más puntos finitos para visualizar."""
    if K2.ndim != 2 or K2.shape[1] < 2:
        return 0
    # K2 es (J,2) en un frame; cuando venga la secuencia usamos fuera
    return 0

def pick_best_frame_seq(K2_seq: np.ndarray) -> int:
    """Para una secuencia (T,J,2), elige el frame con más joints válidos."""
    T = K2_seq.shape[0]
    fin = np.isfinite(K2_seq[:,:,0]) & np.isfinite(K2_seq[:,:,1])
    cnt = fin.sum(axis=1)
    return int(np.argmax(cnt)) if T else 0

def demo_keypoints_debug(ROOT: str):
    print(f" Buscando archivos en: {ROOT}")
    patterns = ["**/*.npz", "**/*.npy", "**/*.pkl", "**/*.pickle"]
    path = None
    for pat in patterns:
        path = find_first_glob(ROOT, pat)
        if path:
            print(f" Encontrado: {path}  (patrón: {pat})")
            break
    if not path:
        print(" No se encontró ningún archivo de keypoints en la ruta dada.")
        return

    t0 = time.perf_counter()
    obj = load_any(path)
    print(f" load_any: {time.perf_counter()-t0:.3f}s | tipo: {type(obj)}")

    t0 = time.perf_counter()
    K = to_TJD(obj)
    print(f" to_TJD: {time.perf_counter()-t0:.3f}s | shape: {None if K is None else K.shape}")

    if K is None:
        print(" No se pudo convertir a (T,J,D). Revisa el contenido del archivo.")
        if isinstance(obj, dict):
            keys = list(obj.keys())
            print(" Claves disponibles:", keys[:10], "..." if len(keys)>10 else "")
        return

    # Estadísticas básicas
    T,J,D = K.shape
    n_nan = int(np.isnan(K).sum())
    print(f"Frames: {T} | Joints: {J} | Dims: {D} | NaNs totales: {n_nan}")

    # Interpolación de NaNs (tu función)
    Kc = interpolate_nan(K)
    print(" Interpolación de NaNs aplicada.")

    # Normalización y visualización 2D si hay al menos 2D
    if D >= 2:
        K2_seq = normalize_2d(Kc[...,:2])  # (T,J,2) normalizado
        f = pick_best_frame_seq(K2_seq)
        print(f" Visualizando frame #{f} (mayor nº de puntos válidos).")
        plot_skeleton_2d(K2_seq[f], title=f"Frame {f}")
    else:
        print("ℹ Datos con D<2; se omite visualización de esqueleto.")

# ==== EJECUTAR DEMO ====
ROOT = "/content/drive/MyDrive/asistente_coreografico/data/annotations/aistpp/aist_plusplus_final/keypoints2d"
demo_keypoints_debug(ROOT)

In [ ]:
import os, pathlib, pickle, json
import numpy as np
import matplotlib.pyplot as plt

BASE = "/content/drive/MyDrive/asistente_coreografico"
ANN  = f"{BASE}/data/annotations/aistpp"

def load_any(path):
    suf = pathlib.Path(path).suffix.lower()
    if suf == ".pkl" or suf == ".pickle":
        with open(path, "rb") as f: obj = pickle.load(f)
        return obj
    if suf == ".npz":
        data = np.load(path, allow_pickle=True)
        return {k: data[k] for k in data.files}
    if suf == ".npy":
        return np.load(path, allow_pickle=True)
    if suf == ".json":
        with open(path, "r") as f: return json.load(f)
    raise ValueError(f"Extensión no soportada: {suf}")

def to_TJD_from_any(obj):
    # Si el PKL/NPZ es dict, busca arrays grandes dentro
    def iter_arrays(o, depth=3):
        if depth < 0: return
        if isinstance(o, np.ndarray):
            yield o
        elif isinstance(o, dict):
            for v in o.values(): yield from iter_arrays(v, depth-1)
        elif isinstance(o, (list,tuple)):
            for v in o: yield from iter_arrays(v, depth-1)

    for arr in iter_arrays(obj, 4):
        if not isinstance(arr, np.ndarray): continue
        if arr.ndim != 3: continue
        # probar permutaciones a (T,J,D) con D=2/3
        for axes in [(0,1,2),(0,2,1),(1,0,2),(1,2,0),(2,0,1),(2,1,0)]:
            T,J,D = arr.shape[axes[0]], arr.shape[axes[1]], arr.shape[axes[2]]
            if D in (2,3) and 5 <= J <= 80 and T >= 8:
                return np.transpose(arr, axes).astype(np.float32)
    # si ya era np.ndarray (T,J,D)
    if isinstance(obj, np.ndarray) and obj.ndim == 3 and obj.shape[-1] in (2,3):
        return obj.astype(np.float32)
    return None

def find_first_existing(patterns, root):
    rootp = pathlib.Path(root)
    for pat in patterns:
        f = sorted(rootp.rglob(pat))
        if f: return str(f[0])
    return None

# Buscar 2D y 3D típicos dentro de aist_plusplus_final
p2 = find_first_existing([
    "aist_plusplus_final/keypoints2d/*.pkl",
    "aist_plusplus_final/**/keypoints2d*.pkl",
    "*keypoints2d*.pkl","*pose2d*.pkl","*2d*keypoints*.pkl",
    "*keypoints2d*.npz","*pose2d*.npz","*2d*keypoints*.npz",
], ANN)

p3 = find_first_existing([
    "aist_plusplus_final/keypoints3d/*.pkl",
    "aist_plusplus_final/**/keypoints3d*.pkl",
    "*keypoints3d*.pkl","*pose3d*.pkl","*3d*keypoints*.pkl",
    "*keypoints3d*.npz","*pose3d*.npz","*3d*keypoints*.npz",
], ANN)

print("Detectado 2D:", p2)
print("Detectado 3D:", p3)

K2 = None
if p2:
    obj2 = load_any(p2)
    K2 = to_TJD_from_any(obj2)
    print("K2 shape:", None if K2 is None else K2.shape)

K3 = None
if p3:
    obj3 = load_any(p3)
    K3 = to_TJD_from_any(obj3)
    print("K3 shape:", None if K3 is None else K3.shape)

# ---------- Limpieza NaN (interpolación temporal por joint,dim) ----------
def interpolate_nan_1d(y):
    """
    y: (T,) con NaN -> forward/backward fill + interp lineal
    """
    y = y.astype(np.float32)
    T = len(y)
    idx = np.arange(T)
    mask = ~np.isnan(y)
    if mask.sum() == 0:
        return y  # todo NaN -> se tratará arriba
    # forward/backward
    # forward fill
    last = np.nan
    for t in range(T):
        if np.isnan(y[t]) and not np.isnan(last):
            y[t] = last
        else:
            last = y[t]
    # backward fill
    last = np.nan
    for t in range(T-1, -1, -1):
        if np.isnan(y[t]) and not np.isnan(last):
            y[t] = last
        else:
            last = y[t]
    # si aún quedan NaN extremos, interp lineal con puntos válidos
    mask = ~np.isnan(y)
    if mask.sum() >= 2 and mask.sum() < T:
        y[~mask] = np.interp(idx[~mask], idx[mask], y[mask])
    return y

def clean_nan_interpolate(K, min_valid_ratio=0.15):
    """
    Interpola NaN en cada joint y dimensión. Descarta joints con pocos datos válidos.
    Devuelve K_clean (T,J',D) y un mapa de joints usados.
    """
    if K is None: return None, []
    Kc = K.copy()
    T,J,D = Kc.shape
    use_joints = []
    for j in range(J):
        valid = ~np.isnan(Kc[:,j,:]).any(axis=1)
        ratio = valid.mean()
        if ratio < min_valid_ratio:
            # demasiados NaN -> descartar este joint completo
            Kc[:,j,:] = np.nan
            continue
        for d in range(D):
            Kc[:,j,d] = interpolate_nan_1d(Kc[:,j,d])
        use_joints.append(j)
    # si todos descartados, devolvemos tal cual para evitar crash
    if not use_joints:
        return Kc, []
    # filtrar a solo joints útiles (al menos uno no NaN)
    Kc = Kc[:, use_joints, :]
    return Kc, use_joints

def nanrobust_metrics(K):
    """
    Métricas que ignoran NaNs: amplitud (nanmax-nanmin), velocidad media, simetría.
    """
    T,J,D = K.shape
    amp = (np.nanmax(K, axis=0) - np.nanmin(K, axis=0))  # (J,D)
    amp_mean = np.nanmean(amp, axis=0)                   # (D,)
    vel = np.diff(K, axis=0)                             # (T-1,J,D)
    vel_norm = np.linalg.norm(vel, axis=2)               # (T-1,J)
    vel_mean = np.nanmean(vel_norm)
    # simetría: pares COCO, usando solo joints presentes
    COCO_PAIRS = [(5,6),(7,8),(9,10),(11,12),(13,14),(15,16)]
    pair_vals = []
    for a,b in COCO_PAIRS:
        if a < J and b < J:
            diff = K[:,a,:] - K[:,b,:]
            # Check number of dimensions before calculating norm
            if diff.ndim > 1:
                d = np.linalg.norm(diff, axis=1).mean()  # Correct axis for (T, D) -> (T,)
                pair_vals.append(np.nanmean(d))
    sym_raw = np.nanmean(pair_vals) if pair_vals else np.nan
    # avoid division by zero if all amplitude means are zero or NaN
    sum_amp_mean = np.nansum(amp_mean)
    sym_index = sym_raw / (sum_amp_mean + 1e-6) if sum_amp_mean > 0 else np.nan
    return {
        "amp_media_por_eje": [float(x) if np.isfinite(x) else float('nan') for x in amp_mean],
        "vel_media": float(vel_mean) if np.isfinite(vel_mean) else float('nan'),
        "simetria_indice": float(sym_index) if np.isfinite(sym_index) else float('nan')
    }

# Limpiar K3 y medir
if K3 is not None:
    K3c, used3 = clean_nan_interpolate(K3, min_valid_ratio=0.10)
    print(f"Joints 3D usados: {len(used3)} de {K3.shape[1]}")
    m3 = nanrobust_metrics(K3c)
    print("Métricas 3D (limpias):", m3)

# Intentar localizar/cargar 2D si no se encontró
if K2 is None:
    # En muchos paquetes 2D falta, trabajamos con proyección (x,y) del 3D limpio
    if K3 is not None:
        K2 = K3c[:,:,:2]  # x,y
        print("K2 generado por proyección de K3 (x,y). Shape:", K2.shape)

# Visualización 2D
def plot_skeleton_2d(frame, title="Skeleton 2D"):
    # Conectividad COCO aproximada
    edges = [(5,7),(7,9),(6,8),(8,10),(11,13),(13,15),
             (12,14),(14,16),(5,6),(11,12),(5,11),(6,12)]
    xs, ys = frame[:,0], frame[:,1]
    plt.figure(figsize=(4,4))
    plt.scatter(xs, ys, s=12)
    for a,b in edges:
        if a < len(frame) and b < len(frame):
            plt.plot([xs[a], xs[b]], [ys[a], ys[b]])
    plt.gca().invert_yaxis()
    plt.axis("equal"); plt.title(title)
    plt.show()

if K2 is not None and np.isfinite(K2).any():
    mid = len(K2)//2
    plot_skeleton_2d(K2[mid], "Frame medio (2D limpio)")

In [ ]:

# Centro de masas aproximado = promedio de caderas (11 y 12 en COCO)
def center_of_mass(K):
    if K.shape[1] >= 13:
        return (K[:,11,:] + K[:,12,:]) / 2
    return K.mean(axis=1)

def features_coreograficos(K):
    T,J,D = K.shape
    feats = {}

    # --- amplitud global ---
    amp = (K.max(axis=0) - K.min(axis=0)).mean(axis=0)
    feats["amplitud_x"], feats["amplitud_y"] = amp[0], amp[1]
    if D == 3: feats["amplitud_z"] = amp[2]

    # --- velocidad / fluidez ---
    vel = np.linalg.norm(np.diff(K,axis=0),axis=2).mean()
    feats["velocidad_media"] = vel

    # --- simetría ---
    pairs = [(5,6),(7,8),(9,10),(11,12),(13,14),(15,16)]
    pair_dists = []
    for a,b in pairs:
        if a<J and b<J:
            d = np.linalg.norm(K[:,a,:]-K[:,b,:],axis=1).mean()
            pair_dists.append(d)
    feats["simetria"] = np.mean(pair_dists)

    # --- niveles espaciales ---
    com = center_of_mass(K)
    feats["nivel_alto"] = np.percentile(com[:,1],90)
    feats["nivel_bajo"] = np.percentile(com[:,1],10)
    feats["nivel_rango"] = feats["nivel_bajo"] - feats["nivel_alto"]

    # --- variedad direccional ---
    disp = np.diff(com,axis=0)
    dirs = np.arctan2(disp[:,1], disp[:,0])
    cambios = np.abs(np.diff(dirs))
    feats["variedad_direcciones"] = np.mean(cambios)

    return feats

def sugerencias(feats: dict):
    """
    Genera sugerencias coreográficas a partir de métricas de análisis.
    Soporta nombres alternativos y métricas opcionales.
    Devuelve lista de strings (sugerencias deduplicadas y priorizadas).
    """

    S = []  # sugerencias

    # ===== Helpers =====
    def val(*keys, default=None):
        for k in keys:
            if k in feats and feats[k] is not None:
                return feats[k]
        return default

    def add(msg, prio=5):
        # prio: 1 (crítico) .. 9 (cosmético)
        if msg:
            S.append((prio, msg.strip()))

    # ===== Normalización de nombres frecuentes =====
    ax   = val("amplitud_x", "amp_x")
    ay   = val("amplitud_y", "amp_y")
    az   = val("amplitud_z", "amp_z")
    vel  = val("velocidad_media", "vel_media")
    vel_sd = val("velocidad_std", "vel_std")
    sim  = val("simetria", "simetria_raw")
    varD = val("variedad_direcciones", "variedad_dir")          # (rad prom |Δθ|)
    nivel = val("nivel_rango", "nivel_rango")                   # (p10 - p90 Y COM)
    disp = val("disp_total", "trayectoria_longitud")            # longitud de COM
    frames = val("frames")
    dims   = val("dims")

    # ===== Técnicos (alineación, equilibrio, extensiones, torso, cabeza) =====
    # Alineación postural (0..100, mayor=mejor)
    alineacion = val("alineacion_postural", "posture_alignment_score")
    if alineacion is not None:
        if alineacion < 40:
            add("Alinear ejes corporales: revisar colocación de pelvis, caja torácica y eje cabeza‑cadera.", prio=2)
        elif alineacion < 70:
            add("Ajustar alineación: activar centro (core) para estabilizar hombros y caderas.", prio=4)

    # Equilibrio/estabilidad (0..100, mayor=mejor) y sway/temblor (grande=peor)
    equilibrio = val("equilibrio", "balance_score")
    sway = val("oscilacion_centro", "sway_mag")  # pix o normalizado
    if equilibrio is not None:
        if equilibrio < 40:
            add("Equilibrio bajo: ensayar apoyos con foco visual y transferencias de peso más claras.", prio=2)
        elif equilibrio < 70:
            add("Mejorar estabilidad: trabajar relevés controlados y anclajes de pie de apoyo.", prio=4)
    if sway is not None and sway > 15:
        add("Reducir oscilación de tronco: reforzar control de core y respiración dirigida.", prio=4)

    # Extensiones (0..180º aproximado)
    ext_pierna = val("extension_pierna", "leg_extension_deg")
    if ext_pierna is not None:
        if ext_pierna < 60:
            add("Extensión de pierna limitada: trabajar flexibilidad de isquios y acción de psoas.", prio=5)
        elif ext_pierna > 150:
            add("Gran extensión de pierna: cuidar cierre limpio y sostén para evitar colapso.", prio=6)

    ext_brazo = val("extension_brazo", "arm_extension_deg")
    if ext_brazo is not None and ext_brazo < 70:
        add("Mayor proyección de brazos: extender a través de dedos manteniendo hombros bajos.", prio=5)

    # Torso / cabeza (0..100, mayor=mejor)
    torso_estab = val("estabilidad_torso", "torso_stability")
    if torso_estab is not None and torso_estab < 50:
        add("Torso inestable: aislar disociaciones pelvis/torso para limpiar líneas.", prio=4)
    control_cabeza = val("control_cabeza", "head_control")
    if control_cabeza is not None and control_cabeza < 50:
        add("Control de cabeza bajo: definir focos (spotting) y direcciones claras de mirada.", prio=5)

    # ===== Espaciales =====
    if ax is not None and ay is not None:
        if ax < 50 or ay < 50:
            add("Aumentar amplitud: proyectar más en horizontal/vertical y ampliar desplazamientos.", prio=3)
        if abs(ax - ay) > 120:
            add("Equilibrar uso de ejes: compensar horizontal/vertical para un espacio más homogéneo.", prio=5)
        if max(ax, ay) > 260:
            add("Amplitud muy alta: alternar micro‑gestos/pausas para no saturar el encuadre.", prio=6)
    if az is not None and az > 40:
        add("Exceso de variación en profundidad: estabilizar planos antes de cambiar de nivel Z.", prio=7)

    if nivel is not None:
        if nivel < 30:
            add("Explorar niveles: añadir suelo (low) y saltos (high) para mayor contraste vertical.", prio=3)
        elif nivel > 120:
            add("Rango vertical muy amplio: introducir tránsitos por nivel medio para cohesión.", prio=6)

    if disp is not None:
        if disp < 150:
            add("Ocupación espacial reducida: abrir diagonales y profundidades para ventilar la escena.", prio=4)
        elif disp > 2000:
            add("Demasiada traslación: crear anclajes y puntos de fijación para foco visual.", prio=6)

    # Curvatura/entropía de trayectoria (0 lineal .. alto curvo/errático)
    curv = val("curvatura_trayectoria", "path_curvature", "trayectoria_curv")
    if curv is not None:
        if curv < 0.15:
            add("Trayectorias lineales: incorporar arcos/espirales para enriquecer el dibujo espacial.", prio=6)
        elif curv > 0.8:
            add("Trayectorias muy sinuosas: incluir líneas rectas para contraste y legibilidad.", prio=7)

    # ===== Direcciones / Orientación =====
    if varD is not None:
        if varD < 0.4:
            add("Poca variedad direccional: sumar giros, cambios de orientación y diagonales.", prio=4)
        elif varD > 1.8:
            add("Exceso de quiebres: sostener orientación más tiempo para dar foco.", prio=6)

    # ===== Dinámica (velocidad/fluidez/pausas/energía) =====
    if vel is not None:
        if vel < 2.0:
            add("Incrementar fluidez: encadenar transiciones y elevar tonicidad en pasajes clave.", prio=3)
        elif vel > 8.0:
            add("Regular velocidad: introducir suspensiones/pausas para respiración escénica.", prio=5)

    if vel_sd is not None:
        if vel_sd < 0.5:
            add("Dinámica monótona: añadir acentos, contrastes de peso y calidad de esfuerzo.", prio=4)
        elif vel_sd > 3.0:
            add("Dinámica errática: clarificar acentos primarios y limpiar transiciones.", prio=5)

    pausas = val("pausas_ratio", "pause_ratio")  # 0..1
    if pausas is not None:
        if pausas < 0.05:
            add("Casi sin pausas: insertar micro‑silencios para realzar acentos.", prio=6)
        elif pausas > 0.35:
            add("Muchas pausas: conectar frases para sostener la continuidad.", prio=6)

    energia = val("energia", "energy_level")  # 0..100
    if energia is not None:
        if energia < 30:
            add("Energía baja: elevar ataque y proyección para sostener presencia escénica.", prio=4)
        elif energia > 85:
            add("Energía muy alta: modular con momentos de contención para diversidad.", prio=6)

    # ===== Simetría / Disociaciones =====
    if sim is not None:
        if sim < 10:
            add("Demasiada asimetría: introducir momentos de equilibrio bilateral.", prio=5)
        elif sim > 80:
            add("Mucha simetría: explorar disociaciones entre lados para enriquecer textura.", prio=5)

    disoc = val("disociacion", "disociation_score")  # 0..100
    if disoc is not None:
        if disoc < 30:
            add("Poca disociación: trabajar independencia de brazos/torso/piernas.", prio=6)
        elif disoc > 85:
            add("Mucha disociación: re‑sincronizar focos corporales para cohesión del fraseo.", prio=7)

    # ===== Suelo / Peso / Tiempo / Espacio / Fluidez (Laban) =====
    tiempo_suelo = val("tiempo_en_suelo", "ground_time_ratio")  # 0..1
    if tiempo_suelo is not None:
        if tiempo_suelo < 0.1:
            add("Casi sin trabajo de suelo: integrar apoyos low para contraste de niveles.", prio=5)
        elif tiempo_suelo > 0.6:
            add("Mucho suelo: alternar elevaciones a nivel medio/alto para balance.", prio=6)

    laban_peso   = val("laban_peso", "effort_weight")     # 0 ligero .. 1 fuerte
    laban_tiempo = val("laban_tiempo", "effort_time")     # 0 sostenido .. 1 súbito
    laban_espacio= val("laban_espacio", "effort_space")   # 0 indirecto .. 1 directo
    laban_fluidez= val("laban_fluidez", "effort_flow")    # 0 ligado .. 1 libre
    if laban_peso is not None:
        if laban_peso < 0.3: add("Calidad ligera: incorporar acentos de peso para densidad expresiva.", prio=6)
        elif laban_peso > 0.8: add("Peso muy marcado: intercalar pasajes ligeros para contraste.", prio=6)
    if laban_tiempo is not None:
        if laban_tiempo < 0.3: add("Tiempo sostenido: sumar ataques súbitos para sorpresa rítmica.", prio=6)
        elif laban_tiempo > 0.8: add("Tiempo muy súbito: incluir prolongaciones/sostenidos para respirar.", prio=6)
    if laban_espacio is not None:
        if laban_espacio < 0.3: add("Espacio indirecto: introducir trayectorias directas y precisas.", prio=7)
        elif laban_espacio > 0.8: add("Espacio muy directo: sumar exploración periférica/curvilínea.", prio=7)
    if laban_fluidez is not None:
        if laban_fluidez < 0.3: add("Flujo muy ligado: liberar articulaciones para mayor libertad.", prio=7)
        elif laban_fluidez > 0.8: add("Flujo muy libre: consolidar cierres para claridad formal.", prio=7)

    # ===== Música / Ritmo =====
    onbeat = val("onbeat_ratio")
    bpm    = val("bpm")
    if onbeat is not None:
        if onbeat < 0.4:
            add("Poca alineación con el pulso: anclar acentos coreográficos a pulsos o contratiempos claros.", prio=4)
        elif onbeat > 0.9:
            add("Adherencia total al pulso: jugar con síncopas/silencios para sorpresa.", prio=6)
    if bpm is not None and vel is not None:
        if bpm >= 120 and vel < 3.0:
            add("Música rápida con baja energía motriz: aumentar ataque y desplazamiento.", prio=4)
        if bpm <= 80 and vel > 6.0:
            add("Música lenta con alta energía: introducir suspensión y foco en calidad del gesto.", prio=5)

    # ===== Grupales (sync/unison/spread/área/proximidad/centros) =====
    gm_sync  = val("gm_sync", "sync")
    gm_unis  = val("gm_unison", "unison")
    gm_sp    = val("gm_spread", "spread")
    gm_area  = val("formation_area", "gm_formation_area")
    prox_col = val("colisiones_proximidad", "near_collisions")  # conteo/frame
    centros  = val("centros_atencion", "attention_centers")     # 1..N

    if gm_sync is not None:
        if gm_sync < 0.2:
            add("Sincronía grupal baja: ensayar cues comunes y respiración compartida.", prio=2)
        elif gm_sync > 0.6:
            add("Buena sincronía: insertar contrapuntos/cánones controlados para riqueza.", prio=6)
    if gm_unis is not None:
        if gm_unis < 0.5:
            add("Unísono débil: trabajar frases espejo y acentos comunes antes del contrapunto.", prio=3)
        elif gm_unis > 0.85:
            add("Unísono muy alto: introducir micro‑variaciones de timing/nivel.", prio=6)
    if gm_sp is not None:
        if gm_sp < 60:
            add("Formación compacta: abrir aperturas y diagonales para respiración escénica.", prio=3)
        elif gm_sp > 300:
            add("Formación muy dispersa: crear subgrupos/centros de atención para dirigir la mirada.", prio=4)
    if gm_area is not None:
        if gm_area < 3000:
            add("Área de formación pequeña: explorar geometrías amplias (V, X, diagonales).", prio=5)
        elif gm_area > 20000:
            add("Área de formación grande: condensar en momentos clave para impacto visual.", prio=6)
    if prox_col is not None and prox_col > 0.2:
        add("Riesgo de colisiones: definir rutas y prioridades de paso entre intérpretes.", prio=3)
    if centros is not None:
        if centros < 1.5:
            add("Un solo foco escénico: sumar focos secundarios o desplazarlos en el tiempo.", prio=7)
        elif centros > 3.5:
            add("Demasiados focos: simplificar para no dispersar la atención del público.", prio=6)

    # ===== Composición / Continuidad / Originalidad =====
    continuidad = val("continuidad", "continuity_score")  # 0..100
    if continuidad is not None and continuidad < 50:
        add("Continuidad débil: mejorar enlaces entre frases y cadencias de cierre/apertura.", prio=4)

    trans_suaves = val("transiciones_suaves", "smooth_transitions")  # 0..100
    if trans_suaves is not None and trans_suaves < 50:
        add("Transiciones bruscas: trabajar anticipación y preparación para cambios limpios.", prio=5)

    originalidad = val("originalidad_trayectoria", "trajectory_novelty")  # 0..100
    if originalidad is not None and originalidad < 40:
        add("Baja originalidad espacial: introducir permutaciones (retrogradación, inversión, desplazamientos).", prio=7)

    densidad = val("densidad_mov", "movement_density")  # acciones/s
    if densidad is not None:
        if densidad < 0.5:
            add("Densidad escasa: añadir ornamentación o capas secundarias en transiciones.", prio=7)
        elif densidad > 3.0:
            add("Densidad alta: limpiar material y dejar respirar los acentos.", prio=7)

    # ===== Comunicación/Escena =====
    contacto_visual = val("contacto_visual", "gaze_contact_ratio")  # 0..1
    if contacto_visual is not None:
        if contacto_visual < 0.15:
            add("Poco contacto visual: definir miradas/focos para aumentar conexión con público y partener.", prio=6)
        elif contacto_visual > 0.8:
            add("Contacto visual constante: alternar miradas periféricas para matizar la relación escénica.", prio=7)

    respiracion = val("respiracion_sync", "breath_sync")  # 0..100
    if respiracion is not None and respiracion < 50:
        add("Baja sincronía respiratoria: utilizar respiración como cue de inicio/cierre de frases.", prio=6)

    # ===== Heurísticas de duración =====
    if frames is not None and frames < 150:  # ~5s a 30 FPS
        add("Frase corta: repetir con variaciones (canon, retrogradación, inversión) para consolidar material.", prio=8)

    # ===== Capa estética global =====
    add("Capa estética: alinear intención (peso/flujo) con cualidad del movimiento para coherencia expresiva.", prio=9)

    # ===== Post-proceso: de-dup + orden por prioridad =====
    # Eliminamos duplicados manteniendo la mayor prioridad (menor número)
    agg = {}
    for p, m in S:
        if m not in agg or p < agg[m]:
            agg[m] = p
    ordered = sorted([(p, m) for m, p in agg.items()], key=lambda x: x[0])

    # Devuelve solo los textos
    out = [m for _, m in ordered]
    return out or ["El movimiento presenta buena diversidad y balance."]

# ---- aplicar sobre K2 (si está cargado) ----
if K2 is not None:
    feats = features_coreograficos(K2)
    print("Features extraídos:\n", feats)
    print("\nSugerencias coreográficas:")
    for sug in sugerencias(feats):
        print(" -", sug)


In [ ]:
import pathlib
import pickle


BASE = "/content/drive/MyDrive/asistente_coreografico"
ANN  = f"{BASE}/data/annotations/aistpp"
PROCESSED = f"{BASE}/data/processed/aistpp"


CSV  = f"{PROCESSED}/features_coreograficos.csv"
ANNOTATION_DIR = f"{ANN}/aist_plusplus_final/keypoints3d" # Se Asumen los 3D keypoints que se estan utilizando

all_features = []

# Encontramos todos los pkl files en el directorio de anotaciones
pkl_files = list(pathlib.Path(ANNOTATION_DIR).rglob("*.pkl"))

print(f"Found {len(pkl_files)} annotation files.")

for pkl_file in pkl_files:
    try:
        obj = load_any(str(pkl_file))
        K = to_TJD_from_any(obj)

        if K is not None and np.isfinite(K).any():
            # Liempieza de valores NaN e interpolación
            Kc, used_joints = clean_nan_interpolate(K, min_valid_ratio=0.10)

            if Kc is not None and np.isfinite(Kc).any():
                 # Extraemos los features
                feats = features_coreograficos(Kc)

                # Añadimos metadata
                feats['filename'] = pkl_file.name
                feats['filepath'] = str(pkl_file)
                feats['frames'] = Kc.shape[0]
                feats['joints'] = Kc.shape[1]
                feats['dims'] = Kc.shape[2]

                all_features.append(feats)
            else:
                print(f"Skipping {pkl_file.name}: No valid data after cleaning.")
        else:
             print(f"Skipping {pkl_file.name}: Could not load or find valid keypoint data.")

    except Exception as e:
        print(f"Error processing {pkl_file.name}: {e}")


if all_features:
    df = pd.DataFrame(all_features)
    df.to_csv(CSV, index=False)
    print(f"\nSuccessfully extracted features and saved to {CSV}")
    display(df.head())
else:
    print("\nNo features were extracted.")

In [ ]:
# ENTRENAMIENTO (umbral por etiqueta)
import numpy as np, pandas as pd, os, json, joblib
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import RobustScaler
from sklearn.linear_model import LogisticRegressionCV
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import f1_score

# --- Rutas ---
BASE = "/content/drive/MyDrive/asistente_coreografico"
PROC = f"{BASE}/data/processed/aistpp"
ART  = f"{BASE}/artifacts"
CSV  = f"{PROC}/features_coreograficos.csv"
os.makedirs(ART, exist_ok=True)

# --- Cargar datos base ---
df = pd.read_csv(CSV)

# Si ya tienes y_complete de celdas previas, lo usamos. Si no, lo reconstruimos pequeño.
if 'y_complete' not in globals():
    # Etiquetas mínimas por si no existe y_complete (usa tus reglas si ya las definiste)
    def _mk_y(d):
        dnum = d.select_dtypes(include=np.number).copy()
        q20, q80 = dnum.quantile(0.20), dnum.quantile(0.80)
        lab = {}
        if {"amplitud_x","amplitud_y"} <= set(dnum.columns):
            lab['amplitud_baja']     = ((dnum['amplitud_x']<=q20['amplitud_x'])|(dnum['amplitud_y']<=q20['amplitud_y'])).astype(int)
            lab['amplitud_excesiva'] = ((dnum['amplitud_x']>=q80['amplitud_x'])|(dnum['amplitud_y']>=q80['amplitud_y'])).astype(int)
            ratio = dnum['amplitud_x']/(dnum['amplitud_y']+1e-6)
            r10,r90 = ratio.quantile(0.10), ratio.quantile(0.90)
            lab['amplitud_desbalance'] = ((ratio<=r10)|(ratio>=r90)).astype(int)
        for k in ["velocidad_media","simetria","nivel_rango","variedad_direcciones","frames"]:
            if k in dnum:
                lab[f"{k}_low"]  = (dnum[k]<=q20.get(k,np.nan)).fillna(False).astype(int)
                lab[f"{k}_high"] = (dnum[k]>=q80.get(k,np.nan)).fillna(False).astype(int)
        return pd.DataFrame(lab)
    y_complete = _mk_y(df)

label_names = y_complete.columns.tolist()

# --- FEATURE ENGINEERING  ---
def build_features(d: pd.DataFrame) -> pd.DataFrame:
    out = pd.DataFrame(index=d.index)
    # básicos (usa los que existan)
    base_cols = ['amplitud_x','amplitud_y','amplitud_z',
                 'velocidad_media','simetria','nivel_rango',
                 'variedad_direcciones','frames']
    for c in base_cols:
        if c in d.columns: out[c] = pd.to_numeric(d[c], errors='coerce')

    # derivados (solo si hay datos fuente)
    if {'amplitud_x','amplitud_y'} <= set(out.columns):
        out['amp_diff']  = (out['amplitud_x'] - out['amplitud_y']).abs()
        out['amp_ratio'] = out['amplitud_x'] / (out['amplitud_y'].abs() + 1e-6)
    if 'velocidad_media' in out:
        out['vel_log1p'] = np.log1p(out['velocidad_media'].clip(lower=0))
    if 'nivel_rango' in out:
        out['nivel_abs'] = out['nivel_rango'].abs()
    if {'velocidad_media','variedad_direcciones'} <= set(out.columns):
        out['vel_x_varDir'] = out['velocidad_media'] * out['variedad_direcciones']
    if {'simetria','amplitud_x'} <= set(out.columns):
        out['sim_x_ampx'] = out['simetria'] * out['amplitud_x']

    return out

X_df = build_features(df)
feature_cols = X_df.columns.tolist()
X_all = X_df.values
Y_all = y_complete.values

# --- Split train/val/test (70/15/15) ---
X_trainval, X_test, y_trainval, y_test = train_test_split(
    X_all, Y_all, test_size=0.15, random_state=42
)
# val = 15% del total => relativo a trainval:
X_train, X_val, y_train, y_val = train_test_split(
    X_trainval, y_trainval, test_size=0.1765, random_state=42
)

# --- Pipeline con LogisticRegressionCV dentro de OneVsRest ---
pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler",  RobustScaler()),
    ("clf", OneVsRestClassifier(
        LogisticRegressionCV(
            Cs=np.logspace(-2, 1, 6),   # 0.01..10
            solver="liblinear",
            class_weight="balanced",
            max_iter=4000,
            cv=3,
            scoring="f1",
            n_jobs=-1,
            refit=True
        ),
        n_jobs=-1
    ))
])

pipe.fit(X_train, y_train)

# --- Buscar UMBRALES por etiqueta en validación para maximizar F1 ---
def find_optimal_thresholds(y_true, y_score, grid=None):
    if grid is None:
        grid = np.linspace(0.2, 0.8, 61)  # 0.2..0.8 paso 0.01
    C = y_true.shape[1]
    thr = np.full(C, 0.5, dtype=float)
    for i in range(C):
        yt = y_true[:, i]
        # si en val hay solo una clase, dejamos 0.5
        if np.unique(yt).size < 2:
            thr[i] = 0.5
            continue
        best_f1, best_t = -1.0, 0.5
        scores_i = y_score[:, i]
        for t in grid:
            yp = (scores_i >= t).astype(int)
            f1 = f1_score(yt, yp, zero_division=0)
            if f1 > best_f1:
                best_f1, best_t = f1, t
        thr[i] = best_t
    return thr

# Probabilidades en validación
try:
    y_val_score = pipe.predict_proba(X_val)
except Exception:
    # fallback: usar decision_function y pasarlo por sigmoid
    logits = pipe.decision_function(X_val)
    y_val_score = 1.0 / (1.0 + np.exp(-logits))

thresholds = find_optimal_thresholds(y_val, y_val_score)

# --- Wrapper compatible con tu celda de evaluación ---
class ThresholdedOVR:
    """Envuelve un pipeline y aplica umbral por etiqueta en predict."""
    def __init__(self, pipeline, thresholds, label_names=None, feature_cols=None):
        self.pipeline = pipeline
        self.thresholds_ = np.asarray(thresholds, dtype=float)
        self.label_names = list(label_names) if label_names is not None else None
        self.feature_cols = list(feature_cols) if feature_cols is not None else None

    def predict_proba(self, X):
        return self.pipeline.predict_proba(X)

    def predict(self, X):
        P = self.predict_proba(X)
        thr = self.thresholds_
        # broadcast seguro
        return (P >= thr.reshape(1, -1)).astype(int)

# construir objeto final esperado por evaluación
complete_model = ThresholdedOVR(
    pipeline=pipe,
    thresholds=thresholds,
    label_names=label_names,
    feature_cols=feature_cols
)

# --- Evaluación rápida en test para ver mejora (opcional) ---
P_test = complete_model.predict_proba(X_test)
Yp_test = (P_test >= thresholds.reshape(1, -1)).astype(int)
print("F1 macro (test, con umbrales óptimos):", f1_score(y_test, Yp_test, average="macro", zero_division=0))
print("F1 micro (test, con umbrales óptimos):", f1_score(y_test, Yp_test, average="micro", zero_division=0))

# --- Guardado de artefactos ---
bundle = {
    "pipeline": pipe,
    "thresholds": thresholds,
    "label_names": label_names,
    "feature_cols": feature_cols
}
joblib.dump(bundle, f"{ART}/complete_model_thresholded_bundle.joblib")
pd.Series(label_names).to_csv(f"{ART}/complete_label_names.csv", index=False, header=False)
pd.Series(feature_cols).to_csv(f"{ART}/complete_feature_cols.csv", index=False, header=False)
with open(f"{ART}/complete_thresholds.json", "w") as f:
    json.dump({label_names[i]: float(thresholds[i]) for i in range(len(label_names))}, f, indent=2)

print("\n✔️ Artefactos guardados en:", ART)
print(" - complete_model_thresholded_bundle.joblib")
print(" - complete_label_names.csv")
print(" - complete_feature_cols.csv")
print(" - complete_thresholds.json")


In [ ]:
# === EVALUACIÓN ===
from sklearn.metrics import (
    accuracy_score, hamming_loss, jaccard_score,
    precision_recall_fscore_support, roc_auc_score, average_precision_score,
    multilabel_confusion_matrix, f1_score # Import f1_score here
)
import json
import numpy as np
import pandas as pd

print("\n" + "="*60)
print("EVALUACIÓN MULTIETIQUETA – RESUMEN GLOBAL")
print("="*60)

# Get predictions from the complete_model
y_pred = complete_model.predict(X_test)

# Attempt to get probabilities for AUC/AP if the estimator allows it
y_score = None
try:
    # Check if the model is a OneVsRestClassifier and its estimators have predict_proba
    if hasattr(complete_model, "estimators_") and all(hasattr(est, "predict_proba") for est in complete_model.estimators_):
        y_score = np.column_stack([est.predict_proba(X_test)[:,1] for est in complete_model.estimators_])
    elif hasattr(complete_model, "predict_proba"):
         y_score = complete_model.predict_proba(X_test)

except Exception as e:
    print(f"[AVISO] No se pudieron obtener probabilidades: {e}")

# Métricas globales
subset_acc   = (y_pred == y_test).all(axis=1).mean()
hamm_loss    = hamming_loss(y_test, y_pred)
f1_micro     = f1_score(y_test, y_pred, average="micro", zero_division=0)
f1_macro2    = f1_score(y_test, y_pred, average="macro", zero_division=0)  # ya mostrado antes, lo repetimos aquí
f1_weighted  = f1_score(y_test, y_pred, average="weighted", zero_division=0)
prec_micro   = precision_recall_fscore_support(y_test, y_pred, average="micro", zero_division=0)[0]
rec_micro    = precision_recall_fscore_support(y_test, y_pred, average="micro", zero_division=0)[1]
jaccard_samples = jaccard_score(y_test, y_pred, average="samples", zero_division=0) # Corrected variable name
jaccard_macro   = jaccard_score(y_test, y_pred, average="macro", zero_division=0)   # Corrected variable name


print(f"Subset accuracy (exact match): {subset_acc:0.4f}")
print(f"Hamming loss:                 {hamm_loss:0.4f}")
print(f"F1 micro:                     {f1_micro:0.4f}")
print(f"F1 macro:                     {f1_macro2:0.4f}")
print(f"F1 weighted:                  {f1_weighted:0.4f}")
print(f"Precisión micro:              {prec_micro:0.4f}")
print(f"Recall micro:                 {rec_micro:0.4f}")
print(f"Jaccard (samples):            {jaccard_samples:0.4f}")
print(f"Jaccard (macro):              {jaccard_macro:0.4f}")

# Métricas por etiqueta
per_label_rows = []
mcm = multilabel_confusion_matrix(y_test, y_pred)
# Use y_complete.columns to get the correct label names
label_names_eval = y_complete.columns.tolist()
for i, label_name in enumerate(label_names_eval):
    tn, fp, fn, tp = mcm[i].ravel()

    # Precisión/Recall/F1 por etiqueta
    p, r, f1, support = precision_recall_fscore_support(
        y_test[:, i], y_pred[:, i], average="binary", zero_division=0
    )

    # Handle support being None explicitly
    support_int = int(support) if support is not None else 0

    # ROC-AUC y AP si hay probabilidades y ambas clases están presentes
    roc_auc = None
    ap = None
    if y_score is not None:
        # Comprobamos que en test hay positivos y negativos
        if len(np.unique(y_test[:, i])) == 2:
            try:
                roc_auc = roc_auc_score(y_test[:, i], y_score[:, i])
            except Exception:
                roc_auc = None
            try:
                ap = average_precision_score(y_test[:, i], y_score[:, i])
            except Exception:
                ap = None

    per_label_rows.append({
        "label": label_name,
        "support": support_int, # Use the handled support value
        "tp": int(tp), "fp": int(fp), "fn": int(fn), "tn": int(tn),
        "precision": float(p), "recall": float(r), "f1": float(f1),
        "roc_auc": (None if roc_auc is None else float(roc_auc)),
        "avg_precision": (None if ap is None else float(ap)),
    })

per_label_df = pd.DataFrame(per_label_rows).sort_values("label")
print("\n" + "-"*60)
print("Métricas por etiqueta (top 10 por menor F1 para priorizar mejoras):")
# Ensure that there are at least 10 rows before trying to print the head of 10
if len(per_label_df) >= 10:
  print(per_label_df.sort_values("f1").head(10).to_string(index=False))
else:
  print(per_label_df.sort_values("f1").to_string(index=False))


# Guardado de resultados
overall_metrics = {
    "subset_accuracy": subset_acc,
    "hamming_loss": hamm_loss,
    "f1_micro": f1_micro,
    "f1_macro": f1_macro2,
    "f1_weighted": f1_weighted,
    "precision_micro": prec_micro,
    "recall_micro": rec_micro,
    "jaccard_samples": jaccard_samples,
    "jaccard_macro": jaccard_macro,
    "num_labels": int(y_complete.shape[1]), # Use y_complete for total number of labels
    "num_samples_test": int(y_test.shape[0])
}
with open(f"{ART}/eval_overall.json", "w") as f:
    json.dump(overall_metrics, f, indent=2)

per_label_df.to_csv(f"{ART}/eval_per_label.csv", index=False)

print("\nArchivos de evaluación guardados en:", ART)
print(" - eval_overall.json (métricas globales)")
print(" - eval_per_label.csv (métricas por etiqueta)")
print("="*60)

In [ ]:
# === INFERENCIA: umbrales por etiqueta + exclusiones + diagnóstico ===
import os, json, numpy as np, pandas as pd, joblib
from sklearn.metrics import f1_score

BASE = "/content/drive/MyDrive/asistente_coreografico"
ART  = f"{BASE}/artifacts"
PROC = f"{BASE}/data/processed/aistpp"
CSV  = f"{PROC}/features_coreograficos.csv"

# === Cargar artefactos del preprocesado y modelo ===
imp          = joblib.load(f"{ART}/imputer.joblib")
scaler       = joblib.load(f"{ART}/scaler.joblib")
model        = joblib.load(f"{ART}/model_ovr_logreg.joblib")
feature_cols = pd.read_csv(f"{ART}/feature_cols.csv", header=None)[0].tolist()
label_names  = pd.read_csv(f"{ART}/label_names.csv", header=None)[0].tolist()

# --- Mapeo de sugerencias (puedes ampliar este diccionario) ---
label_to_text = {
    "amplitud_baja":      "Aumentar amplitud (extensiones y desplazamientos más amplios).",
    "variedad_baja":      "Introducir cambios de dirección y diagonales.",
    "mucha_simetria":     "Explorar asimetrías entre izquierda y derecha.",
    "poca_simetria":      "Equilibrar con momentos de simetría.",
    "fluidez_baja":       "Trabajar transiciones para mayor continuidad/fluidez.",
    "poco_rango_niveles": "Usar niveles alto y bajo además del medio.",
}

# --- Grupos mutuamente excluyentes (mantener solo la más probable) ---
mutex_groups = [
    ["amplitud_baja", "amplitud_excesiva"],
    ["poco_rango_niveles", "exceso_rango_niveles"],
    ["mucha_simetria", "poca_simetria"],
    ["variedad_baja", "variedad_excesiva"],
    ["sincronia_baja", "sincronia_alta"],
    ["unisono_bajo", "unisono_alto"],
    ["formacion_compacta", "formacion_dispers"],
    ["area_formacion_peq", "area_formacion_gran"],
    ["poco_tiempo_suelo", "mucho_tiempo_suelo"],
]

# === Utilidades ===
def _as_dataframe_one(feats_dict, cols):
    row = {c: feats_dict.get(c, np.nan) for c in cols}
    return pd.DataFrame([row], columns=cols)

def _predict_proba_matrix(Xm):
    """Devuelve matriz de probabilidades shape (n_samples, n_labels)."""
    # OneVsRest(LogisticRegression) -> estimators_ con predict_proba
    if hasattr(model, "estimators_"):
        cols = []
        for est in model.estimators_:
            if hasattr(est, "predict_proba"):
                cols.append(est.predict_proba(Xm)[:, 1])
            else:
                # Fallback con decision_function -> sigmoide
                from scipy.special import expit
                cols.append(expit(est.decision_function(Xm)))
        return np.column_stack(cols)
    # Fallback genérico
    if hasattr(model, "predict_proba"):
        yps = model.predict_proba(Xm)
        # Algunos devuelven lista de arrays por etiqueta
        if isinstance(yps, list):
            return np.column_stack([p[:,1] for p in yps])
        return yps
    if hasattr(model, "decision_function"):
        from scipy.special import expit
        return expit(model.decision_function(Xm))
    # Último recurso: binario a float
    return model.predict(Xm).astype(float)

def _enforce_mutex(selected_labels, prob_series):
    """Elimina conflictos en grupos mutuamente excluyentes, dejando la etiqueta con mayor prob."""
    selected = set(selected_labels)
    for group in mutex_groups:
        # Intersección con seleccionadas
        present = [g for g in group if g in selected]
        if len(present) > 1:
            # Mantener la de mayor probabilidad
            best = max(present, key=lambda k: prob_series.get(k, 0.0))
            for g in present:
                if g != best:
                    selected.discard(g)
    return list(selected)

# === Cargar o aprender umbrales por etiqueta (maximizando F1) ===
TH_FILE = f"{ART}/thresholds_f1.json"

def _recreate_labels_proxy(df):
    """Reconstruye etiquetas proxy como en entrenamiento (cuantiles 20/80, low/high)."""
    # Columnas candidatas conocidas
    candidates = ["amplitud_x","amplitud_y","velocidad_media","simetria",
                  "nivel_rango","variedad_direcciones","frames","joints","dims",
                  "velocidad_std","vel_std","disp_total","trayectoria_longitud",
                  "gm_sync","sync","gm_unison","unison","gm_spread","spread",
                  "formation_area","gm_formation_area","onbeat_ratio","bpm",
                  "alineacion_postural","equilibrio","curvatura_trayectoria",
                  "tiempo_en_suelo","amp_x","amp_y","simetria_raw","variedad_dir"]
    exist = [c for c in candidates if c in df.columns]
    q20 = df[exist].quantile(0.20)
    q80 = df[exist].quantile(0.80)

    def low(col):  return (df[col] <= q20[col]).astype(int)
    def high(col): return (df[col] >= q80[col]).astype(int)

    labels_spec = {
        "amplitud_baja":       (["amplitud_x","amplitud_y","amp_x","amp_y"], "low_any"),
        "amplitud_excesiva":   (["amplitud_x","amplitud_y","amp_x","amp_y"], "high_any"),
        "poco_rango_niveles":  (["nivel_rango"], "low"),
        "exceso_rango_niveles":(["nivel_rango"], "high"),
        "fluidez_baja":        (["velocidad_media","vel_media"], "low"),
        "velocidad_excesiva":  (["velocidad_media","vel_media"], "high"),
        "dinamica_monotona":   (["velocidad_std","vel_std"], "low"),
        "dinamica_erratica":   (["velocidad_std","vel_std"], "high"),
        "poca_simetria":       (["simetria","simetria_raw"], "low"),
        "mucha_simetria":      (["simetria","simetria_raw"], "high"),
        "variedad_baja":       (["variedad_direcciones","variedad_dir"], "low"),
        "variedad_excesiva":   (["variedad_direcciones","variedad_dir"], "high"),
        "ocupacion_reducida":  (["disp_total","trayectoria_longitud"], "low"),
        "traslacion_excesiva": (["disp_total","trayectoria_longitud"], "high"),
        "frase_corta":         (["frames"], "low"),
        "sincronia_baja":      (["gm_sync","sync"], "low"),
        "sincronia_alta":      (["gm_sync","sync"], "high"),
        "unisono_bajo":        (["gm_unison","unison"], "low"),
        "unisono_alto":        (["gm_unison","unison"], "high"),
        "formacion_compacta":  (["gm_spread","spread"], "low"),
        "formacion_dispers":   (["gm_spread","spread"], "high"),
        "area_formacion_peq":  (["formation_area","gm_formation_area"], "low"),
        "area_formacion_gran": (["formation_area","gm_formation_area"], "high"),
        "alineacion_musical_baja": (["onbeat_ratio"], "low"),
        "alineacion_musical_alta": (["onbeat_ratio"], "high"),
        "desfase_tempo_rapido":    (["bpm","velocidad_media","vel_media"], "bpm_high_vel_low"),
        "desfase_tempo_lento":     (["bpm","velocidad_media","vel_media"], "bpm_low_vel_high"),
        "alineacion_postural_baja": (["alineacion_postural"], "low"),
        "alineacion_postural_alta": (["alineacion_postural"], "high"),
        "equilibrio_bajo":          (["equilibrio"], "low"),
        "equilibrio_alto":          (["equilibrio"], "high"),
        "curvatura_baja":           (["curvatura_trayectoria"], "low"),
        "curvatura_alta":           (["curvatura_trayectoria"], "high"),
        "poco_tiempo_suelo":        (["tiempo_en_suelo"], "low"),
        "mucho_tiempo_suelo":       (["tiempo_en_suelo"], "high"),
    }

    ys = {}
    for name, (cols, mode) in labels_spec.items():
        missing = [c for c in cols if c not in df.columns]
        if missing:
            continue
        if mode == "low":
            ys[name] = low(cols[0])
        elif mode == "high":
            ys[name] = high(cols[0])
        elif mode == "low_any":
            v = np.zeros(len(df), dtype=int)
            for c in cols:
                v = np.maximum(v, low(c).values)
            ys[name] = v
        elif mode == "high_any":
            v = np.zeros(len(df), dtype=int)
            for c in cols:
                v = np.maximum(v, high(c).values)
            ys[name] = v
        elif mode == "bpm_high_vel_low":
            if "bpm" in df.columns and (("velocidad_media" in df.columns) or ("vel_media" in df.columns)):
                vlow = df["velocidad_media"] if "velocidad_media" in df.columns else df["vel_media"]
                ys[name] = ((df["bpm"] >= df["bpm"].quantile(0.80)) & (vlow <= vlow.quantile(0.20))).astype(int)
        elif mode == "bpm_low_vel_high":
            if "bpm" in df.columns and (("velocidad_media" in df.columns) or ("vel_media" in df.columns)):
                vlow = df["velocidad_media"] if "velocidad_media" in df.columns else df["vel_media"]
                ys[name] = ((df["bpm"] <= df["bpm"].quantile(0.20)) & (vlow >= vlow.quantile(0.80))).astype(int)
    if not ys:
        raise RuntimeError("No se pudieron recrear etiquetas proxy para aprender umbrales.")
    Y = pd.DataFrame(ys)
    # Asegurar mismo orden que label_names (filtrando a las que existan)
    kept = [l for l in label_names if l in Y.columns]
    return Y[kept]

def _learn_thresholds_f1(df, save_path=TH_FILE):
    """Aprende umbral por etiqueta maximizando F1 usando el CSV y el modelo actual."""
    # Preparar X
    Xraw = df[feature_cols].copy()
    Ximp = imp.transform(Xraw)
    X    = scaler.transform(Ximp)
    # Probabilidades
    probs = _predict_proba_matrix(X)  # shape (N, L)
    # Etiquetas proxy
    Y = _recreate_labels_proxy(df)    # shape (N, L') con subset de labels
    # Alinear dimensiones si falta alguna etiqueta
    kept_labels = list(Y.columns)
    kept_idx = [label_names.index(l) for l in kept_labels]
    probs = probs[:, kept_idx]

    thresholds = {}
    for j, lbl in enumerate(kept_labels):
        y_true = Y.values[:, j].astype(int)
        p = probs[:, j]
        # Si todas son 0/1 constantes, usar 0.5 por defecto
        if len(np.unique(y_true)) < 2:
            thresholds[lbl] = 0.5
            continue
        # Buscar umbral que maximiza F1
        # Probamos umbrales únicos de p + bordes
        cand = np.unique(np.r_[0.0, p, 1.0])
        best_f1, best_t = -1.0, 0.5
        for t in cand:
            yhat = (p >= t).astype(int)
            f1 = f1_score(y_true, yhat, zero_division=0)
            if f1 > best_f1:
                best_f1, best_t = f1, t
        thresholds[lbl] = float(best_t)
    # Añadir las etiquetas sin proxy (usar 0.5)
    for lbl in label_names:
        if lbl not in thresholds:
            thresholds[lbl] = 0.5
    with open(save_path, "w") as f:
        json.dump(thresholds, f, indent=2)
    print(f"[INFO] Umbrales por etiqueta guardados en {save_path}")
    return thresholds

def _load_or_learn_thresholds():
    if os.path.exists(TH_FILE):
        with open(TH_FILE, "r") as f:
            th = json.load(f)
        # Asegurar que están todas las labels
        for lbl in label_names:
            th.setdefault(lbl, 0.5)
        return th
    # Aprender al vuelo desde el CSV
    df_all = pd.read_csv(CSV)
    return _learn_thresholds_f1(df_all, save_path=TH_FILE)

label_thresholds = _load_or_learn_thresholds()

def inferir_desde_features_dict(
    feats: dict,
    global_fallback_k=3,
    global_fallback_min=0.25
):
    """
    - Aplica umbral ESPECÍFICO por etiqueta (aprendido o 0.5).
    - Si ninguna supera su umbral -> fallback: top-k ≥ global_fallback_min (o top-1).
    - Resuelve exclusiones mutuamente excluyentes conservando la etiqueta con mayor probabilidad.
    Devuelve: (pred_labels, sugerencias, df_diag_ordenado)
    """
    df1   = _as_dataframe_one(feats, feature_cols)
    X_imp = imp.transform(df1)
    X     = scaler.transform(X_imp)
    probs = _predict_proba_matrix(X)[0]  # vector (L,)
    prob_series = pd.Series(probs, index=label_names)

    # Decisión por umbral específico
    decided = []
    for lbl, p in prob_series.items():
        t = label_thresholds.get(lbl, 0.5)
        if p >= t:
            decided.append(lbl)

    # Fallback si nada supera su umbral
    strategy = "per-label thresholds"
    if len(decided) == 0:
        strategy = f"fallback(top-{global_fallback_k}, min={global_fallback_min})"
        topk = prob_series.sort_values(ascending=False)
        decided = topk[topk >= global_fallback_min].head(global_fallback_k).index.tolist()
        if len(decided) == 0 and len(topk) > 0:
            decided = [topk.index[0]]

    # Resolver exclusiones
    selected = _enforce_mutex(decided, prob_series.to_dict())

    # Construir diagnóstico ordenado
    rows = []
    for lbl in label_names:
        rows.append({
            "label": lbl,
            "prob": float(prob_series[lbl]),
            "thr": float(label_thresholds.get(lbl, 0.5)),
            "passes_thr": lbl in decided,
            "selected_final": lbl in selected,
            "group": next((str(g) for g in mutex_groups if lbl in g), "")
        })
    diag = pd.DataFrame(rows).sort_values("prob", ascending=False)

    # Sugerencias ordenadas por prob
    sugerencias = [label_to_text.get(lbl, lbl) for lbl in sorted(selected, key=lambda k: prob_series[k], reverse=True)]

    # Imprimir resumen legible
    print("\n=== PROBABILIDADES (top 12) con umbral por etiqueta ===")
    print(diag.head(12)[["label","prob","thr","passes_thr","selected_final","group"]].to_string(index=False,
          formatters={"prob": lambda v: f"{v:0.3f}", "thr": lambda v: f"{v:0.3f}"}))
    print(f"\nEstrategia: {strategy}")
    print("Etiquetas finales:", selected)
    print("Sugerencias:")
    for s in sugerencias:
        print(" -", s)

    return selected, sugerencias, diag

# === Ejemplo con un registro del CSV (fila 0) ===
df_all = pd.read_csv(CSV)
row0 = df_all.iloc[0].to_dict()
feats0 = {c: row0[c] for c in feature_cols if c in row0}

preds, suggs, diag = inferir_desde_features_dict(
    feats0,
    global_fallback_k=3,
    global_fallback_min=0.25
)


In [ ]:
# === GUARDAR MODELO EN /models ===

import os, json
import joblib
import pandas as pd
import sklearn
from sklearn.pipeline import Pipeline as SkPipeline

# Rutas base
BASE        = "/content/drive/MyDrive/asistente_coreografico"
ART         = f"{BASE}/artifacts"
MODELS_DIR  = f"{BASE}/models"
os.makedirs(MODELS_DIR, exist_ok=True)

# Cargar objetos desde memoria o desde /artifacts
try:
    imp
    scaler
    clf
except NameError:
    # Si no están en el workspace actual, cargamos desde disco
    imp_path    = f"{ART}/imputer.joblib"
    scaler_path = f"{ART}/scaler.joblib"
    clf_path    = f"{ART}/model_ovr_logreg.joblib"
    assert os.path.exists(imp_path),    f"Falta: {imp_path}"
    assert os.path.exists(scaler_path), f"Falta: {scaler_path}"
    assert os.path.exists(clf_path),    f"Falta: {clf_path}"
    imp    = joblib.load(imp_path)
    scaler = joblib.load(scaler_path)
    clf    = joblib.load(clf_path)

#  Cargar columnas de features y nombres de etiquetas
try:
    feature_cols
except NameError:
    feat_path = f"{ART}/feature_cols.csv"
    assert os.path.exists(feat_path), f"Falta: {feat_path}"
    feature_cols = pd.read_csv(feat_path, header=None)[0].tolist()

try:
    label_names
except NameError:
    labels_path = f"{ART}/label_names.csv"
    assert os.path.exists(labels_path), f"Falta: {labels_path}"
    label_names = pd.read_csv(labels_path, header=None)[0].tolist()

# Construir Pipeline y guardar
pipe = SkPipeline([
    ("imputer", imp),     # SimpleImputer ya ajustado
    ("scaler",  scaler),  # RobustScaler ya ajustado
    ("clf",     clf)      # OneVsRest(LogisticRegression) ya ajustado
])

pipe_path = f"{MODELS_DIR}/pipeline_ovr_logreg.joblib"
joblib.dump(pipe, pipe_path)
print(f"✔️ Pipeline guardado en: {pipe_path}")

# Metadatos del pipeline
meta = {
    "sklearn_version": sklearn.__version__,
    "feature_cols": list(feature_cols),
    "label_names": list(label_names),
    "notes": "Pipeline con SimpleImputer + RobustScaler + OneVsRest(LogReg) entrenado."
}
meta_path = f"{MODELS_DIR}/pipeline_metadata.json"
with open(meta_path, "w", encoding="utf-8") as f:
    json.dump(meta, f, indent=2, ensure_ascii=False)
print(f"✔️ Metadatos guardados en: {meta_path}")

# Ejemplo de carga rápida (opcional)
try:
    _ = X_raw  # si existe en el entorno actual
except NameError:
    # Si no tienes X_raw en memoria, puedes construirlo así (descomenta y ajusta):
    # CSV = f"{BASE}/data/processed/aistpp/features_coreograficos.csv"
    # assert os.path.exists(CSV), f"No existe el CSV de features: {CSV}"
    # df = pd.read_csv(CSV)
    # feature_cols_to_use = ['amplitud_x', 'amplitud_y', 'amplitud_z', 'velocidad_media',
    #                       'simetria', 'nivel_rango', 'variedad_direcciones', 'frames']
    # feature_cols_to_use = [col for col in feature_cols_to_use if col in df.columns]
    # X_raw = df[feature_cols_to_use].copy()
    X_raw = None


pipe = joblib.load(pipe_path)

def _align_features_df(df_in: pd.DataFrame, required_cols: list) -> pd.DataFrame:
    """Devuelve un DF con exactamente las columnas usadas en entrenamiento, en el mismo orden.
       Columnas faltantes se rellenan con NaN (las tratará el Imputer)."""
    df = pd.DataFrame(index=df_in.index)
    for c in required_cols:
        if c in df_in.columns:
            df[c] = pd.to_numeric(df_in[c], errors="coerce")
        else:
            df[c] = np.nan
    return df[required_cols]

# Si no tienes X_raw en memoria, carga el CSV y alinéalo a feature_cols
if 'X_raw' not in globals() or X_raw is None:
    CSV = f"{BASE}/data/processed/aistpp/features_coreograficos.csv"
    assert os.path.exists(CSV), f"No existe el CSV de features: {CSV}"
    df_features = pd.read_csv(CSV)
    X_raw = _align_features_df(df_features, feature_cols)
else:
    # Si ya existe X_raw, asegúrate de que es DataFrame y alinéalo
    if isinstance(X_raw, np.ndarray):
        # Si te pasaron un array, lo convertimos a DF sin nombres => no es válido para verificación de columnas
        # Carga el CSV para obtener columnas y reemplaza X_raw por el dataset alineado
        CSV = f"{BASE}/data/processed/aistpp/features_coreograficos.csv"
        df_features = pd.read_csv(CSV)
        X_raw = _align_features_df(df_features, feature_cols)
    else:
        X_raw = _align_features_df(X_raw, feature_cols)

# Predicción ya con columnas correctas
y_pred  = pipe.predict(X_raw)
# predict_proba disponible con OneVsRest(LogReg)
try:
    y_proba = pipe.predict_proba(X_raw)
except Exception:
    # Fallback si no existiera predict_proba
    if hasattr(pipe, "decision_function"):
        logits = pipe.decision_function(X_raw)
        y_proba = 1.0 / (1.0 + np.exp(-logits))
    else:
        y_proba = None

print("Pred shape:", y_pred.shape, "| Proba shape:", None if y_proba is None else y_proba.shape)
print("✔️ Inferencia realizada con columnas alineadas:", feature_cols)

In [ ]:
import json
from sklearn.metrics import f1_score
import os
import pandas as pd
import numpy as np
import joblib


BASE = "/content/drive/MyDrive/asistente_coreografico"
ART  = f"{BASE}/artifacts"
os.makedirs(ART, exist_ok=True)

# Cargar artefactos de entrenamiento
imp          = joblib.load(f"{ART}/imputer.joblib")
scaler       = joblib.load(f"{ART}/scaler.joblib")
model        = joblib.load(f"{ART}/model_ovr_logreg.joblib")
feature_cols = pd.read_csv(f"{ART}/feature_cols.csv", header=None)[0].tolist()
label_names  = pd.read_csv(f"{ART}/label_names.csv", header=None)[0].tolist()

label_to_text = {
    "amplitud_baja":      "Aumentar amplitud (extensiones y desplazamientos más amplios).",
    "variedad_baja":      "Introducir cambios de dirección y diagonales.",
    "mucha_simetria":     "Explorar asimetrías entre izquierda y derecha.",
    "poca_simetria":      "Equilibrar con momentos de simetría.",
    "fluidez_baja":       "Trabajar transiciones para mayor continuidad/fluidez.",
    "poco_rango_niveles": "Usar niveles alto y bajo además del medio.",
}

def get_pairs(J):
    # COCO-17 (YOLO-Pose): hombros/codos/muñecas/caderas/rodillas/tobillos
    return [(5,6),(7,8),(9,10),(11,12),(13,14),(15,16)]

def center_of_mass(K):
    # COCO: caderas 11 y 12
    T,J,D = K.shape
    if J >= 13:
        return (K[:,11,:] + K[:,12,:]) / 2
    return K.mean(axis=1)

def features_coreograficos(K):
    T,J,D = K.shape
    amp_mean = (np.nanmax(K,axis=0) - np.nanmin(K,axis=0)).mean(axis=0)
    amp_x = float(amp_mean[0]); amp_y = float(amp_mean[1])
    amp_z = float(amp_mean[2]) if D==3 else np.nan
    vel = np.linalg.norm(np.diff(K,axis=0),axis=2)
    vel_media = float(np.nanmean(vel)) if vel.size else 0.0
    pair_vals=[]
    for a,b in get_pairs(J):
        if a<J and b<J:
            diff = K[:,a,:]-K[:,b,:]
            if diff.ndim > 1:
                d = np.linalg.norm(diff,axis=1).mean() # Correct axis for (T, D) -> (T,)
                pair_vals.append(np.nanmean(d))
            elif diff.ndim == 1: # Handle case where diff is 1D (T,)
                 d = np.linalg.norm(diff).mean()
                 pair_vals.append(np.nanmean(d))
    sim_raw = float(np.nanmean(pair_vals)) if pair_vals else np.nan
    com = center_of_mass(K)
    nivel_p10 = float(np.nanpercentile(com[:,1],10))
    nivel_p90 = float(np.nanpercentile(com[:,1],90))
    nivel_rango = float(nivel_p10 - nivel_p90)
    disp = np.diff(com,axis=0)
    dirs = np.arctan2(disp[:,1], disp[:,0] + 1e-9)
    cambios = np.abs(np.diff(dirs))
    variedad_dir = float(np.nanmean(cambios)) if len(cambios)>0 else 0.0
    return {
        "amp_x": amp_x, "amp_y": amp_y, "amp_z": amp_z,
        "vel_media": vel_media,
        "simetria_raw": sim_raw,
        "nivel_rango": nivel_rango,
        "variedad_dir": variedad_dir,
        "frames": int(T), "joints": int(J), "dims": int(D)
    }

def inferir_desde_features_dict(feats: dict):
    x = np.array([[feats.get(c, np.nan) for c in feature_cols]], dtype=float)
    x = scaler.transform(imp.transform(x))
    yhat = model.predict(x)[0]
    pred_labels = [label_names[i] for i,v in enumerate(yhat) if v==1]
    sugerencias = [label_to_text.get(lbl, lbl) for lbl in pred_labels]
    return pred_labels, sugerencias

In [ ]:
import os, glob, shutil
import numpy as np
import pandas as pd
import cv2
import numpy as np
from ultralytics import YOLO

# Carpeta donde quieres guardar los pesos .pt
BASE = "/content/drive/MyDrive/asistente_coreografico"
MODELS_DIR = f"{BASE}/models"
os.makedirs(MODELS_DIR, exist_ok=True)

def _resolve_downloaded_weight_path(model_obj, model_name: str):
    """
    Intenta localizar en disco el .pt que YOLO descargó al crear el modelo.
    """
    # 1) Atributo típico en Ultralytics v8
    p = getattr(model_obj, "ckpt_path", None)
    if isinstance(p, str) and os.path.exists(p):
        return p

    # 2) Búsqueda en caches comunes
    candidates_roots = [
        os.path.expanduser("~/.cache/ultralytics"),
        os.path.expanduser("~/.cache/torch/hub"),
        os.path.expanduser("~/.cache"),
        "/root/.cache",
        "/content",
    ]
    for root in candidates_roots:
        try:
            for hit in glob.glob(os.path.join(root, "**", model_name), recursive=True):
                if os.path.isfile(hit):
                    return hit
        except Exception:
            pass

    # 3) No encontrado
    return None

def _load_or_download_yolo(model_name: str, save_dir: str):
    """
    Si el peso existe en save_dir, lo carga desde ahí.
    Si no existe, instancia YOLO(model_name) para forzar descarga,
    luego copia el .pt descargado a save_dir y devuelve el modelo cargado desde ahí.
    """
    target_path = os.path.join(save_dir, model_name)
    if os.path.exists(target_path):
        # Cargar directamente desde el archivo ya guardado en Drive
        return YOLO(target_path), target_path

    # Forzar descarga (YOLO gestiona la descarga al instanciar con el nombre)
    tmp_model = YOLO(model_name)
    resolved = _resolve_downloaded_weight_path(tmp_model, model_name)

    if resolved and os.path.exists(resolved):
        # Copiar a tu carpeta de modelos en Drive
        shutil.copy2(resolved, target_path)
        print(f"✔️ Peso descargado y copiado a: {target_path}")
        # Cargar desde la ruta destino (para que desde ahora siempre use ese .pt)
        return YOLO(target_path), target_path
    else:
        # Si no logramos resolver el archivo, seguimos usando el objeto ya cargado
        print(" No se pudo resolver la ruta física del .pt descargado; "
              "se usará el modelo en memoria. (Se intentó buscar en caches comunes).")
        return tmp_model, None

def video_to_keypoints_yolo(
    video_path,
    model_name="yolov8n-pose.pt",
    conf=0.25,
    iou=0.5,
    stride=1,
    max_people=1
):
    """
    Extrae keypoints COCO-17 (x,y,score) por frame usando YOLOv8 Pose.
    Devuelve:
      - kpts_main: (N, 17, 3) de la persona principal
      - fps: frames por segundo del vídeo
    """
    assert os.path.exists(video_path), f"No existe el video: {video_path}"

    # Cargar/descargar y guardar en MODELS_DIR
    model, weight_path = _load_or_download_yolo(model_name, MODELS_DIR)
    if weight_path:
        print(f"Usando pesos desde: {weight_path}")

    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise RuntimeError(f"No se pudo abrir el video: {video_path}")
    fps = cap.get(cv2.CAP_PROP_FPS) or 30.0

    frames = []
    idx = 0
    while True:
        ok, frame = cap.read()
        if not ok:
            break
        if (idx % max(1, stride)) != 0:
            idx += 1
            continue

        # Inferencia por frame
        res = model.predict(source=frame, conf=conf, iou=iou, verbose=False)
        kpts_list = []
        if len(res) and hasattr(res[0], "keypoints") and res[0].keypoints is not None:
            xy = res[0].keypoints.xy
            cf = res[0].keypoints.conf
            xy = xy.cpu().numpy() if hasattr(xy, "cpu") else np.asarray(xy)
            if cf is None:
                cf = np.ones_like(xy[..., 0])
            else:
                cf = cf.cpu().numpy() if hasattr(cf, "cpu") else np.asarray(cf)

            for i in range(xy.shape[0]):
                # (17,2) + (17,1) -> (17,3)
                k = np.concatenate([xy[i], cf[i][..., None]], axis=-1)
                kpts_list.append(k)

        if kpts_list:
            # elige persona principal por mayor score medio
            scores = []
            for k in kpts_list:
                # Calcular score medio, manejando posibles NaN
                valid_scores = k[:, 2][~np.isnan(k[:, 2])]
                if len(valid_scores) > 0:
                    scores.append(np.mean(valid_scores))
                else:
                    scores.append(-np.inf)  # Si todos son NaN, usar -infinito

            # Verificar que haya al menos un score válido
            if not all(score == -np.inf for score in scores):
                main = kpts_list[np.argmax(scores)]
                frames.append(main)
            else:
                # Si todos los scores son inválidos, agregar array de NaN
                frames.append(np.full((17, 3), np.nan))
        else:
            # Si no hay detecciones, agregar array de NaN
            frames.append(np.full((17, 3), np.nan))

        idx += 1

    cap.release()
    if not frames:
        return None, fps
    return np.stack(frames, axis=0), fps


# --- EJEMPLO DE USO ---
demo_path = f"{BASE}/demo/solo.mp4"
kpts, fps = video_to_keypoints_yolo(demo_path, model_name="yolov8n-pose.pt", conf=0.25, stride=1)
print("FPS:", fps)
print("Keypoints shape:", None if kpts is None else kpts.shape)

In [ ]:
# Selecciona una sola persona por frame (la de mayor área de bounding box).
# Devuelve K con forma (T, 17, 2) en píxeles.
# Si no detecta a nadie en un frame, deja NaN para ese frame (luego interpolamos).

import cv2, numpy as np
from ultralytics import YOLO

def video_to_keypoints_yolo(video_path, model_name="yolov8n-pose.pt", conf=0.25, stride=1):
    """
    Devuelve K: (T, 17, 2) en píxeles (x,y) con NaNs si no hay detección.
    Selecciona la persona con mayor caja por frame.
    """
    model = YOLO(model_name)  # descarga automática del modelo
    cap = cv2.VideoCapture(video_path)
    assert cap.isOpened(), f"No se pudo abrir el vídeo: {video_path}"

    T = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    W = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    H = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    FPS = cap.get(cv2.CAP_PROP_FPS) or 30.0

    J = 17  # COCO-17 en YOLO-Pose
    K = np.full((T, J, 2), np.nan, dtype=np.float32)

    t = 0
    while True:
        ok, frame = cap.read()
        if not ok:
            break
        if t % stride != 0:
            t += 1
            continue

        # Predicción
        res = model.predict(frame, conf=conf, verbose=False)[0]
        if res.boxes is not None and len(res.boxes) > 0 and res.keypoints is not None:
            # Elegir persona con caja más grande
            boxes = res.boxes.xyxy.cpu().numpy()  # (N,4)
            areas = (boxes[:,2]-boxes[:,0]) * (boxes[:,3]-boxes[:,1])
            idx = int(np.argmax(areas))
            kps = res.keypoints.xy[idx].cpu().numpy()  # (17,2) en píxeles
            # Algunos modelos devuelven flotantes fuera de imagen; clamp suave
            kps[:,0] = np.clip(kps[:,0], 0, W-1)
            kps[:,1] = np.clip(kps[:,1], 0, H-1)
            K[t] = kps

        t += 1

    cap.release()
    return K, FPS


In [ ]:
# ===== limpieza NaNs + features + overlay + EJECUCIÓN =====
import os, json
import numpy as np
import pandas as pd
import cv2
import joblib

# --- Rutas base ---
BASE = "/content/drive/MyDrive/asistente_coreografico"
ART  = f"{BASE}/artifacts"
OUT  = f"{BASE}/data/processed/aistpp"
os.makedirs(OUT, exist_ok=True)

# --- Utilidades NaN ---
def interpolate_nan_1d(y):
    y = y.astype(np.float32)
    T = len(y); idx = np.arange(T)
    mask = np.isfinite(y)
    if mask.sum() == 0: return y
    last = np.nan
    for i in range(T):
        if not np.isfinite(y[i]) and np.isfinite(last): y[i] = last
        else: last = y[i]
    last = np.nan
    for i in range(T-1, -1, -1):
        if not np.isfinite(y[i]) and np.isfinite(last): y[i] = last
        else: last = y[i]
    mask = np.isfinite(y)
    if 2 <= mask.sum() < T:
        y[~mask] = np.interp(idx[~mask], idx[mask], y[mask])
    return y

def clean_nan_interpolate(K, min_valid_ratio=0.10):
    if K is None or len(K) == 0:
        return None, []
    Kc = K.copy()
    T,J,D = Kc.shape
    used = []
    for j in range(J):
        valid = np.isfinite(Kc[:,j,:]).all(axis=1)
        if valid.mean() < min_valid_ratio:
            Kc[:,j,:] = np.nan
            continue
        for d in range(D):
            Kc[:,j,d] = interpolate_nan_1d(Kc[:,j,d])
        used.append(j)
    if not used: return Kc, []
    return Kc[:,used,:], used

# --- Esqueleto COCO-17 (pares básicos brazo/torso/pierna) ---
COCO_EDGES = [(5,7),(7,9),(6,8),(8,10),(11,13),(13,15),
              (12,14),(14,16),(5,6),(11,12),(5,11),(6,12)]

# --- Dibujo de overlay (corrige desempaquetado x,y,score) ---
def overlay_suggestions_on_video(video_path, K, timeline, out_path):
    cap = cv2.VideoCapture(video_path)
    assert cap.isOpened(), f"No se pudo abrir el vídeo: {video_path}"
    W = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    H = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    FPS = cap.get(cv2.CAP_PROP_FPS) or 30.0
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    vw = cv2.VideoWriter(out_path, fourcc, FPS, (W,H))

    t = 0
    while True:
        ok, frame = cap.read()
        if not ok: break

        if K is not None and t < len(K):
            P = K[t]  # (J,3) -> x,y,score
            # esqueleto
            for a,b in COCO_EDGES:
                if a < P.shape[0] and b < P.shape[0]:
                    xa,ya = P[a,:2]; xb,yb = P[b,:2]
                    if np.isfinite(xa) and np.isfinite(ya) and np.isfinite(xb) and np.isfinite(yb):
                        cv2.line(frame, (int(xa),int(ya)), (int(xb),int(yb)), (255,255,255), 2)
            for j in range(P.shape[0]):
                x,y = P[j,:2]
                if np.isfinite(x) and np.isfinite(y):
                    cv2.circle(frame, (int(x),int(y)), 2, (255,255,255), -1)

        # sugerencias activas en frame t
        active = []
        for seg in timeline:
            if seg["start_f"] <= t <= seg["end_f"]:
                active = seg.get("sugerencias", [])
        y0 = 30
        for s in active[:4]:
            cv2.putText(frame, f"• {s}", (20, y0), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,0,0), 4, cv2.LINE_AA)
            cv2.putText(frame, f"• {s}", (20, y0), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,255,255), 1, cv2.LINE_AA)
            y0 += 28

        vw.write(frame); t += 1

    cap.release(); vw.release()
    return out_path

# --- Features simples desde COCO-2D (coherentes con tu set básico) ---
def features_coreograficos(Kw):
    """
    Kw: (T,J,3) con NaNs posibles
    Devuelve dict con claves usadas por tu pipeline clásico.
    """
    T,J,D = Kw.shape
    xy = Kw[:,:,:2]
    # amplitudes globales (rango global X/Y)
    amp_x = float(np.nanmax(xy[:,:,0]) - np.nanmin(xy[:,:,0])) if np.isfinite(xy[:,:,0]).any() else 0.0
    amp_y = float(np.nanmax(xy[:,:,1]) - np.nanmin(xy[:,:,1])) if np.isfinite(xy[:,:,1]).any() else 0.0
    # velocidad media (norma frame-to-frame)
    if T > 1:
        vel = np.linalg.norm(np.diff(xy, axis=0), axis=2)  # (T-1,J)
        vel_media = float(np.nanmean(vel))
    else:
        vel_media = 0.0
    # simetría (distancias medias entre pares L-R)
    pairs = [(5,6),(7,8),(9,10),(11,12),(13,14),(15,16)]
    dists = []
    for a,b in pairs:
        if a < J and b < J:
            d = np.linalg.norm(xy[:,a,:] - xy[:,b,:], axis=1)
            dists.append(np.nanmean(d))
    simetria = float(np.nanmean(dists)) if len(dists) else np.nan
    # centro de masa (cadera aprox: 11,12) y rango de niveles
    if J > 12:
        com = (xy[:,11,:] + xy[:,12,:]) / 2.0
    else:
        com = np.nanmean(xy, axis=1)
    if np.isfinite(com[:,1]).any():
        p10 = float(np.nanpercentile(com[:,1], 10))
        p90 = float(np.nanpercentile(com[:,1], 90))
        nivel_rango = float(p10 - p90)
    else:
        nivel_rango = 0.0
    # variedad direcciones (cambios de ángulo de la trayectoria del COM)
    if T > 2:
        disp = np.diff(com, axis=0)
        dirs = np.arctan2(disp[:,1], disp[:,0] + 1e-9)
        cambios = np.abs(np.diff(dirs))
        variedad_dir = float(np.nanmean(cambios)) if len(cambios)>0 else 0.0
    else:
        variedad_dir = 0.0

    return {
        "amplitud_x": amp_x,
        "amplitud_y": amp_y,
        "velocidad_media": vel_media,
        "simetria": simetria,
        "nivel_rango": nivel_rango,
        "variedad_direcciones": variedad_dir,
        "frames": int(T),
        "joints": int(J),
        "dims": int(D)
    }

# --- Inferencia con artefactos guardados (pipeline o piezas sueltas) ---
def inferir_desde_features_dict(feats: dict):
    """
    Carga pipeline_ovr_logreg.joblib si existe; si no, usa imp+scaler+clf.
    Devuelve (labels_activas, sugerencias_texto)
    """
    # Cargar metadatos
    feature_cols_path = os.path.join(ART, "feature_cols.csv")
    label_names_path  = os.path.join(ART, "label_names.csv")
    if os.path.exists(feature_cols_path):
        feat_cols = pd.read_csv(feature_cols_path, header=None)[0].tolist()
    else:
        feat_cols = list(feats.keys())
    if os.path.exists(label_names_path):
        labels_all = pd.read_csv(label_names_path, header=None)[0].tolist()
    else:
        labels_all = []

    # Preparar vector en orden correcto
    x = np.array([[feats.get(c, np.nan) for c in feat_cols]], dtype=float)

    # Intentar pipeline completo
    pipe_path = os.path.join(BASE, "models", "pipeline_ovr_logreg.joblib")
    if os.path.exists(pipe_path):
        pipe = joblib.load(pipe_path)
        yhat = pipe.predict(x)[0]
    else:
        # Piezas sueltas
        imp_path  = os.path.join(ART, "imputer.joblib")
        scl_path  = os.path.join(ART, "scaler.joblib")
        clf_path  = os.path.join(ART, "model_ovr_logreg.joblib")
        assert all(os.path.exists(p) for p in [imp_path, scl_path, clf_path]), \
            "No se encontraron artefactos de modelo en artifacts/ ni pipeline en models/."
        imp    = joblib.load(imp_path)
        scaler = joblib.load(scl_path)
        clf    = joblib.load(clf_path)
        x = scaler.transform(imp.transform(x))
        yhat = clf.predict(x)[0]

    # Mapear a texto (fallback simple si no hay mapa)
    label_to_text = {
        "amplitud_baja":      "Aumentar amplitud (extensiones y desplazamientos más amplios).",
        "variedad_baja":      "Introducir cambios de dirección y diagonales.",
        "mucha_simetria":     "Explorar asimetrías entre izquierda y derecha.",
        "poca_simetria":      "Equilibrar con momentos de simetría.",
        "fluidez_baja":       "Trabajar transiciones para mayor continuidad/fluidez.",
        "poco_rango_niveles": "Usar niveles alto y bajo además del medio.",
        "exceso_rango_niveles": "Reducir rango vertical para mayor cohesión.",
        "frase_corta":        "Frase corta: considerar repetición o desarrollo del material."
    }
    active_labels = []
    suggs = []
    names = labels_all if labels_all else [f"label_{i}" for i in range(len(yhat))]
    for i, v in enumerate(yhat):
        if v == 1:
            lbl = names[i]
            active_labels.append(lbl)
            suggs.append(label_to_text.get(lbl, f"Mejorar {lbl}"))

    return active_labels, suggs

# --- Pipeline de vídeo con YOLO (usa tu función existente de keypoints) ---
def run_inference_over_video_yolo(video_path, model_name="yolov8n-pose.pt",
                                  conf=0.25, stride=1, win_sec=5.0, hop_sec=2.5):
    # Necesita que video_to_keypoints_yolo ya esté definida en una celda anterior
    assert 'video_to_keypoints_yolo' in globals(), "Define primero video_to_keypoints_yolo(...)"
    K, FPS = video_to_keypoints_yolo(video_path, model_name=model_name, conf=conf, stride=stride)

    if K is None or len(K) == 0:
        print(" Sin detecciones válidas (K es None o vacío). Revisa conf/iluminación/escala.")
        return None

    Kc, used = clean_nan_interpolate(K, min_valid_ratio=0.10)
    if not used:
        print(" No hay articulaciones válidas tras limpieza. Prueba bajar 'conf' o un modelo más grande (yolov8s-pose).")
        return None

    win = max(1, int(round(win_sec * FPS)))
    hop = max(1, int(round(hop_sec * FPS)))
    T = len(Kc)

    rows = []; timeline=[]
    for start in range(0, T-1, hop):
        end = min(T-1, start + win)
        if end - start < max(8, int(0.25*win)): break
        Kw = Kc[start:end]
        feats = features_coreograficos(Kw)
        labels, suggs = inferir_desde_features_dict(feats)
        timeline.append({"start_f":start, "end_f":end, "sugerencias":suggs, "labels":labels})
        row = {"start_f":start, "end_f":end, "start_s":start/FPS, "end_s":end/FPS}
        row.update(feats); row["labels"]="|".join(labels); row["sugerencias"]="|".join(suggs)
        rows.append(row)

    out_csv = os.path.join(OUT, "timeline_sugerencias_yolo.csv")
    pd.DataFrame(rows).to_csv(out_csv, index=False, encoding="utf-8")

    out_mp4 = os.path.join(OUT, "video_anotado_yolo.mp4")
    overlay_suggestions_on_video(video_path, Kc, timeline, out_mp4)

    print("Listo")
    print("CSV:", out_csv)
    print("Vídeo:", out_mp4)
    return {"csv": out_csv, "video": out_mp4, "timeline": timeline}



In [ ]:
def preview_primeras_anotaciones(res, n=10):
    assert res is not None, "res es None: ¿se ejecutó run_inference_over_video_yolo?"
    filas = []

    # Intentar desde el timeline en memoria (si existe)
    if "timeline" in res and isinstance(res["timeline"], list) and len(res["timeline"]) > 0:
        tl = res["timeline"][:n]
        for i, seg in enumerate(tl):
            start_f = seg.get("start_f")
            end_f   = seg.get("end_f")
            # Si además tienes start_s/end_s en CSV, lo mostraremos abajo.
            suger = seg.get("sugerencias") or seg.get("sugerencias_single") or seg.get("sugerencias_global") or []
            labels = seg.get("labels") or []
            filas.append({
                "i": i,
                "start_f": start_f,
                "end_f": end_f,
                "labels": " · ".join(labels) if isinstance(labels, list) else str(labels),
                "sugerencias": " · ".join(suger) if isinstance(suger, list) else str(suger)
            })

    # Completar con el CSV para tener tiempos en segundos y features
    if "csv" in res and res["csv"] and os.path.exists(res["csv"]):
        df = pd.read_csv(res["csv"])
        # Columnas interesantes si existen
        cols_base = [c for c in ["start_s","end_s","labels","sugerencias",
                                 "amp_x","amp_y","vel_media","simetria_raw","nivel_rango","variedad_dir"]
                     if c in df.columns]
        df_preview = df[cols_base].head(n).copy()

        # Limpieza visual de listas en texto
        for c in ["labels","sugerencias"]:
            if c in df_preview.columns:
                df_preview[c] = df_preview[c].fillna("").astype(str).apply(
                    lambda s: " · ".join([x.strip() for x in s.split("|") if x.strip()])[:300]
                )

        print("=== Primeras ventanas (desde CSV) ===")
        display(df_preview)

        # Además, un resumen legible en texto:
        print("\nResumen (texto):")
        for i, row in df_preview.iterrows():
            st = row.get("start_s", 0.0); et = row.get("end_s", 0.0)
            print(f"[{i}] {st:6.2f}s → {et:6.2f}s")
            if "sugerencias" in df_preview.columns and str(row.get("sugerencias","")).strip():
                for s in str(row["sugerencias"]).split(" · "):
                    if s.strip():
                        print("   -", s.strip())
            elif "labels" in df_preview.columns and str(row.get("labels","")).strip():
                for lab in str(row["labels"]).split(" · "):
                    if lab.strip():
                        print("   -", lab.strip())
            print()

    elif filas:
        # Si no hay CSV, al menos muestra lo que haya en memoria
        df_mem = pd.DataFrame(filas)
        print("=== Primeras ventanas (desde res['timeline']) ===")
        display(df_mem)
        print("\nResumen (texto):")
        for _, r in df_mem.iterrows():
            print(f"[{int(r['i'])}] f{int(r['start_f'])} → f{int(r['end_f'])}")
            if str(r.get("sugerencias","")).strip():
                for s in str(r["sugerencias"]).split(" · "):
                    if s.strip():
                        print("   -", s.strip())
            elif str(r.get("labels","")).strip():
                for lab in str(r["labels"]).split(" · "):
                    if lab.strip():
                        print("   -", lab.strip())
            print()
    else:
        raise FileNotFoundError("No encontré timeline en memoria ni CSV en res['csv'].")

# --- Usa la función con tu 'res' ---
preview_primeras_anotaciones(res, n=10)

In [ ]:
BASE = "/content/drive/MyDrive/asistente_coreografico"
ART  = f"{BASE}/artifacts"
os.makedirs(ART, exist_ok=True)

# Carga artefactos (entrenados previamente con tu CSV)
imp          = joblib.load(f"{ART}/imputer.joblib")
scaler       = joblib.load(f"{ART}/scaler.joblib")
model        = joblib.load(f"{ART}/model_ovr_logreg.joblib")
feature_cols = pd.read_csv(f"{ART}/feature_cols.csv", header=None)[0].tolist()
label_names  = pd.read_csv(f"{ART}/label_names.csv", header=None)[0].tolist()

label_to_text = {
    "amplitud_baja":      "Aumentar amplitud (extensiones y desplazamientos más amplios).",
    "variedad_baja":      "Introducir cambios de dirección y diagonales.",
    "mucha_simetria":     "Explorar asimetrías entre izquierda y derecha.",
    "poca_simetria":      "Equilibrar con momentos de simetría.",
    "fluidez_baja":       "Trabajar transiciones para mayor continuidad/fluidez.",
    "poco_rango_niveles": "Usar niveles alto y bajo además del medio.",
}

# --- pares izquierda/derecha COCO-17 ---
COCO_EDGES = [(5,7),(7,9),(6,8),(8,10),(11,13),(13,15),(12,14),(14,16),(5,6),(11,12),(5,11),(6,12)]
COCO_PAIRS = [(5,6),(7,8),(9,10),(11,12),(13,14),(15,16)]

def center_of_mass(K):
    # COCO: caderas 11 y 12 si existen
    T,J,D = K.shape
    if J >= 13:
        return (K[:,11,:] + K[:,12,:]) / 2
    return K.mean(axis=1)

def features_coreograficos(K):
    """K: (T,J,D) con NaNs posibles, devuelve dict de features."""
    T,J,D = K.shape
    amp_mean = (np.nanmax(K,axis=0) - np.nanmin(K,axis=0)).mean(axis=0)
    amp_x = float(amp_mean[0]); amp_y = float(amp_mean[1])
    amp_z = float(amp_mean[2]) if D==3 else np.nan
    vel = np.linalg.norm(np.diff(K,axis=0),axis=2)
    vel_media = float(np.nanmean(vel)) if vel.size else 0.0
    pair_vals=[]
    for a,b in COCO_PAIRS:
        if a<J and b<J:
            diff = K[:,a,:]-K[:,b,:]
            # Check number of dimensions before calculating norm
            if diff.ndim > 1:
                d = np.linalg.norm(diff,axis=1).mean() # Correct axis for (T, D) -> (T,)
                pair_vals.append(np.nanmean(d))
            elif diff.ndim == 1: # Handle case where diff is 1D (T,)
                 d = np.linalg.norm(diff).mean()
                 pair_vals.append(np.nanmean(d))
    sim_raw = float(np.nanmean(pair_vals)) if pair_vals else np.nan
    com = center_of_mass(K)
    nivel_p10 = float(np.nanpercentile(com[:,1],10))
    nivel_p90 = float(np.nanpercentile(com[:,1],90))
    nivel_rango = float(nivel_p10 - nivel_p90)
    disp = np.diff(com,axis=0)
    dirs = np.arctan2(disp[:,1], disp[:,0] + 1e-9)
    cambios = np.abs(np.diff(dirs))
    variedad_dir = float(np.nanmean(cambios)) if len(cambios)>0 else 0.0
    return {
        "amp_x": amp_x, "amp_y": amp_y, "amp_z": amp_z,
        "vel_media": vel_media,
        "simetria_raw": sim_raw,
        "nivel_rango": nivel_rango,
        "variedad_dir": variedad_dir,
        "frames": int(T), "joints": int(J), "dims": int(D)
    }

def inferir_desde_features_dict(feats: dict):
    x = np.array([[feats.get(c, np.nan) for c in feature_cols]], dtype=float)
    x = scaler.transform(imp.transform(x))
    yhat = model.predict(x)[0]
    pred_labels = [label_names[i] for i,v in enumerate(yhat) if v==1]
    sugerencias = [label_to_text.get(lbl, lbl) for lbl in pred_labels]
    return pred_labels, sugerencias

def interpolate_nan_1d(y):
    y = y.astype(np.float32)
    T = len(y); idx = np.arange(T)
    mask = np.isfinite(y)
    if mask.sum() == 0: return y
    # forward/backward
    last = np.nan
    for i in range(T):
        if not np.isfinite(y[i]) and np.isfinite(last): y[i] = last
        else: last = y[i]
    last = np.nan
    for i in range(T-1,-1,-1):
        if not np.isfinite(y[i]) and np.isfinite(last): y[i] = last
        else: last = y[i]
    mask = np.isfinite(y)
    if 2 <= mask.sum() < T:
        y[~mask] = np.interp(idx[~mask], idx[mask], y[mask])
    return y

def clean_nan_interpolate(K, min_valid_ratio=0.10):
    Kc = K.copy()
    T,J,D = Kc.shape
    used = []
    for j in range(J):
        valid = np.isfinite(Kc[:,j,:]).all(axis=1)
        if valid.mean() < min_valid_ratio:
            Kc[:,j,:] = np.nan
            continue
        for d in range(D):
            Kc[:,j,d] = interpolate_nan_1d(Kc[:,j,d])
        used.append(j)
    if not used: return Kc, []
    return Kc[:,used,:], used

In [ ]:
#Deteccion multipersona

import cv2, numpy as np
from ultralytics import YOLO
from scipy.optimize import linear_sum_assignment

def _centroid_from_kps(kps):
    # kps: (J,2) con NaNs; devuelve centro válido
    mask = np.isfinite(kps).all(axis=1)
    if mask.any():
        return np.nanmean(kps[mask], axis=0)
    return np.array([np.nan, np.nan], dtype=np.float32)

def assign_tracks(prev_centroids, det_centroids, max_dist=100.0):
    """
    Asigna detecciones a tracks previos vía Hungarian.
    Devuelve pares (prev_idx -> det_idx), y listas de no asignados.
    """
    if len(prev_centroids)==0 or len(det_centroids)==0:
        return [], list(range(len(prev_centroids))), list(range(len(det_centroids)))
    C = np.zeros((len(prev_centroids), len(det_centroids)), dtype=np.float32)
    for i, pc in enumerate(prev_centroids):
        for j, dc in enumerate(det_centroids):
            if np.any(np.isnan(pc)) or np.any(np.isnan(dc)):
                C[i,j] = 1e6
            else:
                C[i,j] = np.linalg.norm(pc - dc)
    rows, cols = linear_sum_assignment(C)
    matches, prev_un, det_un = [], [], []
    used_prev, used_det = set(), set()
    for r,c in zip(rows, cols):
        if C[r,c] <= max_dist:
            matches.append((r,c))
            used_prev.add(r); used_det.add(c)
    for i in range(len(prev_centroids)):
        if i not in used_prev: prev_un.append(i)
    for j in range(len(det_centroids)):
        if j not in used_det: det_un.append(j)
    return matches, prev_un, det_un

def video_to_tracks_yolo(
    video_path, model_name="yolov8n-pose.pt", conf=0.25, stride=1,
    max_missed=30, max_tracks=12, assign_max_dist=120.0
):
    """
    Devuelve:
      tracks_K: dict track_id -> (T,J,2) con NaNs
      FPS, W, H
    """
    model = YOLO(model_name)
    cap = cv2.VideoCapture(video_path)
    assert cap.isOpened(), f"No se pudo abrir el vídeo: {video_path}"
    T = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    W = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    H = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    FPS = cap.get(cv2.CAP_PROP_FPS) or 30.0
    J = 17

    # Estructuras de tracking
    tracks = {}           # track_id -> dict(last_centroid, last_frame, missed, kps_per_frame)
    next_id = 0

    t = 0
    while True:
        ok, frame = cap.read()
        if not ok:
            break
        if t % stride != 0:
            t += 1
            continue

        res = model.predict(frame, conf=conf, verbose=False)[0]
        det_kps_list, det_centroids = [], []
        if res.keypoints is not None and len(res.keypoints) > 0:
            for i in range(len(res.keypoints)):
                kps = res.keypoints.xy[i].cpu().numpy()  # (17,2)
                kps[:,0] = np.clip(kps[:,0], 0, W-1)
                kps[:,1] = np.clip(kps[:,1], 0, H-1)
                det_kps_list.append(kps.astype(np.float32))
                det_centroids.append(_centroid_from_kps(kps))

        # Preparar centroids previos (de tracks activos)
        active_ids = [tid for tid, tr in tracks.items() if tr["missed"] < max_missed]
        prev_centroids = [tracks[tid]["last_centroid"] for tid in active_ids]

        # Asignación
        matches, prev_un, det_un = assign_tracks(prev_centroids, det_centroids, max_dist=assign_max_dist)

        # Actualizar tracks con matches
        for (pi, di) in matches:
            tid = active_ids[pi]
            kps = det_kps_list[di]
            c = det_centroids[di]
            tracks[tid]["kps"].append((t, kps))
            tracks[tid]["last_centroid"] = c
            tracks[tid]["last_frame"] = t
            tracks[tid]["missed"] = 0

        # Los prev no asignados: incrementar missed
        for pi in prev_un:
            tid = active_ids[pi]
            tracks[tid]["missed"] += 1

        # Nuevas detecciones -> nuevos tracks
        for di in det_un:
            if len(tracks) >= max_tracks:
                continue
            kps = det_kps_list[di]
            c = det_centroids[di]
            tracks[next_id] = {
                "kps": [(t, kps)],
                "last_centroid": c,
                "last_frame": t,
                "missed": 0
            }
            next_id += 1

        t += 1

    cap.release()

    # Convertir a tensores (T,J,2) por track
    tracks_K = {}
    for tid, tr in tracks.items():
        K = np.full((T, J, 2), np.nan, dtype=np.float32)
        for (f, kps) in tr["kps"]:
            if 0 <= f < T:
                K[f] = kps
        tracks_K[tid] = K

    return tracks_K, FPS, W, H

In [ ]:
#Métricas grupales y decisión automática

def nancorr(a, b):
    """Correlación de Pearson nan-robusta."""
    mask = np.isfinite(a) & np.isfinite(b)
    if mask.sum() < 3: return np.nan
    aa = a[mask] - np.nanmean(a[mask])
    bb = b[mask] - np.nanmean(b[mask])
    denom = (np.sqrt(np.nanmean(aa**2))*np.sqrt(np.nanmean(bb**2))) + 1e-9
    return float(np.nanmean(aa*bb)/denom)

def group_metrics(COMs):
    """
    COMs: lista de arrays (T,2) por persona en la ventana (NaNs posibles)
    Devuelve: sincronía, dispersión, unísono, área de formación
    """
    n = len(COMs)
    if n < 2:
        return {"sync": np.nan, "spread": np.nan, "unison": np.nan, "formation_area": np.nan}

    # Velocidades
    vels = [np.linalg.norm(np.diff(c,axis=0),axis=1) for c in COMs]
    # Direcciones (unitarios)
    dirs = []
    for c in COMs:
        d = np.diff(c,axis=0)
        norm = np.linalg.norm(d,axis=1) + 1e-9
        dirs.append(d / norm[:,None])

    # Sincronía = corr de velocidades promedio por pares
    pair_sync = []
    for i in range(n):
        for j in range(i+1, n):
            pair_sync.append(nancorr(vels[i], vels[j]))
    sync = float(np.nanmean(pair_sync)) if pair_sync else np.nan

    # Unísono = media de coseno(dir_i, dir_j) > 0.8 (proporción)
    pair_unison = []
    for i in range(n):
        for j in range(i+1, n):
            doti = (dirs[i]*dirs[j]).sum(axis=1)  # cosenos
            mask = np.isfinite(doti)
            if mask.sum() == 0: continue
            pair_unison.append(np.nanmean((doti[mask] > 0.8).astype(np.float32)))
    unison = float(np.nanmean(pair_unison)) if pair_unison else np.nan

    # Dispersión = distancia media por pares de COM (promedio en ventana)
    pair_dist = []
    for t in range(COMs[0].shape[0]):
        pts = []
        for c in COMs:
            if np.all(np.isfinite(c[t])):
                pts.append(c[t])
        if len(pts) >= 2:
            pts = np.stack(pts)
            # media de distancias por pares
            D = np.linalg.norm(pts[:,None,:]-pts[None,:,:], axis=2)
            pair_dist.append(np.nanmean(D[np.triu_indices(len(pts),1)]))
    spread = float(np.nanmean(pair_dist)) if pair_dist else np.nan

    # Área de formación = área del convex hull promedio (aprox usando todos los puntos)
    all_pts = []
    for c in COMs:
        mask = np.all(np.isfinite(c), axis=1)
        all_pts += [c[m] for m in np.where(mask)[0]]
    if len(all_pts) >= 3:
        pts = np.vstack(all_pts)
        try:
            hull = ConvexHull(pts)
            area = float(hull.area)  # perímetro 3D en 2D? En 2D mejor usar hull.volume
            area = float(hull.volume)  # en 2D, 'volume' es el área
        except Exception:
            # fallback: bbox
            xmin, ymin = np.nanmin(pts, axis=0)
            xmax, ymax = np.nanmax(pts, axis=0)
            area = float((xmax-xmin)*(ymax-ymin))
    else:
        area = np.nan

    return {"sync": sync, "spread": spread, "unison": unison, "formation_area": area}

def auto_decide_mode(tracks_K, min_frames_active=0.5, prop_single_threshold=0.8, multi_threshold=2):
    """
    Decide 'single' vs 'multi':
      - Cuenta personas activas por frame (≥ Joints válidos/ NaNs bajos).
      - Si >80% de frames tienen exactamente 1 → 'single', si no → 'multi'.
    """
    if not tracks_K:
        return "single", None
    T = next(iter(tracks_K.values())).shape[0]
    active_counts = np.zeros(T, dtype=int)
    for K in tracks_K.values():
        valid = np.isfinite(K).all(axis=2)  # (T,J)
        frames_valid = valid.mean(axis=1) >= min_frames_active   # proporción de joints válidos
        active_counts += frames_valid.astype(int)

    prop_single = (active_counts == 1).mean()
    if prop_single >= prop_single_threshold:
        # single: elige el track con más frames válidos
        best_id, best_frames = None, -1
        for tid, K in tracks_K.items():
            frames_valid = np.isfinite(K).all(axis=2).mean(axis=1) >= min_frames_active
            cnt = int(frames_valid.sum())
            if cnt > best_frames:
                best_frames = cnt; best_id = tid
        return "single", best_id
    else:
        # multi si en una proporción relevante hay >=2 personas
        if (active_counts >= multi_threshold).mean() >= 0.2:
            return "multi", None
        # default
        return "single", max(tracks_K, key=lambda tid: np.isfinite(tracks_K[tid]).all(axis=2).mean(axis=1).sum())


In [ ]:
# Timeline por ventanas + overlay simple o multiple

def overlay_video_multi(video_path, tracks_K, timeline, out_path):
    cap = cv2.VideoCapture(video_path)
    assert cap.isOpened(), f"No se pudo abrir el vídeo: {video_path}"
    W = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    H = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    FPS = cap.get(cv2.CAP_PROP_FPS) or 30.0
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    vw = cv2.VideoWriter(out_path, fourcc, FPS, (W,H))

    # colores fijos por track
    palette = [(255,255,255),(0,255,255),(255,0,255),(255,255,0),(0,200,255),(255,200,0),(200,255,0),(200,200,255)]
    track_ids = sorted(tracks_K.keys())
    color_of = {tid: palette[i%len(palette)] for i,tid in enumerate(track_ids)}

    t = 0
    while True:
        ok, frame = cap.read()
        if not ok: break

        # dibujar cada track si hay puntos
        for tid, K in tracks_K.items():
            if t < len(K):
                P = K[t]
                col = color_of[tid]
                for a,b in COCO_EDGES:
                    if a < P.shape[0] and b < P.shape[0]:
                        xa,ya = P[a]; xb,yb = P[b]
                        if np.isfinite(xa) and np.isfinite(ya) and np.isfinite(xb) and np.isfinite(yb):
                            cv2.line(frame, (int(xa),int(ya)), (int(xb),int(yb)), col, 2)
                for (x,y) in P:
                    if np.isfinite(x) and np.isfinite(y):
                        cv2.circle(frame, (int(x),int(y)), 2, col, -1)

        # texto: sugerencias activas (globales)
        active = []
        for seg in timeline:
            if seg["start_f"] <= t <= seg["end_f"]:
                active = seg.get("sugerencias_global", []) or seg.get("sugerencias_single", [])
        y0 = 30
        for s in active[:4]:
            cv2.putText(frame, f"• {s}", (20, y0), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,0,0), 4, cv2.LINE_AA)
            cv2.putText(frame, f"• {s}", (20, y0), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,255,255), 1, cv2.LINE_AA)
            y0 += 28

        vw.write(frame); t += 1

    cap.release(); vw.release()

def run_inference_auto(video_path, model_name="yolov8n-pose.pt",
                       conf=0.25, stride=1, win_sec=5.0, hop_sec=2.5):
    # 1) detección + tracking
    tracks_K, FPS, W, H = video_to_tracks_yolo(video_path, model_name=model_name, conf=conf, stride=stride)

    # 2) decisión auto
    mode, best_id = auto_decide_mode(tracks_K)
    print(f"Modo decidido: {mode}  | best_id: {best_id}")

    # 3) limpiar/interpolar cada track
    tracks_clean = {}
    for tid, K in tracks_K.items():
        Kc, used = clean_nan_interpolate(K, min_valid_ratio=0.10)
        tracks_clean[tid] = Kc

    win = max(1, int(round(win_sec * FPS)))
    hop = max(1, int(round(hop_sec * FPS)))
    T = next(iter(tracks_clean.values())).shape[0]

    rows = []
    timeline = []

    if mode == "single":
        # elegir K del best_id
        K_sel = tracks_clean[best_id]
        for start in range(0, T-1, hop):
            end = min(T-1, start + win)
            if end - start < max(8, int(0.25*win)): break
            Kw = K_sel[start:end]
            feats = features_coreograficos(Kw)
            feats_ml = {c: feats.get(c, np.nan) for c in feature_cols}
            labels, suggs = inferir_desde_features_dict(feats_ml)
            timeline.append({"start_f":start, "end_f":end, "sugerencias_single":suggs})
            row = {"start_f":start, "end_f":end, "start_s":start/FPS, "end_s":end/FPS, "mode":"single", "track_id":best_id}
            row.update(feats); row["labels"]="|".join(labels); row["sugerencias"]="|".join(suggs)
            rows.append(row)

        out_dir = os.path.join(BASE, "data/processed/aistpp")
        os.makedirs(out_dir, exist_ok=True)
        out_csv = os.path.join(out_dir, "timeline_auto_single.csv")
        pd.DataFrame(rows).to_csv(out_csv, index=False, encoding="utf-8")
        out_mp4 = os.path.join(out_dir, "video_auto_single.mp4")
        overlay_video_multi(video_path, {best_id: tracks_clean[best_id]}, timeline, out_mp4)
        print("CSV:", out_csv); print("Video:", out_mp4)
        return {"mode": mode, "csv": out_csv, "video": out_mp4, "timeline": timeline}

    else:
        # MULTI: por ventana, computar features individuales + métricas grupales
        track_ids = sorted(tracks_clean.keys())
        for start in range(0, T-1, hop):
            end = min(T-1, start + win)
            if end - start < max(8, int(0.25*win)): break

            feats_per_track = {}
            suggs_per_track = {}
            COMs = []

            for tid in track_ids:
                Kw = tracks_clean[tid][start:end]
                # si la ventana de ese track está totalmente NaN, saltar
                if not np.isfinite(Kw).any():
                    continue
                feats = features_coreograficos(Kw)
                feats_per_track[tid] = feats
                feats_ml = {c: feats.get(c, np.nan) for c in feature_cols}
                labels, suggs = inferir_desde_features_dict(feats_ml)
                suggs_per_track[tid] = suggs
                COMs.append(center_of_mass(Kw))

            # métricas grupales
            gm = group_metrics(COMs) if len(COMs) >= 2 else {"sync":np.nan,"spread":np.nan,"unison":np.nan,"formation_area":np.nan}

            # sugerencias globales (reglas simples sobre métricas grupales)
            sglob = []
            if np.isfinite(gm["sync"]) and gm["sync"] < 0.2:
                sglob.append("Sincronía grupal baja: ensayar cues de entrada/salida y respiración común.")
            if np.isfinite(gm["unison"]) and gm["unison"] < 0.5:
                sglob.append("Poco unísono: trabajar frases en espejo y acentos compartidos.")
            if np.isfinite(gm["spread"]) and gm["spread"] < 60:
                sglob.append("Formación muy compacta: explorar aperturas y diagonales.")
            if np.isfinite(gm["spread"]) and gm["spread"] > 300:
                sglob.append("Demasiada dispersión: introducir agrupamientos o centros de atención.")
            # (opcional) área de formación poco variable entre ventanas → sugerir cambios de formación

            # fila CSV (agregada)
            row = {"start_f":start, "end_f":end, "start_s":start/FPS, "end_s":end/FPS, "mode":"multi"}
            row.update({f"gm_{k}":v for k,v in gm.items()})
            row["sugerencias_global"] = "|".join(sglob)
            # puedes agregar resumidas por track si quieres
            rows.append(row)

            timeline.append({
                "start_f":start, "end_f":end,
                "sugerencias_global": sglob,
                "sugerencias_por_track": {int(t): suggs_per_track[t] for t in suggs_per_track}
            })

        out_dir = os.path.join(BASE, "data/processed/aistpp")
        os.makedirs(out_dir, exist_ok=True)
        out_csv = os.path.join(out_dir, "timeline_auto_multi.csv")
        pd.DataFrame(rows).to_csv(out_csv, index=False, encoding="utf-8")
        out_mp4 = os.path.join(out_dir, "video_auto_multi.mp4")
        overlay_video_multi(video_path, tracks_clean, timeline, out_mp4)
        print("CSV:", out_csv); print("Video:", out_mp4)
        return {"mode": "multi", "csv": out_csv, "video": out_mp4, "timeline": timeline}



In [ ]:
# ===== SUBIR VÍDEO + EXTRAER KEYPOINTS (MediaPipe Pose) =====
import os, tempfile
import numpy as np
import cv2
import mediapipe as mp
from google.colab import files, drive

# Montar Drive si no está montado
if not os.path.isdir("/content/drive/MyDrive"):
    drive.mount("/content/drive")

BASE = "/content/drive/MyDrive/asistente_coreografico"
OUT_DIR = os.path.join(BASE, "data/processed/custom_videos")
os.makedirs(OUT_DIR, exist_ok=True)

def upload_video():
    up = files.upload()
    if not up:
        raise RuntimeError("No se subió ningún archivo.")
    name = list(up.keys())[0]
    return name, up[name]

def process_video_with_mediapipe(video_path, output_dir):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise RuntimeError(f"No se pudo abrir el vídeo: {video_path}")

    all_keypoints, frame_count = [], 0
    # Pose con FP32; segmentación desactivada para ir más ligero
    with mp.solutions.pose.Pose(
        static_image_mode=False,
        model_complexity=2,
        enable_segmentation=False,
        min_detection_confidence=0.5
    ) as pose:
        while True:
            ok, frame = cap.read()
            if not ok:
                break
            rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            res = pose.process(rgb)

            if res.pose_landmarks:
                # 33 landmarks × [x, y, z, visibility]
                flat = []
                for lm in res.pose_landmarks.landmark:
                    flat.extend([lm.x, lm.y, lm.z, lm.visibility])
                all_keypoints.append(flat)
            else:
                all_keypoints.append([np.nan] * (33 * 4))

            frame_count += 1
            if frame_count % 30 == 0:
                print(f"Procesados {frame_count} frames...")

    cap.release()

    arr = np.asarray(all_keypoints, dtype=np.float32)  # (T, 132)
    out_name = os.path.splitext(os.path.basename(video_path))[0] + "_keypoints.npy"
    out_path = os.path.join(output_dir, out_name)
    np.save(out_path, arr)
    return arr, out_path

# Flujo: subir → temporal → procesar → guardar
video_tmp = None
try:
    name, data = upload_video()
    with tempfile.NamedTemporaryFile(delete=False, suffix=os.path.splitext(name)[1]) as tmp:
        tmp.write(data)
        video_tmp = tmp.name
    print(f"▶ Vídeo '{name}' guardado temporalmente en: {video_tmp}")

    arr, saved = process_video_with_mediapipe(video_tmp, OUT_DIR)
    print(f"\n✅ Keypoints extraídos: shape={arr.shape}  (frames x 132)")
    print(f"📁 Guardado en: {saved}")

finally:
    try:
        if video_tmp and os.path.exists(video_tmp):
            os.remove(video_tmp)
            print(f"🧹 Temporal eliminado: {video_tmp}")
    except Exception:
        pass


In [ ]:
import cv2
import numpy as np
import pandas as pd
import joblib
import os
import tempfile
import mediapipe as mp  # Mantener el import: se usa en process_video_with_mediapipe si hace falta

# Cargar artefactos (entrenados previamente con tu CSV)
BASE = "/content/drive/MyDrive/asistente_coreografico"
ART  = f"{BASE}/artifacts"
imp          = joblib.load(f"{ART}/imputer.joblib")
scaler       = joblib.load(f"{ART}/scaler.joblib")
model        = joblib.load(f"{ART}/model_ovr_logreg.joblib")
feature_cols = pd.read_csv(f"{ART}/feature_cols.csv", header=None)[0].tolist()
label_names  = pd.read_csv(f"{ART}/label_names.csv", header=None)[0].tolist()

label_to_text = {
    "amplitud_baja":      "Aumentar amplitud (extensiones y desplazamientos más amplios).",
    "variedad_baja":      "Introducir cambios de dirección y diagonales.",
    "mucha_simetria":     "Explorar asimetrías entre izquierda y derecha.",
    "poca_simetria":      "Equilibrar con momentos de simetría.",
    "fluidez_baja":       "Trabajar transiciones para mayor continuidad/fluidez.",
    "poco_rango_niveles": "Usar niveles alto y bajo además del medio.",
    "exceso_rango_niveles": "Reducir rango vertical para mayor cohesión.",
    "frase_corta": "Frase corta: considerar repetición o desarrollo del material."
    # Añade aquí el resto de etiquetas de label_names y su sugerencia correspondiente
}

# --- pares izquierda/derecha COCO-17 ---
# Nota: MediaPipe proporciona 33 puntos, pero el modelo fue entrenado con COCO-17.
# Se necesita mapear los puntos de MediaPipe a COCO-17 o reentrenar el modelo.
# Para simplificar en este ejemplo, asumimos un mapeo o un subconjunto
# de puntos MediaPipe que se alinea con COCO-17 para el cálculo de features.
# Este mapeo es simplificado a efectos demostrativos.
# Un mapeo correcto requeriría revisar los índices de MediaPipe y COCO.
# Suponemos que los primeros 17 puntos de MediaPipe se alinean con COCO-17 para estas features.
MEDIAPIPE_TO_COCO_MAP = {
    0: 0,  # Nariz
    1: 1,  # Ojo izquierdo (interno)
    2: 2,  # Ojo izquierdo
    3: 3,  # Ojo izquierdo (externo)
    4: 4,  # Ojo derecho (interno)
    5: 5,  # Ojo derecho
    6: 6,  # Ojo derecho (externo)
    7: 7,  # Oreja izquierda
    8: 8,  # Oreja derecha
    9: 9,  # Boca izquierda
    10: 10,  # Boca derecha
    11: 11,  # Hombro izquierdo
    12: 12,  # Hombro derecho
    13: 13,  # Codo izquierdo
    14: 14,  # Codo derecho
    15: 15,  # Muñeca izquierda
    16: 16,  # Muñeca derecha
    17: None,  # Meñique izquierdo
    18: None,  # Meñique derecho
    19: None,  # Índice izquierdo
    20: None,  # Índice derecho
    21: None,  # Pulgar izquierdo
    22: None,  # Pulgar derecho
    23: None,  # Cadera izquierda
    24: None,  # Cadera derecha
    25: None,  # Rodilla izquierda
    26: None,  # Rodilla derecha
    27: None,  # Tobillo izquierdo
    28: None,  # Tobillo derecho
    29: None,  # Talón izquierdo
    30: None,  # Talón derecho
    31: None,  # Punta pie izquierdo
    32: None,  # Punta pie derecho
}

# Pares COCO simplificados basados en los primeros 17 puntos de MediaPipe
MEDIAPIPE_COCO_PAIRS = [(11,12), (13,14), (15,16)]  # Hombros, codos, muñecas

def center_of_mass_mediapipe(K):
    # En MediaPipe las caderas son 23 y 24
    T,J,D = K.shape
    if J >= 25:  # Asumiendo que se usan puntos de MediaPipe
        return (K[:,23,:2] + K[:,24,:2]) / 2  # Solo x, y para el centro de masas
    return K[:,:,:2].mean(axis=1)  # Solo x, y promediando todos los puntos

def features_coreograficos_mediapipe(K):
    """K: (T,J,D) con posibles NaNs; devuelve un dict de features."""
    T,J,D = K.shape
    # Usar únicamente x, y para features 2D
    K_2d = K[:,:,:2]

    amp_mean = (np.nanmax(K_2d, axis=0) - np.nanmin(K_2d, axis=0)).mean(axis=0)
    amp_x = float(amp_mean[0]); amp_y = float(amp_mean[1])

    vel = np.linalg.norm(np.diff(K_2d, axis=0), axis=2)
    vel_media = float(np.nanmean(vel)) if vel.size else 0.0

    pair_vals=[]
    for a,b in MEDIAPIPE_COCO_PAIRS:
        if a < J and b < J:
            diff = K_2d[:,a,:] - K_2d[:,b,:]
            if diff.ndim > 1:
                d = np.linalg.norm(diff, axis=1).mean()  # Eje correcto para (T, D) -> (T,)
                pair_vals.append(np.nanmean(d))
            elif diff.ndim == 1:  # Caso en el que diff es 1D (T,)
                d = np.linalg.norm(diff).mean()
                pair_vals.append(np.nanmean(d))
    sim_raw = float(np.nanmean(pair_vals)) if pair_vals else np.nan

    com = center_of_mass_mediapipe(K)
    nivel_p10 = float(np.nanpercentile(com[:,1], 10))
    nivel_p90 = float(np.nanpercentile(com[:,1], 90))
    nivel_rango = float(nivel_p10 - nivel_p90)

    disp = np.diff(com, axis=0)
    # Evitar división por cero en arctan2 añadiendo un pequeño epsilon
    dirs = np.arctan2(disp[:,1], disp[:,0] + 1e-9)
    cambios = np.abs(np.diff(dirs))
    variedad_dir = float(np.nanmean(cambios)) if len(cambios) > 0 else 0.0

    return {
        "amplitud_x": amp_x,                  # Mantener nombres originales para consistencia con el modelo
        "amplitud_y": amp_y,
        "velocidad_media": vel_media,
        "simetria": sim_raw,                  # Mantener nombres originales para consistencia con el modelo
        "nivel_rango": nivel_rango,
        "variedad_direcciones": variedad_dir, # Mantener nombres originales para consistencia con el modelo
        "frames": int(T),
        "joints": int(J),
        "dims": int(D)
    }

def inferir_desde_features_dict(feats: dict):
    # Asegurar que el orden de las features coincide con el de entrenamiento
    x = np.array([[feats.get(c, np.nan) for c in feature_cols]], dtype=float)
    x = scaler.transform(imp.transform(x))
    yhat = model.predict(x)[0]
    # Obtener etiquetas predichas (1 = predicha, 0 = no)
    predicted_labels = [label_names[i] for i, v in enumerate(yhat) if v == 1]
    sugerencias = [label_to_text.get(lbl, lbl) for lbl in predicted_labels]
    return predicted_labels, sugerencias

def interpolate_nan_1d(y):
    y = y.astype(np.float32)
    T = len(y); idx = np.arange(T)
    mask = np.isfinite(y)
    if mask.sum() == 0: return y
    # Relleno hacia delante/atrás
    last = np.nan
    for i in range(T):
        if not np.isfinite(y[i]) and np.isfinite(last): y[i] = last
        else: last = y[i]
    last = np.nan
    for i in range(T-1, -1, -1):
        if not np.isfinite(y[i]) and np.isfinite(last): y[i] = last
        else: last = y[i]
    mask = np.isfinite(y)
    if 2 <= mask.sum() < T:
        y[~mask] = np.interp(idx[~mask], idx[mask], y[mask])
    return y

def clean_nan_interpolate(K, min_valid_ratio=0.10):
    # Limpia NaNs por articulación y dimension, e interpola si hay datos suficientes
    Kc = K.copy()
    T,J,D = Kc.shape
    used = []
    for j in range(J):
        # Validez: frames donde todas las dimensiones del punto son finitas
        valid = np.isfinite(Kc[:,j,:]).all(axis=1)
        if valid.mean() < min_valid_ratio:
            Kc[:,j,:] = np.nan
            continue
        for d in range(D):
            # Interpolar cada dimensión por separado
            Kc[:,j,d] = interpolate_nan_1d(Kc[:,j,d])
        # Comprobar si sigue habiendo algún valor válido tras la interpolación
        if np.isfinite(Kc[:,j,:]).any():
            used.append(j)

    if not used:
        print("No hay articulaciones válidas tras limpieza e interpolación.")
        return Kc, []  # Devuelve con NaNs si no hay puntos utilizables

    # Filtrar para incluir solo las articulaciones usadas
    return Kc[:,used,:], used

# Aristas COCO simplificadas para dibujado basadas en los primeros 17 puntos de MediaPipe
# Es una simplificación; para dibujo preciso se necesitaría un mapeo riguroso.
MEDIAPIPE_COCO_EDGES_SIMPLIFIED = [
    (11,13), (13,15),      # Brazo izquierdo (hombro a muñeca)
    (12,14), (14,16),      # Brazo derecho (hombro a muñeca)
    (11,12),               # Hombros
    (23,24),               # Caderas
    (23,25), (25,27),      # Pierna izquierda (cadera a tobillo)
    (24,26), (26,28),      # Pierna derecha (cadera a tobillo)
    (11,23), (12,24)       # Tronco (hombro a cadera)
]

def overlay_suggestions_on_video(video_path, K, timeline, out_path):
    # Pinta esqueleto y sugerencias activas en cada frame y exporta vídeo anotado
    cap = cv2.VideoCapture(video_path)
    assert cap.isOpened(), f"No se pudo abrir el vídeo: {video_path}"
    W = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    H = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    FPS = cap.get(cv2.CAP_PROP_FPS) or 30.0
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    vw = cv2.VideoWriter(out_path, fourcc, FPS, (W,H))

    t = 0
    while True:
        ok, frame = cap.read()
        if not ok: break

        if t < len(K):
            P = K[t][:,:2]  # Solo x, y para dibujar
            # Dibujar esqueleto (simplificado)
            for a,b in MEDIAPIPE_COCO_EDGES_SIMPLIFIED:
                if a < P.shape[0] and b < P.shape[0]:
                    xa,ya = P[a]; xb,yb = P[b]
                    if np.isfinite(xa) and np.isfinite(ya) and np.isfinite(xb) and np.isfinite(yb):
                        cv2.line(frame, (int(xa),int(ya)), (int(xb),int(yb)), (255,255,255), 2)
            for (x,y) in P:
                if np.isfinite(x) and np.isfinite(y):
                    cv2.circle(frame, (int(x),int(y)), 2, (255,255,255), -1)

        # Sugerencias activas para el frame t
        active = []
        for seg in timeline:
            if seg["start_f"] <= t <= seg["end_f"]:
                active = seg["sugerencias"]
        y0 = 30
        for s in active[:4]:
            cv2.putText(frame, f"• {s}", (20, y0), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,0,0), 4, cv2.LINE_AA)
            cv2.putText(frame, f"• {s}", (20, y0), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,255,255), 1, cv2.LINE_AA)
            y0 += 28

        vw.write(frame); t += 1

    cap.release(); vw.release()

def run_inference_on_custom_video(video_path, keypoints_array, output_dir, win_sec=5.0, hop_sec=2.5):
    """
    Ejecuta la inferencia sobre un vídeo custom usando keypoints ya extraídos.

    Args:
        video_path (str): Ruta del vídeo original.
        keypoints_array (np.ndarray): Array de keypoints de MediaPipe (T, J*D).
        output_dir (str): Carpeta para guardar las salidas.
        win_sec (float): Tamaño de ventana en segundos para calcular features.
        hop_sec (float): Salto entre ventanas en segundos.

    Returns:
        dict: Rutas al CSV y al vídeo anotado, y la estructura 'timeline'.
    """
    # FPS del vídeo original
    cap = cv2.VideoCapture(video_path)
    FPS = cap.get(cv2.CAP_PROP_FPS) or 30.0
    cap.release()

    # Reajustar array de keypoints a (T, J, D)
    # MediaPipe devuelve 33 puntos con (x, y, z, visibility)
    T = keypoints_array.shape[0]
    J = 33  # MediaPipe Pose: 33 puntos
    D = 4   # x, y, z, visibility
    K_raw = keypoints_array.reshape(T, J, D)

    # 2) Limpieza e interpolación de keypoints
    Kc, used = clean_nan_interpolate(K_raw, min_valid_ratio=0.10)
    if not used:
        print("No hay articulaciones válidas tras limpieza e interpolación. No se puede continuar.")
        return None

    win = max(1, int(round(win_sec * FPS)))
    hop = max(1, int(round(hop_sec * FPS)))
    T_clean = len(Kc)

    rows = []
    timeline = []

    # 3) Calcular features y generar sugerencias con ventana deslizante
    for start in range(0, T_clean - 1, hop):
        end = min(T_clean - 1, start + win)
        if end - start < max(8, int(0.25 * win)):
             break  # Garantizar ventana suficientemente grande

        Kw = Kc[start:end]

        # Calcular features adaptadas a puntos MediaPipe
        feats = features_coreograficos_mediapipe(Kw)

        # Preparar features para la inferencia ML
        feats_ml = {c: feats.get(c, np.nan) for c in feature_cols}

        # Generar sugerencias
        labels, suggs = inferir_desde_features_dict(feats_ml)

        timeline.append({"start_f": start, "end_f": end, "sugerencias": suggs})

        row = {"start_f": start, "end_f": end, "start_s": start / FPS, "end_s": end / FPS}
        row.update(feats)
        row["labels"] = "|".join(labels)
        row["sugerencias"] = "|".join(suggs)
        rows.append(row)

    # 4) Guardar timeline a CSV
    out_csv = os.path.join(output_dir, "custom_video_suggestions_timeline.csv")
    pd.DataFrame(rows).to_csv(out_csv, index=False, encoding="utf-8")
    print(f"Sugerencias (timeline) guardadas en: {out_csv}")

    # 5) Crear vídeo anotado
    out_mp4 = os.path.join(output_dir, "custom_video_annotated.mp4")
    overlay_suggestions_on_video(video_path, Kc, timeline, out_mp4)
    print(f"Vídeo anotado guardado en: {out_mp4}")

    return {"csv": out_csv, "video": out_mp4, "timeline": timeline}

# --- Ejecutar el proceso sobre el vídeo custom ---
# Se asume que video_path y keypoints_array existen desde la celda previa.

# Para demostración, asumimos que las variables están definidas por la ejecución anterior.
# Comprobamos que el vídeo original aún está accesible para la superposición.
# Si el archivo temporal se eliminó, habrá que volver a subir o usar una ruta persistente.
# Asumimos que video_path apunta a una ubicación persistente o re-subimos si es necesario.

# Re-subir el vídeo si el archivo temporal ya no existe
try:
    # Verificar si el archivo temporal previo sigue existiendo
    # Assuming 'video_path' variable exists and has the path from previous execution
    if 'video_path' not in locals() or not os.path.exists(video_path):
         print("No se encontró el vídeo anterior o la ruta no está definida. Vuelve a subir el vídeo, por favor.")
         video_name, video_content = upload_video()
         with tempfile.NamedTemporaryFile(delete=False, suffix=os.path.splitext(video_name)[1]) as temp_video_file:
             temp_video_file.write(video_content)
             video_path = temp_video_file.name
         print(f"Vídeo '{video_name}' re-subido y guardado temporalmente en: {video_path}")

    # Asegurar que keypoints_array también esté disponible
    if 'keypoints_array' not in locals() or keypoints_array is None:
        print("No se encontró el array de keypoints. Ejecuta primero el paso de MediaPipe.")
        # cargar desde el .npy guardado
        keypoints_path = os.path.join(output_dir, "custom_video_keypoints.npy")
        if os.path.exists(keypoints_path):
            keypoints_array = np.load(keypoints_path)
        else:
            raise FileNotFoundError("No se encontró el archivo de keypoints.")
        # Para este ejemplo, asumimos que keypoints_array estará disponible tras la subida.


    if 'video_path' in locals() and video_path and os.path.exists(video_path) and 'keypoints_array' in locals() and keypoints_array is not None:
        output_dir = os.path.join(BASE, "data/processed/custom_videos")
        os.makedirs(output_dir, exist_ok=True)
        print(f"Procesando vídeo: {video_path}")
        print(f"Usando array de keypoints con forma: {keypoints_array.shape}")
        res_custom = run_inference_on_custom_video(video_path, keypoints_array, output_dir)
        print("\nProcesamiento del vídeo custom finalizado.")
        print(res_custom)

        # Limpieza: eliminar el archivo temporal del vídeo al terminar
        if 'video_path' in locals() and video_path and os.path.exists(video_path) and video_path.startswith(tempfile.gettempdir()):
             os.remove(video_path)
             print(f"Archivo temporal de vídeo eliminado: {video_path}")

    else:
        print("No se encontró el vídeo, la ruta del vídeo o el array de keypoints. No se puede continuar.")

except Exception as e:
    print(f"Ocurrió un error durante el procesamiento del vídeo custom: {e}")
    # Asegurar que el archivo temporal se elimine también en caso de error
    if 'video_path' in locals() and video_path and os.path.exists(video_path) and video_path.startswith(tempfile.gettempdir()):
         os.remove(video_path)
         print(f"Archivo temporal eliminado por error: {video_path}")

In [ ]:

# Visualización de frames de un vídeo


import os, math
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Opcional (para modo slider)
try:
    from ipywidgets import interact, IntSlider
    HAS_WIDGETS = True
except Exception:
    HAS_WIDGETS = False

# -----------------------------
# Config: intenta usar el vídeo anotado por defecto
# -----------------------------
BASE = "/content/drive/MyDrive/asistente_coreografico"
DEFAULT_DIR = os.path.join(BASE, "data/processed/custom_videos")
DEFAULT_ANNOTATED = os.path.join(DEFAULT_DIR, "custom_video_annotated.mp4")

# Cambia esta ruta si quieres otro vídeo
video_path = DEFAULT_ANNOTATED if os.path.exists(DEFAULT_ANNOTATED) else None
print("Vídeo detectado:", video_path)

# -----------------------------
# Utilidades de vídeo
# -----------------------------
def get_video_info(path):
    """Devuelve diccionario con info básica del vídeo."""
    cap = cv2.VideoCapture(path)
    assert cap.isOpened(), f"No se pudo abrir el vídeo: {path}"
    fps   = cap.get(cv2.CAP_PROP_FPS) or 30.0
    nfrm  = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) if cap.get(cv2.CAP_PROP_FRAME_COUNT) else 0
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height= int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    cap.release()
    duration = nfrm / fps if fps > 0 and nfrm > 0 else 0
    return {"fps": fps, "frames": nfrm, "width": width, "height": height, "duration": duration}

def read_frame_at(path, t_sec=None, frame_index=None):
    """
    Lee un frame en 't_sec' (segundos) o en 'frame_index'.
    Devuelve (frame_rgb, idx, t_real_sec).
    """
    assert os.path.exists(path), f"No existe el archivo: {path}"
    cap = cv2.VideoCapture(path)
    assert cap.isOpened(), f"No se pudo abrir el vídeo: {path}"

    fps  = cap.get(cv2.CAP_PROP_FPS) or 30.0
    nfrm = int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) if cap.get(cv2.CAP_PROP_FRAME_COUNT) else 0

    if frame_index is None:
        if t_sec is None:
            t_sec = 0.0
        frame_index = int(round(t_sec * fps))

    frame_index = int(np.clip(frame_index, 0, max(0, nfrm-1)))
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_index)
    ok, bgr = cap.read()
    cap.release()

    if not ok or bgr is None:
        return None, frame_index, (frame_index / fps if fps > 0 else 0.0)

    rgb = cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)
    t_real = frame_index / fps if fps > 0 else 0.0
    return rgb, frame_index, t_real

def show_frame_at_time(path, t_sec):
    """Muestra un frame en el segundo 't_sec'."""
    info = get_video_info(path)
    img, idx, t_real = read_frame_at(path, t_sec=t_sec)
    if img is None:
        print("No se pudo leer el frame.")
        return
    plt.figure(figsize=(6, 4))
    plt.imshow(img); plt.axis('off')
    plt.title(f"Frame {idx} @ {t_real:.2f}s  |  {info['width']}x{info['height']}  |  fps={info['fps']:.2f}")
    plt.show()

def show_frames_grid(path, num_frames=12, cols=4, margin=0.02):
    """
    Muestra una rejilla de 'num_frames' frames muestreados uniformemente.
    """
    info = get_video_info(path)
    n = info["frames"]
    if n <= 0:
        print("No se pudo determinar el número de frames.")
        return

    # ÍNDICES muestreados uniformemente (evita extremos por si hay negro)
    idxs = np.linspace(int(n*0.02), max(int(n*0.98)-1, 0), num_frames).astype(int)
    rows = int(math.ceil(num_frames / cols))
    plt.figure(figsize=(4*cols, 3*rows))
    for i, idx in enumerate(idxs):
        img, _, t_real = read_frame_at(path, frame_index=idx)
        plt.subplot(rows, cols, i+1)
        if img is None:
            plt.text(0.5, 0.5, f"Frame {idx}\nNo leído", ha='center', va='center')
            plt.axis('off')
        else:
            plt.imshow(img); plt.axis('off')
            plt.title(f"{idx} ({t_real:.2f}s)", fontsize=10)
        plt.subplots_adjust(wspace=margin, hspace=margin)
    plt.suptitle(f"Rejilla de frames | total={n} | fps={info['fps']:.2f}", y=1.02)
    plt.show()

def show_slider(path):
    """
    Slider interactivo para navegar frames.
    Requiere ipywidgets; si no está disponible, avisa.
    """
    if not HAS_WIDGETS:
        print("ipywidgets no disponible. Instala con: !pip install ipywidgets y reinicia.")
        return
    info = get_video_info(path)
    n = info["frames"]
    def _show(idx):
        img, _, t_real = read_frame_at(path, frame_index=idx)
        plt.figure(figsize=(6,4))
        if img is None:
            plt.text(0.5, 0.5, f"Frame {idx}\nNo leído", ha='center', va='center')
            plt.axis('off')
        else:
            plt.imshow(img); plt.axis('off')
            plt.title(f"Frame {idx} @ {t_real:.2f}s")
        plt.show()
    interact(_show, idx=IntSlider(min=0, max=max(0,n-1), step=1, value=min(0, n//2)))


# Ejemplos de uso
if video_path:
    info = get_video_info(video_path)
    print(f"Info: {info}")
    show_frames_grid(video_path, num_frames=12, cols=4)
else:
    print("No se encontró el vídeo por defecto. Ajusta la variable 'video_path' manualmente.")


In [ ]:
# =========================
# Bootstrap robusto (imports + checks + carga artefactos)
# =========================
import os, glob, warnings
from pathlib import Path

# Imports que faltaban
import numpy as np
import pandas as pd
from joblib import load



#  Rutas base
BASE        = "/content/drive/MyDrive/asistente_coreografico"
ART         = f"{BASE}/artifacts"
MODELS_DIR  = f"{BASE}/models"
REPORTS_DIR = f"{BASE}/reports"
os.makedirs(MODELS_DIR, exist_ok=True)
os.makedirs(REPORTS_DIR, exist_ok=True)

# 3) Verificación de artefactos requeridos
required = [
    f"{ART}/imputer.joblib",
    f"{ART}/scaler.joblib",
    f"{ART}/model_ovr_logreg.joblib",
    f"{ART}/feature_cols.csv",
    f"{ART}/label_names.csv",
]
missing = [p for p in required if not os.path.exists(p)]
if missing:
    raise FileNotFoundError(
        "Faltan artefactos necesarios en ART:\n  - " + "\n  - ".join(missing)
    )

# 4) Carga artefactos (ya con pandas importado)
imp          = load(f"{ART}/imputer.joblib")
scaler       = load(f"{ART}/scaler.joblib")
model_basic  = load(f"{ART}/model_ovr_logreg.joblib")
feature_cols = pd.read_csv(f"{ART}/feature_cols.csv", header=None)[0].tolist()
label_names  = pd.read_csv(f"{ART}/label_names.csv", header=None)[0].tolist()

# 5) Diccionario de textos (si no existe)
try:
    label_to_text
except NameError:
    label_to_text = {ln: f"Sugerencia para {ln}" for ln in label_names}

# 6) Utilidades de datos (con glob y Path ya disponibles)
SEARCH_ROOTS = [
    f"{BASE}/data/aistppp",
    f"{BASE}/data/AIST_ppp",
    f"{BASE}/data/aistppp_features",
    f"{BASE}/data/processed",
    f"{BASE}/data/custom_videos/processed",
    f"{BASE}/data/demo",
]

def _align_features_df_any(df_like, cols):
    """Asegura DataFrame con columnas EXACTAS (ordenadas); rellena faltantes con NaN."""
    if isinstance(df_like, pd.DataFrame):
        for c in cols:
            if c not in df_like.columns:
                df_like[c] = np.nan
        return df_like[cols]
    elif isinstance(df_like, (dict, pd.Series)):
        return pd.DataFrame([{c: df_like.get(c, np.nan) for c in cols}])[cols]
    else:
        raise ValueError("Se esperaba DataFrame o dict/Series con nombres de columnas.")

def find_feature_files(roots=SEARCH_ROOTS):
    files = []
    for r in roots:
        if not os.path.isdir(r):
            continue
        files += glob.glob(os.path.join(r, "**", "features.parquet"), recursive=True)
        files += glob.glob(os.path.join(r, "**", "features.csv"), recursive=True)
    return sorted(set(files))

def load_video_features(path):
    if path.endswith(".parquet"):
        # Si falla por motor de parquet, instala pyarrow (ver arriba)
        df = pd.read_parquet(path)
    else:
        df = pd.read_csv(path)
    # normalizar variantes de tiempo
    ren = {}
    if "start_sec" not in df.columns and "start_s" in df.columns: ren["start_s"] = "start_sec"
    if "end_sec"   not in df.columns and "end_s"   in df.columns: ren["end_s"]   = "end_sec"
    if ren: df = df.rename(columns=ren)
    return df, Path(path).parent.name

def basic_predict_proba_df(df_feat):
    """imp + scaler + modelo básico → probabilidades multilabel (N, C)."""
    Xdf  = _align_features_df_any(df_feat, feature_cols)
    Ximp = imp.transform(Xdf)
    Xs   = scaler.transform(Ximp)
    proba = model_basic.predict_proba(Xs)
    proba = np.asarray(proba, dtype=np.float32)
    if proba.ndim == 1:
        proba = proba[:, None]
    return proba

def build_sequence_dataset_from_features(files, seq_len=12, hop=6):
    X_list, y_list, vids = [], [], []
    for f in files:
        df, vid = load_video_features(f)
        # asegurar columnas del modelo básico
        for c in feature_cols:
            if c not in df.columns:
                df[c] = np.nan
        Xw = df[feature_cols].astype(np.float32)
        Yw = basic_predict_proba_df(Xw)  # (Nw, C)

        N = len(Xw)
        if N < seq_len:
            continue
        for s in range(0, N - seq_len + 1, hop):
            e = s + seq_len
            X_list.append(Xw.iloc[s:e].to_numpy(np.float32))
            y_list.append(Yw[s:e].mean(axis=0).astype(np.float32))
            vids.append(vid)

    if not X_list:
        return None

    X = np.stack(X_list, 0)  # (Nseq, T, F)
    y = np.stack(y_list, 0)  # (Nseq, C)
    vids = np.array(vids)

    uniq = np.unique(vids)
    if len(uniq) >= 2:
        np.random.shuffle(uniq)
        n_val = max(1, int(round(0.2 * len(uniq))))
        val_set = set(uniq[:n_val])
        tr = np.array([v not in val_set for v in vids]); va = ~tr
    else:
        n = len(X)
        cut = max(1, int(0.8 * n))
        tr = np.zeros(n, bool); tr[:cut] = True; va = ~tr

    return {
        "X_train": X[tr], "y_train": y[tr],
        "X_val":   X[va], "y_val":   y[va],
        "vids": vids
    }

print(" Artefactos y utilidades cargados correctamente.")
print("   - #feature_cols:", len(feature_cols), "| #labels:", len(label_names))
print("   - MODELS_DIR:", MODELS_DIR)
print("   - REPORTS_DIR:", REPORTS_DIR)


In [ ]:
#  Setup, imports, rutas, hiperparámetros y flags

import os, re, json, pickle, math, random, warnings, time, gc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

try:
    import seaborn as sns
    _HAS_SNS = True
except Exception:
    _HAS_SNS = False

warnings.filterwarnings("ignore")

# ---- Rutas ----
BASE        = "/content/drive/MyDrive/asistente_coreografico"
DATA_DIR    = f"{BASE}/data/annotations/aistpp/aist_plusplus_final"
MODELS_DIR  = f"{BASE}/models"
REPORTS_DIR = f"{BASE}/reports"
ART         = f"{BASE}/artifacts"
os.makedirs(MODELS_DIR, exist_ok=True)
os.makedirs(REPORTS_DIR, exist_ok=True)
os.makedirs(ART, exist_ok=True)

# ---- Hiperparámetros ----
SEQ_LEN    = 30
HOP        = 15
MODEL_TYPE = "lstm"    # 'lstm' | 'bilstm' | 'transformer'
BATCH_SIZE = 32
EPOCHS     = 50
PATIENCE   = 10
SEED       = 42

# ---- Flags de ejecución ----
USE_CACHE      = True     # usa/crea cache de X,y,label_names en ART
CLEAR_CACHE    = False    # fuerza ignorar caches
LIMIT_FILES    = None     # p.ej. 200 para desarrollo rápido; None = todos
VERBOSE_DATA   = True

# ---- Semillas y GPU ---
np.random.seed(SEED); random.seed(SEED); tf.random.set_seed(SEED)
try:
    gpus = tf.config.list_physical_devices('GPU')
    for g in gpus:
        tf.config.experimental.set_memory_growth(g, True)
    if gpus: print(f"[GPU] {len(gpus)} GPU(s) detectadas.")
except Exception as e:
    print("[GPU] Mem growth no aplicado:", e)

print("[OK] Entorno inicializado")
print("TF:", tf.__version__)
print("DATA_DIR:", DATA_DIR)


In [ ]:
# Conversión y limpieza de keypoints

def to_TJD_from_any(obj):
    """Devuelve (T,J,D) con D in {2,3} o None."""
    arr = None
    if isinstance(obj, dict):
        for k in ["keypoints3d", "joints3d", "poses_3d", "kp3d", "keypoints"]:
            if k in obj:
                arr = np.array(obj[k]); break
        if arr is None:
            for v in obj.values():
                if isinstance(v, np.ndarray):
                    arr = v; break
    elif isinstance(obj, np.ndarray):
        arr = obj
    else:
        return None

    arr = np.asarray(arr)
    if arr.ndim != 3:
        return None

    T, A, B = arr.shape
    for c in [arr, np.transpose(arr, (1,0,2)), np.transpose(arr, (0,2,1))]:
        if c.shape[-1] in (2,3):
            if c.shape[0] >= c.shape[1]:
                return c.astype(np.float32)
            else:
                return np.transpose(c, (1,0,2)).astype(np.float32)
    if A in (17,24,25) and B in (2,3):
        return np.transpose(arr, (1,0,2)).astype(np.float32)
    if B in (17,24,25) and A in (2,3):
        return np.transpose(arr, (0,2,1)).astype(np.float32)
    return None

def clean_nan_interpolate(kp_TJD, min_valid_ratio=0.10):
    """Interpola NaNs temporalmente; descarta si ratio finitos < umbral."""
    x = kp_TJD.copy().astype(np.float32)
    valid_ratio = np.isfinite(x).mean()
    if valid_ratio < min_valid_ratio:
        return None, valid_ratio
    T,J,D = x.shape
    for j in range(J):
        for d in range(D):
            series = x[:,j,d]
            idx = np.where(np.isfinite(series))[0]
            if len(idx) == 0:
                x[:,j,d] = 0.0; continue
            first, last = idx[0], idx[-1]
            series[:first] = series[first]
            series[last+1:] = series[last]
            miss = np.where(~np.isfinite(series))[0]
            if len(miss) > 0:
                good = np.where(np.isfinite(series))[0]
                x[miss,j,d] = np.interp(miss, good, series[good])
    return x, valid_ratio

def extract_label_from_filename(fname):
    base = os.path.splitext(fname)[0]
    m = re.search(r"(ch\d+)", base)
    return m.group(1) if m else base

print("[OK] Funciones KP listas")


In [ ]:
def prepare_sequences(kp_list, y_list, seq_len=SEQ_LEN, hop=HOP):
    """Devuelve X:[N,seq_len,J*D], y:[N,C]."""
    X, Y = [], []
    for kp, y in zip(kp_list, y_list):
        T,J,D = kp.shape
        if T <= 0: continue
        for s in range(0, max(1, T - seq_len + 1), hop):
            e = s + seq_len
            if e <= T:
                win = kp[s:e]
            else:
                pad = np.zeros((e - T, J, D), dtype=kp.dtype)
                win = np.concatenate([kp[s:], pad], axis=0)
            X.append(win.reshape(seq_len, J*D))
            Y.append(y)
    X = np.asarray(X, dtype=np.float32)
    Y = np.asarray(Y, dtype=np.float32)
    print(f"[SEQ] X={X.shape} y={Y.shape} (seq_len={seq_len}, hop={hop})")
    return X, Y

def dataset_cache_paths():
    tag = f"seq{SEQ_LEN}_hop{HOP}"
    return (os.path.join(ART, f"aist_{tag}_X.npz"),
            os.path.join(ART, f"aist_{tag}_y.npz"),
            os.path.join(ART, f"aist_{tag}_label_names.json"))

def load_or_build_dataset():
    X_path, y_path, ln_path = dataset_cache_paths()
    if CLEAR_CACHE:
        for p in [X_path, y_path, ln_path]:
            if os.path.exists(p): os.remove(p)
        print("[CACHE] Limpiado.")

    if USE_CACHE and all(os.path.exists(p) for p in [X_path, y_path, ln_path]):
        print("[CACHE] Cargando dataset desde cache...")
        X = np.load(X_path)["X"]; y = np.load(y_path)["y"]
        label_names = list(json.load(open(ln_path)))
        print(f"[CACHE] X:{X.shape} y:{y.shape} | clases={len(label_names)}")
        return X, y, label_names

    print("[BUILD] Generando dataset (esto puede tardar)...")
    kp_list, y_list, label_names = load_aist_data()
    X, y = prepare_sequences(kp_list, y_list, SEQ_LEN, HOP)

    if USE_CACHE:
        np.savez_compressed(X_path, X=X)
        np.savez_compressed(y_path, y=y)
        with open(ln_path, "w") as f: json.dump(label_names, f, indent=2, ensure_ascii=False)
        print(f"[CACHE] Guardado: {X_path} | {y_path} | {ln_path}")

    # libera memoria
    del kp_list, y_list; gc.collect()
    return X, y, label_names

print("[OK] Ventanas + cache listo")

In [ ]:

def create_lstm_model(input_shape, num_labels):
    model = tf.keras.Sequential([
        tf.keras.layers.Masking(mask_value=0., input_shape=input_shape),
        tf.keras.layers.LSTM(128, return_sequences=True, dropout=0.3, recurrent_dropout=0.2),
        tf.keras.layers.LSTM(64, dropout=0.3, recurrent_dropout=0.2),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dropout(0.4),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(num_labels, activation='sigmoid')
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(1e-3),
                  loss='binary_crossentropy',
                  metrics=['accuracy', 'binary_accuracy'])
    return model

def create_bilstm_model(input_shape, num_labels):
    model = tf.keras.Sequential([
        tf.keras.layers.Masking(mask_value=0., input_shape=input_shape),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True, dropout=0.3, recurrent_dropout=0.2)),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, dropout=0.3, recurrent_dropout=0.2)),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dropout(0.4),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(num_labels, activation='sigmoid')
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(1e-3),
                  loss='binary_crossentropy',
                  metrics=['accuracy', 'binary_accuracy'])
    return model

def create_transformer_model(input_shape, num_labels):
    """Transformer simple con proyección para residual."""
    seq_len, d_model = input_shape
    inputs = tf.keras.Input(shape=input_shape)

    pos = tf.range(start=0, limit=seq_len, delta=1)
    pos_emb = tf.keras.layers.Embedding(input_dim=seq_len, output_dim=d_model)(pos)  # [T,d_model]
    x = inputs + pos_emb  # broadcast sobre batch

    attn = tf.keras.layers.MultiHeadAttention(num_heads=4, key_dim=max(1, d_model//8))(x, x)
    attn = tf.keras.layers.Dense(d_model)(attn)  # asegura misma dim para residual
    x = tf.keras.layers.Add()([x, attn])
    x = tf.keras.layers.LayerNormalization(epsilon=1e-6)(x)

    ffn = tf.keras.Sequential([tf.keras.layers.Dense(128, activation='relu'),
                               tf.keras.layers.Dense(d_model)])(x)
    x = tf.keras.layers.Add()([x, ffn])
    x = tf.keras.layers.LayerNormalization(epsilon=1e-6)(x)

    x = tf.keras.layers.GlobalAveragePooling1D()(x)
    x = tf.keras.layers.Dropout(0.4)(x)
    x = tf.keras.layers.Dense(64, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.3)(x)
    outputs = tf.keras.layers.Dense(num_labels, activation='sigmoid')(x)

    model = tf.keras.Model(inputs, outputs)
    model.compile(optimizer=tf.keras.optimizers.Adam(1e-3),
                  loss='binary_crossentropy',
                  metrics=['accuracy', 'binary_accuracy'])
    return model

print("[OK] Modelos listos")

In [ ]:
def plot_training_history(history, model_type):
    hist = history.history
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

    ax1.plot(hist.get('loss', []), label='Train')
    if 'val_loss' in hist: ax1.plot(hist['val_loss'], label='Val')
    ax1.set_title(f'Pérdida - {model_type.upper()}'); ax1.set_xlabel('Época'); ax1.set_ylabel('Loss'); ax1.legend()

    mkey = 'binary_accuracy' if 'binary_accuracy' in hist else ('accuracy' if 'accuracy' in hist else None)
    if mkey:
        ax2.plot(hist[mkey], label='Train')
        if f"val_{mkey}" in hist: ax2.plot(hist[f"val_{mkey}"], label='Val')
        ax2.set_title(f'{mkey} - {model_type.upper()}'); ax2.set_xlabel('Época'); ax2.set_ylabel(mkey); ax2.legend()
    else:
        ax2.axis('off')

    plt.tight_layout()
    out_png = os.path.join(REPORTS_DIR, f"{MODEL_TYPE}_training_curves.png")
    plt.savefig(out_png, dpi=150); plt.show()
    print(f"[OK] Curvas guardadas en: {out_png}")

def _safe_evaluate(model, X, y):
    try:
        out = model.evaluate(X, y, verbose=0, return_dict=True)
        if isinstance(out, dict):
            return out
    except TypeError:
        pass
    vals = model.evaluate(X, y, verbose=0)
    names = getattr(model, "metrics_names", None)
    if names and isinstance(vals, (list, tuple)):
        return {n: float(v) for n, v in zip(names, vals)}
    return {"loss": float(vals) if np.isscalar(vals) else float(vals[0])}

def generate_evaluation_report(model, X_test, y_test, model_type, label_names):
    y_pred = model.predict(X_test, verbose=0)
    y_bin  = (y_pred > 0.5).astype(int)

    if len(label_names) != y_test.shape[1]:
        print("[WARN] target_names ≠ nº clases; usaré nombres genéricos.")
        label_names = [f"cls_{i}" for i in range(y_test.shape[1])]

    report = classification_report(y_test, y_bin, target_names=label_names, output_dict=True, zero_division=0)
    rp_path = os.path.join(REPORTS_DIR, f"{model_type}_classification_report.csv")
    pd.DataFrame(report).transpose().to_csv(rp_path)
    print(f"[OK] Reporte de clasificación en: {rp_path}")

    # Matriz de confusión para UNA etiqueta (binaria)
    if _HAS_SNS and y_test.shape[1] > 0:
        present = np.where(np.any(y_test, axis=0) | np.any(y_bin, axis=0))[0]
        if len(present) > 0:
            idx0 = int(present[0])
            cm = confusion_matrix(y_test[:, idx0], y_bin[:, idx0])
            plt.figure(figsize=(7,6))
            sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                        xticklabels=["Pred 0","Pred 1"],
                        yticklabels=["Real 0","Real 1"])
            plt.title(f'Confusión - {model_type.upper()} - {label_names[idx0]}')
            plt.ylabel('Real'); plt.xlabel('Predicha')
            cm_path = os.path.join(REPORTS_DIR, f"{model_type}_confusion_matrix.png")
            plt.savefig(cm_path, dpi=150); plt.show()
            print(f"[OK] Matriz de confusión en: {cm_path}")
        else:
            print("[INFO] Sin clases presentes para matriz de confusión.")
    else:
        print("[INFO] seaborn no disponible o sin clases -> sin matriz de confusión.")

    metrics = _safe_evaluate(model, X_test, y_test)
    metrics.update({
        "model_type": model_type,
        "input_shape": tuple(int(x) for x in X_test.shape[1:]),
        "num_samples": int(len(X_test))
    })
    with open(os.path.join(REPORTS_DIR, f"{model_type}_metrics.json"), "w") as f:
        json.dump({k: float(v) if not isinstance(v, str) else v for k, v in metrics.items()}, f, indent=2)
    print(f"[OK] Métricas guardadas en: {os.path.join(REPORTS_DIR, f'{model_type}_metrics.json')}")

In [ ]:
#Entrenamiento + Evaluación

def train_and_evaluate():
    X, y, label_names = load_or_build_dataset()
    assert len(X) > 0 and len(y) > 0, "No hay datos preparados."

    unique_classes = np.unique(np.argmax(y, axis=1))
    if len(unique_classes) > 1:
        y_int = np.argmax(y, axis=1)
        try:
            X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.2, random_state=SEED, stratify=y_int)
        except ValueError as e:
            print("[WARN] Stratify falló:", e)
            X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.2, random_state=SEED)
        try:
            y_tr_int = np.argmax(y_tr, axis=1)
            X_tr, X_va, y_tr, y_va = train_test_split(X_tr, y_tr, test_size=0.2, random_state=SEED, stratify=y_tr_int)
        except ValueError as e:
            print("[WARN] Stratify val falló:", e)
            X_tr, X_va, y_tr, y_va = train_test_split(X_tr, y_tr, test_size=0.2, random_state=SEED)
    else:
        print("[WARN] Solo una clase. Split sin estratificar.")
        X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.2, random_state=SEED)
        X_tr, X_va, y_tr, y_va = train_test_split(X_tr, y_tr, test_size=0.2, random_state=SEED)

    print(f"[SPLIT] Train={X_tr.shape} | Val={X_va.shape} | Test={X_te.shape}")

    input_shape = X_tr.shape[1:]
    num_labels  = y_tr.shape[1]

    if MODEL_TYPE == "lstm":
        model = create_lstm_model(input_shape, num_labels)
    elif MODEL_TYPE == "bilstm":
        model = create_bilstm_model(input_shape, num_labels)
    elif MODEL_TYPE == "transformer":
        model = create_transformer_model(input_shape, num_labels)
    else:
        raise ValueError("MODEL_TYPE debe ser 'lstm' | 'bilstm' | 'transformer'.")

    ckpt_path = os.path.join(MODELS_DIR, f"best_{MODEL_TYPE}_model.h5")
    callbacks = [
        EarlyStopping(monitor='val_loss', patience=PATIENCE, restore_best_weights=True),
        ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=max(1, PATIENCE//2), min_lr=1e-7, verbose=1),
        ModelCheckpoint(ckpt_path, monitor='val_loss', save_best_only=True, save_weights_only=False, verbose=1)
    ]

    print(f"[TRAIN] {MODEL_TYPE.upper()} ...")
    history = model.fit(
        X_tr, y_tr,
        validation_data=(X_va, y_va),
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        callbacks=callbacks,
        verbose=1
    )

    metrics = _safe_evaluate(model, X_te, y_te)
    loss = metrics.get("loss", np.nan)
    acc  = metrics.get("accuracy", np.nan)
    bacc = metrics.get("binary_accuracy", np.nan)
    print(f"[TEST] loss={loss:.4f} | acc={acc:.4f} | bin_acc={bacc:.4f}")

    final_path = os.path.join(MODELS_DIR, f"final_{MODEL_TYPE}_model.h5")
    model.save(final_path)
    print(f"[OK] Modelo final guardado en: {final_path}")

    hist_path = os.path.join(REPORTS_DIR, f"{MODEL_TYPE}_training_history.csv")
    pd.DataFrame(history.history).to_csv(hist_path, index=False)
    print(f"[OK] Historial guardado en: {hist_path}")

    plot_training_history(history, MODEL_TYPE)
    generate_evaluation_report(model, X_te, y_te, MODEL_TYPE, label_names)

    return model, history, metrics, label_names

print("[OK] train_and_evaluate listo")


In [ ]:
# Demo con best model

def run_demo_with_best_model(label_names):
    best_path = os.path.join(MODELS_DIR, f"best_{MODEL_TYPE}_model.h5")
    if not os.path.exists(best_path):
        print("[INFO] No hay best model; salto demo.")
        return
    print(f"[DEMO] Cargando: {best_path}")
    model = tf.keras.models.load_model(best_path)

    X, y, _ = load_or_build_dataset()
    if len(X) == 0:
        print("[DEMO] No hay datos."); return
    demo_idx = 0
    y_pred = model.predict(X[demo_idx:demo_idx+1], verbose=0)[0]
    top = np.argsort(y_pred)[::-1][:min(5, len(label_names))]
    print("\n=== DEMO: Top predicciones ===")
    for i, j in enumerate(top, 1):
        print(f"{i}. {label_names[j]}: {y_pred[j]:.3f}")

    plt.figure(figsize=(10,6))
    plt.bar(range(len(y_pred)), y_pred)
    plt.xlabel("Clases"); plt.ylabel("Prob."); plt.title(f"Predicciones ({MODEL_TYPE.upper()})")
    plt.xticks(range(len(label_names)), label_names, rotation=45, ha='right')
    plt.tight_layout()
    demo_png = os.path.join(REPORTS_DIR, f"{MODEL_TYPE}_demo_predictions.png")
    plt.savefig(demo_png, dpi=150); plt.show()
    print(f"[OK] Demo guardada en: {demo_png}")

print("[OK] Demo listo")


In [ ]:
# === Sustituye SOLO esta función por la versión de abajo ===
import tempfile, shutil
import tensorflow as tf

def export_best_model_to_onnx(model_type=None, models_dir=None, opset_candidates=(17, 15, 13)):
    """
    Exporta a ONNX de forma robusta:
      1) Carga el mejor .keras/.h5 (o lo reconstruye si sólo hay pesos).
      2) Intenta convertir con ConcreteFunction (from_function).
      3) Si falla, exporta a SavedModel y convierte desde ahí.
    """
    model_type = model_type or MODEL_TYPE
    models_dir = models_dir or MODELS_DIR

    # 1) Obtener el modelo listo
    best_path = _ensure_best_model_file(model_type)  # usa tu helper
    model = tf.keras.models.load_model(best_path)

    # 2) Deducir la forma de entrada (T,F)
    in_shape = model.inputs[0].shape  # (None, T, F)
    T = int(in_shape[1]) if in_shape[1] is not None else SEQ_LEN
    Fok = int(in_shape[2]) if in_shape[2] is not None else (F or 0)
    assert T > 0 and Fok > 0, f"Input shape inválido: {in_shape}"

    # 3) Firma de entrada explícita
    spec = [tf.TensorSpec([None, T, Fok], tf.float32, name="input")]
    onnx_out = os.path.join(models_dir, f"best_{model_type}_model.onnx")

    last_err = None
    for ops in opset_candidates:
        print(f"[ONNX] Exportando con opset={ops} ...")

        # ---- A) Intento: ConcreteFunction (compatible Keras 3) ----
        try:
            @tf.function(input_signature=spec)
            def _wrapped(x):
                return model(x)
            concrete = _wrapped.get_concrete_function()

            _onnx_model, _ = tf2onnx.convert.from_function(
                concrete_function=concrete,
                opset=ops,
                input_signature=spec,
                output_path=onnx_out
            )
            print(f"[OK] Exportado (from_function) → {onnx_out}")
            return onnx_out, (T, Fok)
        except Exception as e_func:
            last_err = e_func
            print(f"[WARN] Falla from_function (opset={ops}): {e_func}")

        # ---- B) Fallback: SavedModel → ONNX ----
        tmpdir = tempfile.mkdtemp(prefix="savedmodel_for_onnx_")
        try:
            # Keras 3 tiene .export; si no, usamos tf.saved_model.save
            try:
                model.export(tmpdir)  # Keras 3
            except Exception:
                tf.saved_model.save(model, tmpdir)  # Keras 2.x compat

            _onnx_model, _ = tf2onnx.convert.from_saved_model(
                saved_model_dir=tmpdir,
                opset=ops,
                output_path=onnx_out
            )
            print(f"[OK] Exportado (from_saved_model) → {onnx_out}")
            return onnx_out, (T, Fok)
        except Exception as e_sm:
            last_err = (last_err, e_sm)
            print(f"[WARN] Falla from_saved_model (opset={ops}): {e_sm}")
        finally:
            shutil.rmtree(tmpdir, ignore_errors=True)

    raise RuntimeError(f"No se pudo exportar a ONNX. Último error: {last_err}")


In [ ]:
# INTEGRACION FINAL: PIPELINE COMPLETO para un vídeo subido por el usuario
# - Sube vídeo del usuario
# - Procesa con MediaPipe para keypoints
# - Ejecuta análisis básico (features + sugerencias básicas)
# - Carga/usa modelo avanzado (LSTM/Transformer entrenado previamente)
# - Ejecuta inferencia avanzada (predicción por secuencias)
# - Combina sugerencias (básicas + avanzadas)
# - Visualiza resultados (anota vídeo)


import os, json, math, io, glob, tempfile
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import cv2
import mediapipe as mp
from sklearn.metrics import classification_report, roc_auc_score, average_precision_score, precision_recall_curve, roc_curve
from joblib import load
from google.colab import files

# --- Variables base (compatibles con celdas 1–11) ---
try:
    BASE
except NameError:
    BASE = "/content/drive/MyDrive/asistente_coreografico"

try:
    MODEL_TYPE
except NameError:
    MODEL_TYPE = "lstm"  # 'lstm' | 'bilstm' | 'transformer'

MODELS_DIR  = f"{BASE}/models"
REPORTS_DIR = f"{BASE}/reports"
ART_DIR     = f"{BASE}/artifacts"
os.makedirs(MODELS_DIR, exist_ok=True)
os.makedirs(REPORTS_DIR, exist_ok=True)

# --- Cargar artefactos del modelo básico (si existen) ---
try:
    imp          = load(f"{ART_DIR}/imputer.joblib")
    scaler       = load(f"{ART_DIR}/scaler.joblib")
    model_basic  = load(f"{ART_DIR}/model_ovr_logreg.joblib")
    feature_cols = pd.read_csv(f"{ART_DIR}/feature_cols.csv", header=None)[0].tolist()
    label_names  = pd.read_csv(f"{ART_DIR}/label_names.csv", header=None)[0].tolist()
    print("Loaded basic model artifacts.")
except Exception as e:
    print(f"Could not load basic model artifacts: {e}")
    imp = None; scaler = None; model_basic = None
    feature_cols = []
    label_names  = ["label1", "label2", "label3"]  # fallback

# --- Unificar label_names con clases del modelo avanzado (si existen) ---
classes_json = os.path.join(ART_DIR, "classes.json")
if os.path.exists(classes_json):
    try:
        label_names_adv = json.load(open(classes_json))
        if isinstance(label_names_adv, list) and len(label_names_adv) > 0:
            label_names = label_names_adv
            print(f"Using label_names from classes.json ({len(label_names)})")
    except Exception as e:
        print("WARN: could not read classes.json:", e)

# --- Mapa de sugerencias ---
try:
    label_to_text
except NameError:
    label_to_text = {ln: f"Sugerencia para {ln}" for ln in label_names}

# --- Utilidades ---
def ensure_dir(p): os.makedirs(p, exist_ok=True)

def sliding_windows(X, win_len=100, hop=20):
    """X: (T, F) -> (Nw, win_len, F) + índices [(s,e),...]. Padding si T<win_len."""
    X = np.asarray(X, dtype=np.float32)
    T, F = X.shape
    if T <= 0:
        return np.zeros((0, win_len, F), dtype=np.float32), []
    if T < win_len:
        pad = np.zeros((win_len - T, F), dtype=np.float32)
        Xp = np.vstack([X, pad])
        return np.expand_dims(Xp, 0), [(0, min(T, win_len))]
    idx, ws = [], []
    for s in range(0, T - win_len + 1, hop):
        e = s + win_len
        ws.append(X[s:e])
        idx.append((s, e))
    return np.stack(ws, axis=0), idx

def plot_topk_bars(scores_vec, labels, k=10, title="Top etiquetas"):
    scores = np.array(scores_vec).astype(float)
    if scores.ndim > 1: scores = scores.mean(axis=0)
    k = min(k, len(labels))
    idx = np.argsort(scores)[::-1][:k]
    plt.figure(figsize=(8, max(4, 0.45*k)))
    plt.barh(range(k), scores[idx])
    plt.yticks(range(k), [labels[i] for i in idx]); plt.gca().invert_yaxis()
    plt.xlabel("Score"); plt.title(title); plt.tight_layout(); plt.show()

def plot_window_heatmap(win_scores, labels, win_idx, title="Heatmap temporal (ventanas x etiquetas)"):
    if win_scores is None or len(win_scores)==0:
        print("No hay puntuaciones por ventana para heatmap.")
        return
    H = np.asarray(win_scores)
    plt.figure(figsize=(min(14, 2 + 0.6*H.shape[0]), 0.5*H.shape[1] + 2))
    plt.imshow(H.T, aspect="auto", interpolation="nearest")
    plt.yticks(range(len(labels)), labels)
    xt = [f"{s}-{e}" for (s,e) in win_idx]
    plt.xticks(range(len(win_idx)), xt, rotation=45, ha='right')
    plt.xlabel("Ventanas (frames)"); plt.ylabel("Etiquetas")
    plt.title(title); plt.tight_layout(); plt.show()

def overlay_text_on_video(video_path, timeline, out_path, max_lines=4):
    """Escribe sugerencias activas en cada frame y exporta MP4 anotado."""
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"No se pudo abrir el vídeo: {video_path}")
        return None
    W = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    H = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    FPS = cap.get(cv2.CAP_PROP_FPS) or 30.0
    vw = cv2.VideoWriter(out_path, cv2.VideoWriter_fourcc(*'mp4v'), FPS, (W,H))
    t = 0
    while True:
        ok, frame = cap.read()
        if not ok:
            break
        active = []
        for seg in timeline:
            if int(seg["start_f"]) <= t <= int(seg["end_f"]):
                active = seg.get("sugerencias", [])
        y0 = 30
        for s in active[:max_lines]:
            cv2.putText(frame, f"• {s}", (20, y0), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,0,0), 4, cv2.LINE_AA)
            cv2.putText(frame, f"• {s}", (20, y0), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,255,255), 1, cv2.LINE_AA)
            y0 += 28
        vw.write(frame); t += 1
    cap.release(); vw.release()
    return out_path

def _normalize_combined_suggestions(sug_list, default_conf=0.4, default_source="basic_model"):
    """Normaliza lista de sugerencias a [{'suggestion','confidence','source'}, ...]."""
    out = []
    for s in (sug_list or []):
        if isinstance(s, dict):
            out.append({
                "suggestion": str(s.get("suggestion","")),
                "confidence": float(s.get("confidence", default_conf)),
                "source": s.get("source", default_source)
            })
        else:
            out.append({"suggestion": str(s), "confidence": float(default_conf), "source": default_source})
    return out

# --- Entrada / extracción de vídeo ---
def upload_video():
    """Permite al usuario subir un vídeo desde su dispositivo (interfaz de Colab)."""
    uploaded = files.upload()
    if not uploaded:
        print("No se subió ningún archivo.")
        return None
    video_name = list(uploaded.keys())[0]
    with tempfile.NamedTemporaryFile(delete=False, suffix=os.path.splitext(video_name)[1]) as temp_video_file:
        temp_video_file.write(uploaded[video_name])
        video_path = temp_video_file.name
    print(f"Vídeo '{video_name}' subido y guardado temporalmente en: {video_path}")
    return video_path

def process_video_with_mediapipe(video_path, output_dir):
    """Procesa vídeo con MediaPipe Pose y guarda keypoints (33*4 por frame)."""
    mp_pose = mp.solutions.pose
    pose = mp_pose.Pose(static_image_mode=False, model_complexity=2, enable_segmentation=False, min_detection_confidence=0.5)
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"No se pudo abrir el vídeo para MediaPipe: {video_path}")
        return None, None
    all_keypoints = []; frame_count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret: break
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(rgb_frame)
        if results.pose_landmarks:
            frame_keypoints = []
            for landmark in results.pose_landmarks.landmark:
                frame_keypoints.extend([landmark.x, landmark.y, landmark.z, landmark.visibility])
            all_keypoints.append(frame_keypoints)
        else:
            all_keypoints.append([np.nan] * 33 * 4)
        frame_count += 1
        if frame_count % 100 == 0:
            print(f"Procesados {frame_count} frames...")
    cap.release()
    if not all_keypoints:
        print("No keypoints extracted from the video.")
        return None, None
    keypoints_array = np.array(all_keypoints)
    output_path = os.path.join(output_dir, "custom_video_keypoints.npy")
    np.save(output_path, keypoints_array)
    print(f"Keypoints saved to: {output_path}")
    return keypoints_array, output_path

# --- Features coreográficos básicos con MediaPipe ---
def interpolate_nan_1d(y):
    y = y.astype(np.float32); T = len(y); idx = np.arange(T); mask = np.isfinite(y)
    if mask.sum() == 0: return y
    last = np.nan
    for i in range(T):
        if not np.isfinite(y[i]) and np.isfinite(last): y[i] = last
        else: last = y[i]
    last = np.nan
    for i in range(T-1,-1,-1):
        if not np.isfinite(y[i]) and np.isfinite(last): y[i] = last
        else: last = y[i]
    mask = np.isfinite(y)
    if 2 <= mask.sum() < T: y[~mask] = np.interp(idx[~mask], idx[mask], y[mask])
    return y

def clean_nan_interpolate(K, min_valid_ratio=0.10):
    Kc = K.copy(); T,J,D = Kc.shape; used = []
    for j in range(J):
        valid = np.isfinite(Kc[:,j,:]).all(axis=1)
        if valid.mean() < min_valid_ratio: Kc[:,j,:] = np.nan; continue
        for d in range(D): Kc[:,j,d] = interpolate_nan_1d(Kc[:,j,d])
        if np.isfinite(Kc[:,j,:]).any(): used.append(j)
    if not used:
        print("No hay articulaciones válidas tras limpieza e interpolación.")
        return Kc, []
    return Kc[:,used,:], used

def features_coreograficos_mediapipe(K):
    """Resumen de features robusto para sugerencias básicas."""
    T,J,D = K.shape
    K_2d = K[:,:,:2]

    amp_mean = (np.nanmax(K_2d, axis=0) - np.nanmin(K_2d, axis=0)).mean(axis=0)
    amp_x = float(amp_mean[0]); amp_y = float(amp_mean[1])

    vel = np.linalg.norm(np.diff(K_2d, axis=0), axis=2)
    vel_media = float(np.nanmean(vel)) if vel.size else 0.0

    MEDIAPIPE_COCO_PAIRS_SIMPLIFIED = [(11,12), (13,14), (15,16), (23,24), (25,26), (27,28)]
    pair_vals=[]
    for a,b in MEDIAPIPE_COCO_PAIRS_SIMPLIFIED:
        if a < J and b < J:
            diff = K_2d[:,a,:] - K_2d[:,b,:]     # [T,2]
            d = np.linalg.norm(diff, axis=1)     # [T]
            pair_vals.append(np.nanmean(d))
    sim_raw = float(np.nanmean(pair_vals)) if pair_vals else np.nan

    com = (K[:,23,:2] + K[:,24,:2]) / 2 if J >= 25 else K[:,:,:2].mean(axis=1)
    nivel_p10 = float(np.nanpercentile(com[:,1], 10))
    nivel_p90 = float(np.nanpercentile(com[:,1], 90))
    nivel_rango = float(nivel_p10 - nivel_p90)

    disp = np.diff(com,axis=0)
    dirs = np.arctan2(disp[:,1], disp[:,0] + 1e-9)
    cambios = np.abs(np.diff(dirs))
    variedad_dir = float(np.nanmean(cambios)) if len(cambios)>0 else 0.0

    return {
        "amplitud_x": amp_x, "amplitud_y": amp_y, "velocidad_media": vel_media,
        "simetria": sim_raw, "nivel_rango": nivel_rango, "variedad_direcciones": variedad_dir,
        "frames": int(T), "joints": int(J), "dims": int(D)
    }

def inferir_desde_features_dict(feats: dict):
    """Inferencia con pipeline básico (imputer + scaler + OVR-LogReg)."""
    if imp is None or scaler is None or model_basic is None or not feature_cols or not label_names:
        print("Cannot perform basic inference: missing basic model artifacts or labels/features.")
        return [], []
    x = np.array([[feats.get(c, np.nan) for c in feature_cols]], dtype=float)
    x_imp = imp.transform(x)
    x_scaled = scaler.transform(x_imp)
    if hasattr(model_basic, 'predict'):
        yhat = model_basic.predict(x_scaled)[0]
        predicted_labels = [label_names[i] for i, v in enumerate(yhat) if v==1]
        sugerencias = [label_to_text.get(lbl, lbl) for lbl in predicted_labels]
        return predicted_labels, sugerencias
    else:
        print("Basic model does not have 'predict'.")
        return [], []

def run_basic_analysis(video_path, keypoints_array, output_dir, win_sec=5.0, hop_sec=2.5):
    """Ejecuta análisis básico por ventanas; devuelve timeline y keypoints procesados."""
    cap = cv2.VideoCapture(video_path)
    FPS = cap.get(cv2.CAP_PROP_FPS) or 30.0
    cap.release()

    T = keypoints_array.shape[0]; J = 33; D = 4
    K_raw = keypoints_array.reshape(T, J, D)

    Kc, used = clean_nan_interpolate(K_raw, min_valid_ratio=0.10)
    if not used:
        print("No hay articulaciones válidas tras limpieza/interpolación.")
        return {"csv": None, "video": video_path, "timeline": [], "processed_keypoints": None, "suggestions": []}

    win = max(1, int(round(win_sec * FPS)))
    hop = max(1, int(round(hop_sec * FPS)))
    T_clean = len(Kc)

    rows = []; timeline = []; suggestions_list = []
    for start in range(0, T_clean - 1, hop):
        end = min(T_clean - 1, start + win)
        if end - start < max(8, int(0.25 * win)): break
        Kw = Kc[start:end]
        feats = features_coreograficos_mediapipe(Kw)
        labels, suggs = inferir_desde_features_dict(feats)

        timeline.append({"start_f": int(start), "end_f": int(end), "sugerencias": suggs})
        suggestions_list.extend(suggs)

        row = {"start_f": int(start), "end_f": int(end), "start_s": start / FPS, "end_s": end / FPS}
        row.update(feats)
        row["labels"] = "|".join(labels)
        row["sugerencias"] = "|".join(suggs)
        rows.append(row)

    out_csv = os.path.join(output_dir, "custom_video_suggestions_timeline.csv")
    pd.DataFrame(rows).to_csv(out_csv, index=False, encoding="utf-8")
    print(f"Sugerencias (timeline) guardadas en: {out_csv}")

    return {
        "csv": out_csv,
        "video": video_path,
        "timeline": timeline,
        "processed_keypoints": Kc,
        "suggestions": list(set(suggestions_list))
    }

# --- Modelo avanzado (estructura LSTM Bi) para carga de pesos ---
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers

class CoreographyLSTM:
    def __init__(self, input_shape=(100,51), num_labels=8):
        inp = layers.Input(shape=input_shape)
        x = layers.Masking(mask_value=0.0)(inp)
        x = layers.Bidirectional(layers.LSTM(128, return_sequences=True))(x)
        x = layers.Bidirectional(layers.LSTM(64))(x)
        x = layers.Dense(128, activation="relu")(x)
        out = layers.Dense(num_labels, activation="sigmoid")(x)
        self.model = models.Model(inp, out)
        self.model.compile(optimizer=optimizers.Adam(1e-3),
                           loss="binary_crossentropy",
                           metrics=[tf.keras.metrics.AUC(name="auc"),
                                    tf.keras.metrics.Precision(name="precision"),
                                    tf.keras.metrics.Recall(name="recall")])
    def predict_sequence(self, seq_2d):
        # seq_2d: (T,F) -> ventanas (Nw, win, F), promedio de scores
        Xw, idx = sliding_windows(seq_2d, win_len=self.model.input_shape[1], hop=20)
        if len(Xw)==0: return np.zeros((self.model.output_shape[-1],), dtype=float)
        probs = self.model.predict(Xw, verbose=0)  # (Nw, C)
        return probs.mean(axis=0), probs, idx

# --- Pipeline Integrado ---
def full_analysis_pipeline(video_path, win_sec=5.0, hop_sec=2.5, conf_threshold=0.5):
    """
    Ejecuta el pipeline completo para un vídeo custom.
    """
    print(f"Iniciando pipeline completo para: {video_path}")
    output_dir_process = os.path.join(BASE, "data/processed/custom_videos")
    ensure_dir(output_dir_process)

    # Keypoints con MediaPipe
    print("1. Extrayendo keypoints con MediaPipe...")
    keypoints_array, keypoints_path = process_video_with_mediapipe(video_path, output_dir_process)
    if keypoints_array is None:
        print("Fallo al extraer keypoints. Abortando pipeline.")
        return None

    # Análisis básico
    print("\n2. Ejecutando análisis básico...")
    basic_results = run_basic_analysis(video_path, keypoints_array, output_dir_process, win_sec=win_sec, hop_sec=hop_sec)
    if basic_results.get("processed_keypoints") is None:
        print("Análisis básico no produjo keypoints procesados. Abortando pipeline.")
        return basic_results

    #  Cargar modelo
    print("\n3. Cargando modelo ...")
    # Prioridad: best_advanced_model.h5 → fallback best_{MODEL_TYPE}_model.h5
    advanced_model_paths = [
        os.path.join(MODELS_DIR, "best_advanced_model.h5"),
        os.path.join(MODELS_DIR, f"best_{MODEL_TYPE}_model.h5")
    ]
    advanced_model_path = next((p for p in advanced_model_paths if os.path.exists(p)), None)
    advanced_model = None

    T_processed, J_processed, D_processed = basic_results["processed_keypoints"].shape

    # Obtener FPS para fijar tamaño de ventana temporal
    cap_tmp = cv2.VideoCapture(video_path)
    fps_tmp = cap_tmp.get(cv2.CAP_PROP_FPS) or 30.0
    cap_tmp.release()
    assumed_input_shape = (int(win_sec * fps_tmp), J_processed * D_processed)
    assumed_num_labels = len(label_names)

    try:
        if advanced_model_path is not None:
            print(f"Using CoreographyLSTM structure for weights: {os.path.basename(advanced_model_path)}")
            advanced_model_keras = CoreographyLSTM(input_shape=assumed_input_shape, num_labels=assumed_num_labels).model
            advanced_model_keras.load_weights(advanced_model_path)
            advanced_model = advanced_model_keras
            print("Advanced model loaded.")
        else:
            print("No advanced model weights found. Skipping advanced analysis.")
    except Exception as e:
        print(f"Could not load advanced model: {e}")
        print("Skipping advanced analysis.")
        advanced_model = None

    #  Preparar estructura de resultados combinados
    combined_results = {
        "basic_analysis": basic_results,
        "advanced_prediction_mean": None,
        "window_scores": None,
        "window_index": None,
        "combined_suggestions": basic_results.get("suggestions", []),  # strings al principio
        "trained_now": False
    }
    combined_results["combined_suggestions"] = _normalize_combined_suggestions(
        combined_results.get("combined_suggestions", [])
    )

    # Inferencia sobre el  modelo
    if advanced_model:
        print("\n4. Ejecutando inferencia avanzada...")
        Kc = basic_results["processed_keypoints"]
        if Kc.ndim == 3:
            T_kc, J_kc, D_kc = Kc.shape
            X_fit_advanced = Kc.reshape(T_kc, J_kc * D_kc)
        else:
            X_fit_advanced = Kc

        # Alinear dimensión de features
        model_input_features = advanced_model.input_shape[-1]
        if X_fit_advanced.shape[1] < model_input_features:
            pad = np.zeros((X_fit_advanced.shape[0], model_input_features - X_fit_advanced.shape[1]), dtype=np.float32)
            X_fit_aligned = np.hstack([X_fit_advanced, pad])
        elif X_fit_advanced.shape[1] > model_input_features:
            X_fit_aligned = X_fit_advanced[:, :model_input_features]
        else:
            X_fit_aligned = X_fit_advanced

        # Ventanas deslizantes
        win_len_model = advanced_model.input_shape[1]
        Xw, idx = sliding_windows(X_fit_aligned, win_len=win_len_model, hop=max(5, win_len_model//5))

        if len(Xw) > 0:
            yw = advanced_model.predict(Xw, verbose=0)  # (Nseq, C)
            y_vec = yw.mean(axis=0)                     # (C,)
            combined_results["advanced_prediction_mean"] = y_vec.tolist()
            combined_results["window_scores"] = yw.tolist()
            combined_results["window_index"] = idx

            # Fusionar sugerencias
            print("\n5. Combinando sugerencias...")
            # Arrancamos con las ya normalizadas (básicas)
            sug = list(combined_results["combined_suggestions"])
            # Añadir avanzadas por umbral global
            for lbl_idx, s in enumerate(y_vec):
                if s >= conf_threshold and lbl_idx < len(label_names):
                    sug_text = label_to_text.get(label_names[lbl_idx], label_names[lbl_idx])
                    if not any(d.get("suggestion") == sug_text for d in sug):
                        sug.append({"suggestion": sug_text, "confidence": float(s), "source": "advanced_model"})
            # Orden final
            combined_results["combined_suggestions"] = sorted(sug, key=lambda x: x.get("confidence", 0.0), reverse=True)

            # Visualizaciones (Top etiquetas + Heatmap)
            print("\n6. Visualizaciones...")
            plot_topk_bars(y_vec, label_names, k=min(12, len(label_names)), title="Top etiquetas (modelo avanzado)")
            plot_window_heatmap(yw, label_names, idx, title="Heatmap temporal (modelo avanzado)")
        else:
            print("No se pudieron construir ventanas para el modelo avanzado. Saltando inferencia avanzada.")

    #  Generar vídeo anotado
    print("\n7. Generando vídeo anotado...")
    timeline_combined = []
    basic_timeline = basic_results.get("timeline", [])
    if basic_timeline:
        # Usa top-K globales por ventana (simple)
        top_k_texts = [d["suggestion"] for d in combined_results["combined_suggestions"][:4]]
        for basic_win in basic_timeline:
            start_f = int(basic_win["start_f"]); end_f = int(basic_win["end_f"])
            timeline_combined.append({"start_f": start_f, "end_f": end_f, "sugerencias": list(top_k_texts)})
    elif combined_results.get("window_index"):
        # Si no hay línea base, usa los índices del modelo avanzado
        idx = combined_results["window_index"]
        win_scores = combined_results.get("window_scores")
        for k, (s, e) in enumerate(idx):
            s = int(s); e = int(e)
            active_suggs_text = []
            if win_scores and len(win_scores) > k:
                # Umbral por ventana (simple)
                arr = np.array(win_scores[k], dtype=np.float32)  # (C,)
                j_idx = np.where(arr >= conf_threshold)[0]
                active_suggs_text = [label_to_text.get(label_names[j], label_names[j]) for j in j_idx if j < len(label_names)]
            if not active_suggs_text:
                active_suggs_text = [d["suggestion"] for d in combined_results["combined_suggestions"][:4]]
            timeline_combined.append({"start_f": s, "end_f": e, "sugerencias": active_suggs_text[:4]})
    else:
        print("No hay timeline de básico ni avanzado. Saltando anotación de vídeo.")
        combined_results["annotated_video"] = None
        print("\nPipeline completo finalizado (sin vídeo anotado).")
        return combined_results

    out_mp4 = os.path.join(REPORTS_DIR, f"custom_video_full_analysis_annotated.mp4")
    annotated_video_path = overlay_text_on_video(video_path, timeline_combined, out_mp4)
    combined_results["annotated_video"] = annotated_video_path
    print(f"Vídeo anotado generado en: {annotated_video_path}")

    #  Guardar resumen
    print("\n8. Guardando resultados combinados...")
    summary_path = os.path.join(REPORTS_DIR, "full_analysis_summary.json")
    summary_data = {
        "video_path": video_path,
        "basic_analysis_csv": basic_results.get("csv"),
        "annotated_video": combined_results.get("annotated_video"),
        "combined_suggestions": combined_results.get("combined_suggestions"),
        "advanced_prediction_mean": combined_results.get("advanced_prediction_mean"),
        "trained_now": combined_results.get("trained_now", False)
    }
    with open(summary_path, "w", encoding="utf-8") as f:
        json.dump(summary_data, f, indent=2, ensure_ascii=False)
    print(f"Resumen del análisis guardado en: {summary_path}")

    print("\nPipeline completo finalizado.")
    return combined_results

# --- Ejecutar el pipeline completo ---
print("Por favor, sube un vídeo custom para iniciar el pipeline.")
try:
    custom_video_path = upload_video()
    if custom_video_path and os.path.exists(custom_video_path):
        print(f"\nEjecutando pipeline completo para: {custom_video_path}")
        final_results = full_analysis_pipeline(custom_video_path)

        print("\n=== Resultados Finales ===")
        if final_results:
            print("Sugerencias Combinadas:")
            if final_results.get("combined_suggestions"):
                for i, sug in enumerate(final_results["combined_suggestions"], 1):
                    print(f"{i}. {sug.get('suggestion', 'N/A')} (Confianza: {sug.get('confidence', np.nan):.2f}) [Fuente: {sug.get('source', 'N/A')}]")
            else:
                print("No combined suggestions generated.")

            if final_results.get("annotated_video"):
                print("\nVídeo anotado disponible en:", final_results["annotated_video"])
            else:
                print("\nVídeo anotado no generado.")

            # Limpiar el archivo de vídeo temporal
            if custom_video_path.startswith(tempfile.gettempdir()):
                os.remove(custom_video_path)
                print(f"Archivo temporal de vídeo eliminado: {custom_video_path}")
        else:
            print("El pipeline completo no se ejecutó correctamente.")
    else:
        print("No se subió ningún vídeo o el archivo no se encontró.")
except Exception as e:
    print(f"Ocurrió un error durante la ejecución del pipeline completo: {e}")
    if 'custom_video_path' in locals() and custom_video_path and os.path.exists(custom_video_path) and custom_video_path.startswith(tempfile.gettempdir()):
        os.remove(custom_video_path)
        print(f"Archivo temporal de vídeo eliminado por error: {custom_video_path}")


In [ ]:
# === GUARDAR MODELO EN /models ===

import os
import json
import joblib
import pandas as pd
import sklearn
from sklearn.pipeline import Pipeline as SkPipeline

# Rutas base
BASE        = "/content/drive/MyDrive/asistente_coreografico"
ART         = f"{BASE}/artifacts"
MODELS_DIR  = f"{BASE}/models"
os.makedirs(MODELS_DIR, exist_ok=True)


# Cargar objetos desde memoria o desde /artifacts

try:
    imp      # SimpleImputer ya en memoria
    scaler   # RobustScaler ya en memoria
    clf      # OneVsRest(LogReg) ya en memoria
except NameError:
    # Si no están en el workspace actual, cargamos desde disco
    imp_path    = f"{ART}/imputer.joblib"
    scaler_path = f"{ART}/scaler.joblib"
    clf_path    = f"{ART}/model_ovr_logreg.joblib"

    assert os.path.exists(imp_path),    f"Falta: {imp_path}"
    assert os.path.exists(scaler_path), f"Falta: {scaler_path}"
    assert os.path.exists(clf_path),    f"Falta: {clf_path}"

    imp    = joblib.load(imp_path)
    scaler = joblib.load(scaler_path)
    clf    = joblib.load(clf_path)


# Cargar columnas de features y nombres de etiquetas

try:
    feature_cols
except NameError:
    feat_path = f"{ART}/feature_cols.csv"
    assert os.path.exists(feat_path), f"Falta: {feat_path}"
    feature_cols = pd.read_csv(feat_path, header=None)[0].tolist()

try:
    label_names
except NameError:
    labels_path = f"{ART}/label_names.csv"
    assert os.path.exists(labels_path), f"Falta: {labels_path}"
    label_names = pd.read_csv(labels_path, header=None)[0].tolist()


# Construir Pipeline y guardar

pipe = SkPipeline([
    ("imputer", imp),     # SimpleImputer ya ajustado
    ("scaler",  scaler),  # RobustScaler ya ajustado
    ("clf",     clf)      # OneVsRest(LogisticRegression) ya ajustado
])

pipe_path = f"{MODELS_DIR}/pipeline_ovr_logreg.joblib"
joblib.dump(pipe, pipe_path)
print(f" Pipeline guardado en: {pipe_path}")

# Metadatos del pipeline
meta = {
    "sklearn_version": sklearn.__version__,
    "model_path": pipe_path,
    "onnx_path": None,  # Export no realizado en esta celda
    "feature_cols": list(feature_cols),
    "label_names": list(label_names),
    "notes": "Pipeline con SimpleImputer + RobustScaler + OneVsRest(LogReg) entrenado."
}
meta_path = f"{MODELS_DIR}/pipeline_metadata.json"
with open(meta_path, "w", encoding="utf-8") as f:
    json.dump(meta, f, indent=2, ensure_ascii=False)
print(f" Metadatos guardados en: {meta_path}")


#  Ejemplo de carga rápida y prueba de inferencia (opcional)

# Si X_raw no existe en el workspace, intenta cargar el CSV de features;
# si no existe o falta alguna columna, no ejecuta la demo.
try:
    X_raw
except NameError:
    X_raw = None

CSV = f"{BASE}/data/processed/aistpp/features_coreograficos.csv"
if X_raw is None and os.path.exists(CSV):
    try:
        df = pd.read_csv(CSV)
        missing = [c for c in feature_cols if c not in df.columns]
        if missing:
            print(f"No se puede crear X_raw: faltan {len(missing)} columnas en el CSV (ej: {missing[:5]})")
        else:
            X_raw = df[feature_cols].copy()
    except Exception as e:
        print(f" No se pudo cargar {CSV}: {e}")

# Carga el pipeline y, si hay X_raw, prueba la inferencia
pipe = joblib.load(pipe_path)

if X_raw is not None:
    try:
        y_pred  = pipe.predict(X_raw)        # 0/1 por etiqueta
        y_proba = pipe.predict_proba(X_raw)  # (n_samples, n_classes)
        print("Pred shape:", y_pred.shape, "| Proba shape:", y_proba.shape)
    except Exception as e:
        print(f" Error en inferencia de ejemplo: {e}")
else:
    print(" Ejemplo de inferencia omitido (no hay X_raw). "
          "Carga un DataFrame y selecciona feature_cols para probar.")


In [ ]:

# Construir ART/kp_windows.npz desde AIST++
# Usa tus funciones: load_aist_data() y prepare_sequences()
# Salida: artifacts/kp_windows.npz con:
#   - 'kp'      -> [N, T, K, D]  (D=2 ó 3, según dataset)
#   - 'y'       -> [N] enteros (clase por ventana)
#   - 'classes' -> lista de nombres de clase
#   - 'mask'    -> [N, T, K] (opcional, 1 si KP finito en todos los dims)

import os, json, numpy as np

# --- Rutas ---
BASE = "/content/drive/MyDrive/asistente_coreografico"
ART  = f"{BASE}/artifacts"
os.makedirs(ART, exist_ok=True)

# --- Chequeo de dependencias (deben existir en celdas previas) ---
for fn in ("load_aist_data", "prepare_sequences"):
    assert fn in globals(), f"❌ Falta la función '{fn}'. Ejecuta las celdas donde se definió."

# --- Parámetros (usa los mismos entrenados) ---
SEQ_LEN = globals().get("SEQ_LEN", 30)
HOP     = globals().get("HOP", 15)

print(f"[INFO] Usando SEQ_LEN={SEQ_LEN}, HOP={HOP}")

# Cargar clips crudos (lista de arrays (T,J,D)) + etiquetas y nombres
kp_list, y_list, label_names = load_aist_data()
assert len(kp_list) > 0, "❌ No hay clips en AIST++."
J, D = kp_list[0].shape[1], kp_list[0].shape[2]
print(f"[DATA] Clips: {len(kp_list)} | J={J} | D={D}")

# Crear ventanas (devuelve X:[N,T,J*D] aplanado y y:[N,C])
X_flat, y_oh = prepare_sequences(kp_list, y_list, seq_len=SEQ_LEN, hop=HOP)
N, T = X_flat.shape[0], X_flat.shape[1]
print(f"[SEQ] Ventanas: N={N}, T={T}, feat={X_flat.shape[2]}  (esperado J*D={J*D})")
assert X_flat.shape[2] == J * D, "❌ El número de features no coincide con J*D. Revisa SEQ_LEN/HOP o los datos."

# Re-dar forma: [N, T, J*D] -> [N, T, J, D]
kp_windows = X_flat.reshape(N, T, J, D).astype(np.float32)

# Etiquetas enteras por ventana y máscara opcional
y_int = np.argmax(y_oh, axis=1).astype(np.int64)  # [N]
mask  = np.isfinite(kp_windows).all(axis=-1).astype(np.uint8)  # [N,T,K]

# Guardar .npz con el formato esperado por tu loader
out_npz = f"{ART}/kp_windows.npz"
np.savez_compressed(
    out_npz,
    kp=kp_windows,                  # [N,T,K,D]
    y=y_int,                        # [N]
    classes=np.array(label_names, dtype=object),
    mask=mask                       # [N,T,K] (opcional)
)
print(f"[OK] Guardado: {out_npz}")
print(f"     kp: {kp_windows.shape} | y: {y_int.shape} | classes: {len(label_names)} | mask: {mask.shape}")


In [ ]:

# Normalización de keypoints por ventana
#  - Carga kp si no existe en memoria
#  - Devuelve kp_norm [N,T,K,2] y mask_kp [N,T,K]
#  - Guarda  ART/kp_windows_norm.npz

import os, json
import numpy as np

# --- Rutas por defecto si no existen en el entorno ---
try:
    ART
except NameError:
    BASE = "/content/drive/MyDrive/asistente_coreografico"
    ART  = f"{BASE}/artifacts"
os.makedirs(ART, exist_ok=True)

# --- Loader mínimo, compatible con tus formatos previos ---
def _try_load_keypoints(art_dir):
    npz_path = f"{art_dir}/kp_windows.npz"
    if os.path.exists(npz_path):
        data = np.load(npz_path, allow_pickle=True)
        kp = data["kp"]                         # [N,T,K,2/3]
        y  = data["y"] if "y" in data else None
        mask = data["mask"] if "mask" in data else None
        classes = data["classes"].tolist() if "classes" in data else None
        return kp, y, mask, classes
    npy_path = f"{art_dir}/kp_windows.npy"
    if os.path.exists(npy_path):
        kp = np.load(npy_path, allow_pickle=True)
        y_path = f"{art_dir}/y_labels.npy"
        y = np.load(y_path) if os.path.exists(y_path) else None
        classes_path = f"{art_dir}/classes.json"
        classes = json.load(open(classes_path)) if os.path.exists(classes_path) else None
        return kp, y, None, classes
    raise FileNotFoundError("No encuentro ART/kp_windows.(npz|npy). Genera primero las ventanas.")

# --- Si kp no está en memoria, lo cargamos ---
try:
    kp  # variable ya existente
except NameError:
    kp, y, mask_file, classes = _try_load_keypoints(ART)
else:
    # si ya existía, ponemos None para no reusar máscara externa accidentalmente
    mask_file = None
    classes = None
    y = None

# --- Normalización: centra y escala robusto por ventana ---
# Índices opcionales (si los supieras); aquí usamos centrado/escala robustos
IDX_PELVIS = None
IDX_L_SHO  = None
IDX_L_HIP  = None

def compute_mask_and_xy(kp_arr):
    # kp_arr: [N,T,K,2] o [N,T,K,3]
    if kp_arr.shape[-1] == 3:
        xy   = kp_arr[..., :2]
        conf = kp_arr[..., 2]
        mask = (conf > 0.05).astype(np.float32)
    else:
        xy   = kp_arr
        mask = (~np.isnan(kp_arr[..., 0])).astype(np.float32)
    return xy.astype(np.float32), mask

def robust_center_scale(xy, mask):
    # xy: [N,T,K,2], mask: [N,T,K]
    N, T, K, _ = xy.shape
    out  = xy.copy()
    m_out = mask.copy()
    for i in range(N):
        for t in range(T):
            valid = m_out[i, t] > 0.5
            if not np.any(valid):
                continue
            pts = out[i, t, valid, :]
            center = np.median(pts, axis=0, keepdims=True)     # [1,2]
            out[i, t, :, :] -= center
            d = np.linalg.norm(pts - center, axis=1)
            scale = np.percentile(d, 90) if d.size > 0 else 1.0
            if scale < 1e-3:
                scale = 1.0
            out[i, t, :, :] /= scale
    return out, m_out

#  Extrae xy y máscara inferida
xy, mask_kp = compute_mask_and_xy(kp)

#  Si el archivo traía 'mask', se fuciona (AND lógico)
if 'mask_file' in locals() and mask_file is not None:
    mask_kp = (mask_kp * (mask_file > 0.5).astype(np.float32))

# Normaliza
kp_norm, mask_kp = robust_center_scale(xy, mask_kp)

print("[OK] Normalización completada")
print("kp_norm:", kp_norm.shape, "| mask_kp:", mask_kp.shape, "| D(norm)=", kp_norm.shape[-1])

# Guardar normalizados para etapas siguientes ---
save_norm = True
if save_norm:
    out = {"kp": kp_norm, "mask": mask_kp}
    if y is not None:
        out["y"] = y
    if classes is not None:
        out["classes"] = np.array(classes, dtype=object)
    out_path = f"{ART}/kp_windows_norm.npz"
    np.savez_compressed(out_path, **out)
    print(f"[OK] Guardado: {out_path}")


In [ ]:
def _align_features_df_any(df_like, cols):
    """DataFrame con columnas EXACTAS en 'cols' (orden), faltantes -> NaN."""
    if isinstance(df_like, pd.DataFrame):
        for c in cols:
            if c not in df_like.columns:
                df_like[c] = np.nan
        return df_like[cols]
    elif isinstance(df_like, (dict, pd.Series)):
        return pd.DataFrame([{c: df_like.get(c, np.nan) for c in cols}])[cols]
    elif isinstance(df_like, np.ndarray):
        # si viene ndarray con el nº correcto de columnas, lo envolvemos
        if df_like.ndim != 2:
            raise ValueError(f"Esperaba 2D array, got shape {df_like.shape}")
        if len(cols) != df_like.shape[1]:
            raise ValueError(f"Las dimensiones no coinciden: X.shape[1]={df_like.shape[1]} vs len(feature_cols)={len(cols)}")
        return pd.DataFrame(df_like, columns=cols)
    else:
        raise ValueError("Formato no soportado para df_like")

def try_load_teacher(ART, y, C, kp_norm=None, feature_cols=None):
    """
    Prioridad:
      1) ART/teacher_probas.npy
      2) Si hay imp+scaler+clf y features X_*.npy -> predict_proba
      3) Si NO hay X_*.npy: construir un DF con columnas=feature_cols y NaN (imputer rellena);
         opcionalmente añadir stats simples desde kp_norm si existe.
      4) Fallback final: label smoothing desde y (single-label o multi-label).
    """
    if feature_cols is None or len(feature_cols) == 0:
        raise RuntimeError("feature_cols no está disponible; carga ART/feature_cols.csv antes.")

    p_teacher_path = f"{ART}/teacher_probas.npy"
    if os.path.exists(p_teacher_path):
        P = np.load(p_teacher_path)
        print("[OK] Cargadas teacher_probas.npy:", P.shape)
        return P

    imp_path, scaler_path, clf_path = f"{ART}/imputer.joblib", f"{ART}/scaler.joblib", f"{ART}/model_ovr_logreg.joblib"
    has_pipeline = all(os.path.exists(p) for p in [imp_path, scaler_path, clf_path])

    if has_pipeline:
        imp    = joblib.load(imp_path)
        scaler = joblib.load(scaler_path)
        clf    = joblib.load(clf_path)

        # 2a) ¿Existen features precomputadas del profesor?
        for c in (f"{ART}/X_fusion.npy", f"{ART}/X_feats.npy", f"{ART}/features.npy"):
            if os.path.exists(c):
                X_raw = np.load(c)
                if X_raw.shape[1] != len(feature_cols):
                    print(f"[WARN] {os.path.basename(c)} tiene {X_raw.shape[1]} cols y el pipeline espera {len(feature_cols)}. Ignoro y genero alineado.")
                    break
                X_df = _align_features_df_any(X_raw, feature_cols)
                Ximp = imp.transform(X_df); Xs = scaler.transform(Ximp)
                P = clf.predict_proba(Xs).astype(np.float32)
                np.save(p_teacher_path, P)
                print("[OK] Teacher probas generadas desde features guardadas:", P.shape)
                return P
        # 2b) No hay X_*.npy válidas: construir DF vacío alineado y (opcional) inyectar stats simples
        N_guess = None
        feats_dict = {col: np.nan for col in feature_cols}  # plantilla por fila
        rows = []

        if kp_norm is not None:
            # kp_norm: [N,T,K,2] -> N filas
            N_guess = kp_norm.shape[0]
            # estadísticas simples opcionales (si te suenan nombres en feature_cols, añade aquí los cálculos)
            # p.ej., si existe una columna llamada "mean_x", "mean_y", etc., podrías mapear algo:
            # (dejamos NaN por defecto para que el imputer rellene)
            for i in range(N_guess):
                rows.append(feats_dict.copy())
        else:
            # si no tenemos kp_norm, al menos intenta deducir N de y
            if y is not None:
                N_guess = len(y)
                rows = [feats_dict.copy() for _ in range(N_guess)]
            else:
                raise RuntimeError("No hay features del profesor ni kp_norm ni y para construir P_teacher.")

        X_df = pd.DataFrame(rows, columns=feature_cols)
        Ximp = imp.transform(X_df); Xs = scaler.transform(Ximp)
        P = clf.predict_proba(Xs).astype(np.float32)
        np.save(p_teacher_path, P)
        print("[OK] Teacher probas generadas desde DF alineado con imputer:", P.shape)
        return P

    # 3) Fallback final: label smoothing a partir de y
    if y is None:
        raise RuntimeError("No hay teacher_probas, ni pipeline clásico, ni y para fallback.")
    y_arr = np.asarray(y)
    if y_arr.ndim == 1:  # single-label (clase entera por muestra)
        C_fbk = int(C) if C is not None else int(np.max(y_arr) + 1)
        eps = 0.1
        P = np.full((len(y_arr), C_fbk), eps/(C_fbk-1), dtype=np.float32)
        P[np.arange(len(y_arr)), y_arr.astype(int)] = 1.0 - eps
    else:  # multilabel (N,C)
        eps = 0.1
        P = (1.0 - eps) * y_arr.astype(np.float32) + eps * (1.0 - y_arr.astype(np.float32)) / max(1, y_arr.shape[1]-1)
    print("[OK] Teacher probas fallback (label smoothing):", P.shape)
    return P

# ==== USO ====
# Requiere: ART, feature_cols, y (opcional), C (opcional), kp_norm (opcional)
P_teacher = try_load_teacher(ART, y, C, kp_norm=kp_norm, feature_cols=feature_cols)

# Alinear N con kp_norm si difiere
N_pt, N_kp = P_teacher.shape[0], kp_norm.shape[0]
if N_pt != N_kp:
    N_min = min(N_pt, N_kp)
    print(f"[WARN] N mismatch entre teacher ({N_pt}) y kp_norm ({N_kp}); recorto a {N_min}.")
    P_teacher = P_teacher[:N_min]
    kp_norm   = kp_norm[:N_min]
    if isinstance(y, np.ndarray):
        y = y[:N_min]

C = P_teacher.shape[1]
print("Clases (C):", C)

In [ ]:
# Dataset y splits
# Entrada estudiante: secuencia [T, K*2] (aplanamos K,2)

from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit

#  Inferir N, T, K a partir de kp_norm
assert 'kp_norm' in globals(), "kp_norm no está en memoria."
assert kp_norm.ndim == 4, f"kp_norm debe ser [N,T,K,D], recibido {kp_norm.shape}"

N, T, K, D = kp_norm.shape
if D >= 2:
    # nos quedamos con (x,y) si viniera [x,y,(z/conf)]
    kp_xy = kp_norm[..., :2]
else:
    raise ValueError(f"Última dimensión de kp_norm debe ser ≥2 (x,y). Recibido D={D}")

# Construir X_seq = [N, T, K*2]
X_seq = kp_xy.reshape(N, T, K * 2).astype(np.float32)

# Alinear longitudes con P_teacher si hiciera falta
assert 'P_teacher' in globals(), "P_teacher no está en memoria; genera/ carga las probabilidades del profesor."
if len(P_teacher) != N:
    n_min = min(N, len(P_teacher))
    print(f"[WARN] N mismatch (kp_norm={N}, P_teacher={len(P_teacher)}). Recorto a {n_min}.")
    X_seq = X_seq[:n_min]
    kp_xy = kp_xy[:n_min]
    P_teacher = P_teacher[:n_min]
    if 'y' in globals() and y is not None:
        y = y[:n_min]
    N = n_min  # actualizar

# Etiqueta dura para estratificación
if 'y' in globals() and y is not None:
    y_arr = np.asarray(y)
    if y_arr.ndim == 2:        # multilabel -> tomamos argmax como proxy
        y_hard = np.argmax(y_arr, axis=1).astype(int)
    else:                      # single label
        y_hard = y_arr.astype(int)
else:
    y_hard = np.argmax(P_teacher, axis=1).astype(int)

# Elegir n_splits seguro (cada clase debe tener ≥ n_splits muestras)
SEED = globals().get('SEED', 42)
counts = np.bincount(y_hard)
min_per_class = counts[counts > 0].min() if np.any(counts > 0) else 0
n_splits = min(5, int(min_per_class)) if min_per_class >= 2 else 2

# Si solo hay una clase o no hay suficientes muestras por clase, usar ShuffleSplit estratificado
if len(np.unique(y_hard)) < 2 or min_per_class < 2:
    print("[INFO] Estratificación no viable (clases insuficientes). Uso StratifiedShuffleSplit con test_size=0.2.")
    sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=SEED)
    tr_idx, va_idx = next(sss.split(X_seq, y_hard))
else:
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=SEED)
    tr_idx, va_idx = next(skf.split(X_seq, y_hard))

# Construir splits
X_tr, X_va = X_seq[tr_idx], X_seq[va_idx]
y_tr, y_va = y_hard[tr_idx], y_hard[va_idx]
P_tr, P_va = P_teacher[tr_idx], P_teacher[va_idx]

print("X_tr:", X_tr.shape, "| X_va:", X_va.shape, "| P_tr:", P_tr.shape)

In [ ]:
# =========================================================
# Modelo estudiante: BiGRU + MLP  (KD = alpha*KL + (1-alpha)*CE)
# - Entrada: [B, T, D], D = K*2
# - Salida : [B, C]
# - Alinea clases: etiquetas y teacher_probs coherentes
# =========================================================
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from inspect import signature
from torch.utils.data import TensorDataset, DataLoader

# ---------- Comprobaciones previas ----------
need = ['X_tr','X_va','y_tr','y_va','P_tr','P_va']
missing = [n for n in need if n not in globals()]
if missing:
    raise NameError(f"Faltan variables previas {missing}. Ejecuta antes el paso de split/estratificación.")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

# ---------- Alineación de clases (evita 'Target out of bounds') ----------
def normalize_labels(y):
    y = np.asarray(y)
    if y.ndim != 1:
        # si llega one-hot / multilabel, tomar argmax como etiqueta dura
        y = np.argmax(y, axis=1)
    y = y.astype(np.int64)
    uniq = np.unique(y)
    # si las clases empiezan en 1 (o >0), desplaza a 0..C-1
    if 0 not in uniq:
        y = y - uniq.min()
    return y

def pad_probs(P, C_eff):
    P = np.asarray(P, dtype=np.float32)
    if P.shape[1] == C_eff:
        return P
    P_pad = np.zeros((P.shape[0], C_eff), dtype=np.float32)
    P_pad[:, :P.shape[1]] = P
    return P_pad

y_tr = normalize_labels(y_tr)
y_va = normalize_labels(y_va)

# Número de clases efectivo: que quepan todas las etiquetas y las columnas de P
C_eff = int(max(P_tr.shape[1], P_va.shape[1], y_tr.max()+1, y_va.max()+1))
P_tr = pad_probs(P_tr, C_eff)
P_va = pad_probs(P_va, C_eff)

# D_in y C desde los splits (evita depender de T/K globales)
D_in = int(np.prod(X_tr.shape[2:]))  # asume X_tr shape [N, T, D]
C    = C_eff

# MODELS_DIR por si no existe en el entorno
MODELS_DIR = globals().get("MODELS_DIR", "./models")
os.makedirs(MODELS_DIR, exist_ok=True)

# ---------- Modelo ----------
class StudentBiGRU(nn.Module):
    def __init__(self, d_in, hidden=192, layers=1, num_classes=8, dropout=0.15):
        super().__init__()
        self.gru = nn.GRU(
            input_size=d_in, hidden_size=hidden, num_layers=layers,
            batch_first=True, bidirectional=True,
            dropout=0.0 if layers == 1 else dropout
        )
        self.head = nn.Sequential(
            nn.Linear(hidden*2, hidden),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden, num_classes)
        )
    def forward(self, x):          # x: [B,T,D]
        out, _ = self.gru(x)       # [B,T,2H]
        h = out.mean(dim=1)        # average pooling temporal
        return self.head(h)        # [B,C]

model = StudentBiGRU(d_in=D_in, hidden=192, layers=1, num_classes=C, dropout=0.15).to(device)

# ---------- DataLoaders ----------
BATCH = 128

def make_loader(X, y, P, train=True):
    X_t = torch.tensor(X, dtype=torch.float32)
    y_t = torch.tensor(y, dtype=torch.long)
    P_t = torch.tensor(P, dtype=torch.float32)
    ds  = TensorDataset(X_t, y_t, P_t)
    return DataLoader(ds, batch_size=BATCH, shuffle=train, drop_last=False)

dl_tr = make_loader(X_tr, y_tr, P_tr, train=True)
dl_va = make_loader(X_va, y_va, P_va, train=False)

# ---------- Pérdidas y Optimizador ----------
ce     = nn.CrossEntropyLoss()
kl     = nn.KLDivLoss(reduction="batchmean")
T_temp = 2.5
alpha  = 0.7

opt   = optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)

# Scheduler ReduceLROnPlateau compatible con versiones sin 'verbose'
def make_plateau_scheduler(optimizer, mode="min", factor=0.5, patience=3, min_lr=0.0, threshold=1e-4, cooldown=0):
    params = signature(optim.lr_scheduler.ReduceLROnPlateau.__init__).parameters
    if 'verbose' in params:
        return optim.lr_scheduler.ReduceLROnPlateau(
            optimizer, mode=mode, factor=factor, patience=patience,
            threshold=threshold, cooldown=cooldown, min_lr=min_lr, verbose=True
        )
    else:
        return optim.lr_scheduler.ReduceLROnPlateau(
            optimizer, mode=mode, factor=factor, patience=patience,
            threshold=threshold, cooldown=cooldown, min_lr=min_lr
        )

sched = make_plateau_scheduler(opt, mode="min", factor=0.5, patience=3)

# ---------- Utilidades KD ----------
def _row_normalize(p, eps=1e-8):
    """Normaliza por fila para que la KL tenga un target válido (suma=1)."""
    p = p.clone()
    s = p.sum(dim=1, keepdim=True)
    p = p / (s + eps)
    mask_zero = (s.squeeze(1) < eps)
    if mask_zero.any():
        p[mask_zero] = 1.0 / p.shape[1]
    return p

# ---------- Train / Eval ----------
def train_one_epoch(model, dl, optimizer, device):
    model.train()
    total_loss = total_ce = total_kd = total_n = 0.0
    for Xb, yb, Pb in dl:
        Xb, yb, Pb = Xb.to(device), yb.to(device), Pb.to(device)
        Pb_soft = _row_normalize(Pb)

        optimizer.zero_grad()
        logits = model(Xb)                              # [B,C]
        loss_ce = ce(logits, yb)
        log_qT  = F.log_softmax(logits / T_temp, dim=1)
        loss_kd = kl(log_qT, Pb_soft) * (T_temp ** 2)

        loss = alpha * loss_kd + (1.0 - alpha) * loss_ce
        loss.backward()
        optimizer.step()

        bsz = Xb.size(0)
        total_loss += loss.item()    * bsz
        total_ce   += loss_ce.item() * bsz
        total_kd   += loss_kd.item() * bsz
        total_n    += bsz

    return total_loss/max(1,total_n), total_ce/max(1,total_n), total_kd/max(1,total_n)

@torch.no_grad()
def evaluate(model, dl, device):
    model.eval()
    total_loss = total_ce = total_kd = total_n = 0.0
    correct = 0
    for Xb, yb, Pb in dl:
        Xb, yb, Pb = Xb.to(device), yb.to(device), Pb.to(device)
        Pb_soft = _row_normalize(Pb)

        logits = model(Xb)
        loss_ce = ce(logits, yb)
        log_qT  = F.log_softmax(logits / T_temp, dim=1)
        loss_kd = kl(log_qT, Pb_soft) * (T_temp ** 2)
        loss = alpha * loss_kd + (1.0 - alpha) * loss_ce

        bsz = Xb.size(0)
        total_loss += loss.item()    * bsz
        total_ce   += loss_ce.item() * bsz
        total_kd   += loss_kd.item() * bsz
        total_n    += bsz

        preds = logits.argmax(dim=1)
        correct += (preds == yb).sum().item()

    acc = correct / max(1,total_n)
    return total_loss/max(1,total_n), total_ce/max(1,total_n), total_kd/max(1,total_n), acc

# ---------- Loop principal ----------
EPOCHS = 25
best_va = float('inf')
best_path = os.path.join(MODELS_DIR, "student_bigru_best.pt")

for epoch in range(1, EPOCHS+1):
    tr_loss, tr_ce, tr_kd = train_one_epoch(model, dl_tr, opt, device)
    va_loss, va_ce, va_kd, va_acc = evaluate(model, dl_va, device)

    # Reducir LR según validación
    sched.step(va_loss)
    current_lr = opt.param_groups[0]['lr']

    print(f"Epoch {epoch:02d} | "
          f"TR {tr_loss:.4f} (CE {tr_ce:.4f} | KD {tr_kd:.4f}) | "
          f"VA {va_loss:.4f} (CE {va_ce:.4f} | KD {va_kd:.4f}) | "
          f"ACC {va_acc:.3f} | LR {current_lr:.2e}")

    # Guardar mejor checkpoint por VA loss
    if va_loss < best_va:
        best_va = va_loss
        torch.save(model.state_dict(), best_path)
        print("  ↳ guardado best checkpoint:", best_path)


In [ ]:
# ============================================
# Inferencia 1 ventana con StudentBiGRU (robusto)
# - Autodetecta rutas, clases y dimensiones
# - Sin dependencias de ART, C, kp globales
# ============================================
import os, json
from pathlib import Path
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

# ---------- Rutas del proyecto ----------
BASE = globals().get("BASE", "/content/drive/MyDrive/asistente_coreografico")
ART  = globals().get("ART",  f"{BASE}/artifacts")
MOD  = globals().get("MOD",  f"{BASE}/models")
os.makedirs(ART, exist_ok=True)
os.makedirs(MOD, exist_ok=True)

# ---------- Modelo ----------
class StudentBiGRU(nn.Module):
    def __init__(self, d_in, num_classes, hidden=192, layers=1, dropout=0.15):
        super().__init__()
        self.gru = nn.GRU(
            input_size=d_in, hidden_size=hidden, num_layers=layers,
            batch_first=True, bidirectional=True,
            dropout=0.0 if layers == 1 else dropout
        )
        self.head = nn.Sequential(
            nn.Linear(hidden*2, hidden),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden, num_classes)
        )
    def forward(self, x):                 # x: [B,T,D]
        y, _ = self.gru(x)                # [B,T,2H]
        h = y.mean(dim=1)                 # media temporal
        return self.head(h)               # [B,C]

# ---------- Normalización de una ventana ----------
def normalize_one_window(kp_window: np.ndarray) -> np.ndarray:
    xy = kp_window[..., :2].astype(np.float32)
    if kp_window.shape[-1] == 3:
        conf = kp_window[..., 2]
        mask = (conf > 0.05).astype(np.float32)
    else:
        mask = (~np.isnan(xy[..., 0])).astype(np.float32)
    pts = xy[mask > 0.5].reshape(-1, 2)
    if len(pts) == 0:
        return np.zeros_like(xy, dtype=np.float32)
    center = np.median(pts, axis=0, keepdims=True)
    d = np.linalg.norm(pts - center, axis=1)
    scale = np.percentile(d, 90) if len(d) else 1.0
    if scale < 1e-3: scale = 1.0
    return (xy - center) / scale

# ---------- Localizar checkpoint del estudiante ----------
def find_student_checkpoint() -> str:
    # 1) variable global
    path = globals().get("STUDENT_PT", f"{MOD}/student_kp.pt")
    if os.path.exists(path):
        return path
    # 2) fallback típico
    alt = f"{MOD}/student_bigru_best.pt"
    if os.path.exists(alt):
        print(f"[WARN] No existe {path}. Uso alternativo: {alt}")
        return alt
    # 3) el más reciente con patrón razonable
    cands = sorted(Path(MOD).glob("*.pt"), key=lambda p: p.stat().st_mtime, reverse=True)
    for p in cands:
        if "student" in p.name or "bigru" in p.name:
            print(f"[WARN] Uso checkpoint más reciente: {p}")


In [ ]:
# ============================================
# - Validación + Export
# - Alinea versiones sklearn
# - Carga dataset/artefactos del proyecto
# - Corre inferencia y genera sugerencias
# - Reexporta artefactos consistentes para la app (/artifacts)
# - Emite timeline.csv y suggestions.csv
# ============================================

# ---------- 0) Dependencias y versiones ----------
import sys, os, json, time, glob, math, warnings, textwrap
from pathlib import Path

# Fija versiones coherentes con tus artefactos entrenados
# (ajusta si fuera necesario en tu entorno de Colab)
!pip -q install "scikit-learn==1.7.1" "joblib==1.4.2" "numpy==1.26.4" "pandas==2.2.2"

import numpy as np
import pandas as pd
import joblib
from sklearn.pipeline import Pipeline as SkPipeline

warnings.filterwarnings("ignore")

# ---------- 1) Rutas del proyecto ----------
BASE = "/content/drive/MyDrive/asistente_coreografico"  # <- AJUSTA si usas otra carpeta
ART  = f"{BASE}/artifacts"
MOD  = f"{BASE}/models"
DATA = f"{BASE}/data"

os.makedirs(ART, exist_ok=True)
os.makedirs(MOD, exist_ok=True)
os.makedirs(f"{BASE}/reports", exist_ok=True)

print(f"[RUTAS]\nBASE={BASE}\nART={ART}\nMOD={MOD}\nDATA={DATA}")

# ---------- 2) Localización de dataset / features ----------
# Intentamos detectar características usadas en el entrenamiento anterior.
# Soporta múltiples formatos (npz, parquet, csv) para ser robusto.
def find_first(*patterns):
    for pat in patterns:
        matches = sorted(glob.glob(pat))
        if matches:
            return matches[0]
    return None

FEATS_NPZ   = find_first(f"{DATA}/processed/*feats*.npz", f"{DATA}/*feats*.npz")
FEATS_NPY   = find_first(f"{DATA}/processed/*feats*.npy", f"{DATA}/*feats*.npy")
FEATS_PARQ  = find_first(f"{DATA}/processed/*feats*.parquet")
FEATS_CSV   = find_first(f"{DATA}/processed/*feats*.csv")
WINS_JSON   = find_first(f"{DATA}/processed/*windows*.json", f"{DATA}/*windows*.json")
WINS_CSV    = find_first(f"{DATA}/processed/*windows*.csv", f"{DATA}/*windows*.csv")

print("[DATASET DETECTADO]")
print("  FEATS_NPZ :", FEATS_NPZ)
print("  FEATS_NPY :", FEATS_NPY)
print("  FEATS_PARQ:", FEATS_PARQ)
print("  FEATS_CSV :", FEATS_CSV)
print("  WINS_JSON :", WINS_JSON)
print("  WINS_CSV  :", WINS_CSV)

def load_features_and_windows():
    # --- Carga features ---
    X = None
    if FEATS_NPZ:
        arr = np.load(FEATS_NPZ, allow_pickle=True)
        X = arr["X"] if "X" in arr else list(arr.values())[0]
    elif FEATS_NPY:
        X = np.load(FEATS_NPY, allow_pickle=True)
    elif FEATS_PARQ:
        df = pd.read_parquet(FEATS_PARQ)
        X = df.values
    elif FEATS_CSV:
        df = pd.read_csv(FEATS_CSV)
        X = df.values
    else:
        raise FileNotFoundError("No se encontraron features (npz/npy/parquet/csv).")

    # --- Carga ventanas ---
    if WINS_JSON:
        with open(WINS_JSON, "r", encoding="utf-8") as f:
            wins = json.load(f)
    elif WINS_CSV:
        wdf = pd.read_csv(WINS_CSV)
        # Normalización esperada: columnas t0, t1
        wins = wdf.to_dict(orient="records")
    else:
        # Si no hay "windows" guardadas, creamos ventanas sintéticas 1s seguidas.
        wins = [{"t0": float(i), "t1": float(i+1)} for i in range(len(X))]

    # Aseguramos listas/np arrays
    X = np.array(X)
    if isinstance(wins, dict):
        wins = [wins]
    return X, wins

X, windows = load_features_and_windows()
print(f"[OK] X shape = {X.shape} | windows = {len(windows)}")

# ---------- 3) Carga de artefactos de entrenamiento ----------
# Esperamos estos ficheros (ajústalos si tus nombres cambian):
IMP_PATH   = f"{ART}/imputer.joblib"
SCL_PATH   = f"{ART}/scaler.joblib"
CLF_PATH   = f"{ART}/model_ovr_logreg.joblib"  # o el que tengas
LBL_PATH   = f"{ART}/labels.json"              # lista de etiquetas
THR_PATH   = f"{ART}/thresholds.json"          # map etiqueta->umbral (float)

assert os.path.exists(IMP_PATH), f"Falta: {IMP_PATH}"
assert os.path.exists(SCL_PATH), f"Falta: {SCL_PATH}"
assert os.path.exists(CLF_PATH), f"Falta: {CLF_PATH}"

imp    = joblib.load(IMP_PATH)
scaler = joblib.load(SCL_PATH)
clf    = joblib.load(CLF_PATH)

labels_list = []
if os.path.exists(LBL_PATH):
    with open(LBL_PATH, "r", encoding="utf-8") as f:
        labels_list = json.load(f)

thr_map = {}
if os.path.exists(THR_PATH):
    with open(THR_PATH, "r", encoding="utf-8") as f:
        thr_map = json.load(f)

print(f"[ARTEFACTOS] labels={len(labels_list)} | thresholds={len(thr_map)}")

# ---------- 4) Clasificación y umbrales ----------
# pred_prob -> aplica umbrales por etiqueta si existen; si no, usa 'thr_default'
def predict_labels_batch(X_batch, thr_default=0.5):
    # Pipeline simple y robusto con imputer + scaler + clf
    X_imp = imp.transform(X_batch)
    X_std = scaler.transform(X_imp)

    # Probabilidades por clase (OneVsRestClassifier / LogisticRegressionCV)
    if hasattr(clf, "predict_proba"):
        P = clf.predict_proba(X_std)
    else:
        # Fallback a decision_function si no hay predict_proba
        if hasattr(clf, "decision_function"):
            dec = clf.decision_function(X_std)
            # Sigmoide para pseudo-probas
            P = 1.0 / (1.0 + np.exp(-dec))
        else:
            raise RuntimeError("El clasificador no soporta predict_proba ni decision_function.")

    # P shape: (N, C)
    N, C = P.shape
    out_labels = []
    out_scores = []
    for i in range(N):
        labels_i, scores_i = [], []
        for c in range(C):
            label_name = labels_list[c] if c < len(labels_list) else f"class_{c}"
            thr = float(thr_map.get(label_name, thr_default)) if thr_map else thr_default
            if P[i, c] >= thr:
                labels_i.append(label_name)
                scores_i.append(float(P[i, c]))
        out_labels.append(labels_i)
        out_scores.append(scores_i)
    return out_labels, out_scores

# ---------- 5) Motor de sugerencias ----------
# Define aquí el mismo mapping que usas en el Colab/app
SUGGESTIONS_MAP = {
    # "etiqueta": "texto de sugerencia"
    "alineacion_brazos": "Ajusta la alineación de codos y muñecas en ángulos complementarios al torso para ganar limpieza.",
    "centro_gravedad": "Rebaja el centro de gravedad al inicio del giro y estabiliza el core antes del impulso.",
    "transferencia_peso": "Marca un micro-plié al transferir peso para amortiguar y mantener el eje.",
    "mirada_foco": "Define un foco visual claro antes del desplazamiento para orientar direccionalidad.",
    "sincronizacion_grupo": "Escucha el phrasing común y ancla entradas en los tiempos fuertes para cohesión.",
    "extensiones_linea": "Extiende dedos y empeines para completar la línea hasta el extremo distal.",
    "dinamica_contrastes": "Introduce contraste de dinámicas (sostenido vs. acento) para clarificar intención.",
    "respiracion_fraseo": "Apoya la musicalidad con respiración en anacrusas y cierres para articular el fraseo.",
    "espacial_path": "Optimiza el path espacial evitando zigzags no intencionales; usa diagonales limpias.",
    "aterrizaje_silencioso": "Aterriza con metatarso-suelo progresivo para minimizar ruido y proteger articulaciones.",
}

def suggestions_from_labels(labels):
    sugs = []
    for lb in labels:
        if lb in SUGGESTIONS_MAP:
            sugs.append(SUGGESTIONS_MAP[lb])
    # Si no hubo match exacto, ofrece un comodín si te interesa:
    if not sugs and labels:
        sugs.append("Refina el eje postural y verifica la alineación cabeza-hombros-cadera en la transición.")
    return sugs

# ---------- 6) Inferencia sobre el dataset y export ----------
def run_full_inference_and_export(X, windows, thr_default=0.5, out_dir=BASE):
    t0 = time.time()
    out_rows = []
    Y_labels, Y_scores = predict_labels_batch(X, thr_default=thr_default)

    for i, (lbls, scs) in enumerate(zip(Y_labels, Y_scores)):
        w = windows[i] if i < len(windows) else {}
        t_start = float(w.get("t0", i*1.0))
        t_end   = float(w.get("t1", (i+1)*1.0))
        sug = suggestions_from_labels(lbls)

        out_rows.append({
            "win_id": i,
            "t_start": t_start,
            "t_end": t_end,
            "labels": ",".join(lbls),
            "scores": ",".join([f"{s:.4f}" for s in scs]),
            "suggestions": " | ".join(sug)
        })

    df = pd.DataFrame(out_rows)
    timeline_csv   = f"{out_dir}/reports/timeline.csv"
    suggs_csv      = f"{out_dir}/reports/suggestions.csv"
    df.to_csv(timeline_csv, index=False, encoding="utf-8")
    df[["win_id","t_start","t_end","suggestions"]].to_csv(suggs_csv, index=False, encoding="utf-8")

    # Exporta un "bundle" de control de versiones para la app:
    versions = {
        "scikit_learn": "1.7.1",
        "joblib": "1.4.2",
        "numpy": "1.26.4",
        "pandas": "2.2.2",
        "created_at": time.strftime("%Y-%m-%d %H:%M:%S"),
    }
    with open(f"{ART}/versions.json", "w", encoding="utf-8") as f:
        json.dump(versions, f, ensure_ascii=False, indent=2)

    # Garantiza labels.json y thresholds.json para la app
    if labels_list:
        with open(f"{ART}/labels.json", "w", encoding="utf-8") as f:
            json.dump(labels_list, f, ensure_ascii=False, indent=2)
    if thr_map:
        with open(f"{ART}/thresholds.json", "w", encoding="utf-8") as f:
            json.dump(thr_map, f, ensure_ascii=False, indent=2)

    dt = time.time() - t0
    print(f"[OK] Exportados:\n  {timeline_csv}\n  {suggs_csv}\nTiempo: {dt:.2f}s | filas={len(df)}")
    return df

df = run_full_inference_and_export(X, windows, thr_default=0.5, out_dir=BASE)
display(df.head(10))
